# **ECR - Engenharia**

### Importação de bibliotecas


In [1]:
import pandas as pd
import numpy as np
import re
import os
import pytesseract
from PIL import Image
from datetime import datetime
import requests
import json
from tabulate import tabulate
from openpyxl import Workbook
import time 
import io 



#### **Chaves de acesso**

In [2]:
APP_KEY = "596450736882"
APP_SECRET = "37a10edb420d8cd6c7afddcaaffcac05"

#### Enetendimento da Planilhas (Colunas/Linhas)

In [3]:
arquivo = 'APIS OMIE ECR.xlsx'

# Lista todas as abas da planilha
xls = pd.ExcelFile(arquivo)
print("Abas encontradas:", xls.sheet_names)

# Carrega a primeira aba (ou troque pelo nome desejado)
df = pd.read_excel(xls, sheet_name=0)

# Mostra as primeiras linhas para entender os dados
print("\nPrimeiras linhas:")
print(df.head())

# Informações gerais
print("\nInformações do DataFrame:")
print(df.info())

# Colunas disponíveis
print("\nColunas disponíveis:")
print(df.columns)

Abas encontradas: ['Planilha1']

Primeiras linhas:
                                               Geral  \
0  Alguns cadastros no Omie são compartilhados po...   
1       Clientes, Fornecedores, Transportadoras, etc   
2                         Clientes - Características   
3                                               Tags   
4                                           Projetos   

                                          Unnamed: 1 Unnamed: 2  \
0                                                NaN        NaN   
1  Cria/edita/consulta o cadastro de clientes, fo...         v1   
2    Cria/edita/consulta características de clientes         v1   
3  Cria/edita/consulta tags quem são usadas no ca...         v1   
4                       Cria/edita/consulta projetos         v1   

   DADO UTILIZADO NO GERENCIAL? Unnamed: 4  
0                           NaN        NaN  
1                           NaN         OK  
2                           NaN         OK  
3                           N

#### Filtro de endpoints

In [4]:
df.columns = ['Geral', 'Descricao', 'Versao', 'UsadoGerencial', 'Status']

# Filtrar linhas que possuem descrição e versão preenchidas
filtro_endpoints = df[ df['Versao'].notna()]

# Exibir os primeiros endpoints filtrados
filtro_endpoints.reset_index(drop=True)

,Geral,Descricao,Versao,UsadoGerencial,Status
0,"Clientes, Fornecedores, Transportadoras, etc","Cria/edita/consulta o cadastro de clientes, fo...",v1,NaN,OK
1,Clientes - Características,Cria/edita/consulta características de clientes,v1,NaN,OK
2,Tags,Cria/edita/consulta tags quem são usadas no ca...,v1,NaN,OK
3,Projetos,Cria/edita/consulta projetos,v1,NaN,OK
4,Empresas,Lista o cadastro da empresa,v1,NaN,OK
...,...,...,...,...,...
138,Etapas de Faturamento,Lista as etapas do faturamento,v1,NaN,OK
139,Tipo de utilização,Lista os Tipos de utilização,v1,NaN,OK
140,Classificação do Serviço,Lista as Classificações do Serviço,v1,NaN,OK
141,Documentos Fiscais,Listagem dos XMLs de Documentos Fiscais (NF-e/...,v1,NaN,OK


#### **Extração teste**

In [5]:
endpoint = "clientes"
versao = "v1"

base_url = f"https://app.omie.com.br/api/v1/geral/{endpoint}/"


headers = {
    'Content-Type': 'application/json'
}


body = {
    'call': 'ListarClientes',
    'app_key': APP_KEY,
    'app_secret': APP_SECRET,
    'param': [
        {
            "pagina": 1,
            "registros_por_pagina": 50
        }
    ]
}


response = requests.post(base_url, headers=headers, data=json.dumps(body))

if response.status_code == 200:
    print("Dados extraídos com sucesso!")
    print(json.dumps(response.json(), indent=2))
else:
    print(f"Erro na requisição: {response.status_code}")
    print(response.text)


Dados extraídos com sucesso!
{
  "pagina": 1,
  "total_de_paginas": 60,
  "registros": 50,
  "total_de_registros": 2996,
  "clientes_cadastro": [
    {
      "bairro": "",
      "bloquear_faturamento": "N",
      "cep": "",
      "cidade": "",
      "cidade_ibge": "",
      "cnpj_cpf": "07.575.651/0001-59",
      "codigo_cliente_integracao": "",
      "codigo_cliente_omie": 679084426,
      "codigo_pais": "1058",
      "complemento": "",
      "dadosBancarios": {
        "agencia": "",
        "cChavePix": "",
        "codigo_banco": "",
        "conta_corrente": "",
        "doc_titular": "",
        "nome_titular": "",
        "transf_padrao": "N"
      },
      "endereco": "",
      "enderecoEntrega": {},
      "endereco_numero": "",
      "enviar_anexos": "N",
      "estado": "",
      "exterior": "N",
      "inativo": "N",
      "info": {
        "cImpAPI": "N",
        "dAlt": "07/01/2019",
        "dInc": "07/01/2019",
        "hAlt": "12:09:27",
        "hInc": "12:09:27",
    

#### **Extração Clientes**

In [6]:

APP_KEY = "596450736882"
APP_SECRET = "37a10edb420d8cd6c7afddcaaffcac05"
endpoint = "clientes"
versao = "v1"

base_url = f"https://app.omie.com.br/api/{versao}/geral/{endpoint}/"

headers = {
    'Content-Type': 'application/json'
}

# Parâmetros iniciais para a primeira chamada (para descobrir o total de páginas)
pagina_atual = 1
registros_por_pagina = 50 # Você pode ajustar isso, mas 50 é um bom padrão
max_tentativas_por_pagina = 3 # Número máximo de tentativas para cada página
delay_entre_tentativas_base = 5 # Segundos para esperar antes da primeira tentativa (aumenta exponencialmente)


print(f"Iniciando extração de clientes da API Omie. Registros por página: {registros_por_pagina}")

# Lista para armazenar todos os clientes de todas as páginas
todos_os_clientes_lista = []
total_de_paginas_api = 1 # Inicializa com 1, será atualizado após a primeira chamada

try:
    while pagina_atual <= total_de_paginas_api:
        body = {
            'call': 'ListarClientes',
            'app_key': APP_KEY,
            'app_secret': APP_SECRET,
            'param': [
                {
                    "pagina": pagina_atual,
                    "registros_por_pagina": registros_por_pagina
                }
            ]
        }

        tentativa_atual = 0
        sucesso_pagina = False
        while tentativa_atual < max_tentativas_por_pagina and not sucesso_pagina:
            tentativa_atual += 1
            print(f"Buscando página {pagina_atual} de {total_de_paginas_api if total_de_paginas_api > 1 else '...' } (Tentativa {tentativa_atual}/{max_tentativas_por_pagina})...")
            
            try:
                response = requests.post(base_url, headers=headers, data=json.dumps(body), timeout=30) # Adicionado timeout de 30s
                response.raise_for_status() # Levanta um erro para status HTTP 4xx/5xx
                
                dados_api_pagina = response.json()
                sucesso_pagina = True # Marca que a página foi buscada com sucesso

                if pagina_atual == 1 and tentativa_atual == 1: # Na primeira chamada bem sucedida da primeira página
                    total_de_paginas_api = dados_api_pagina.get('total_de_paginas', 1)
                    total_de_registros_api = dados_api_pagina.get('total_de_registros', 0)
                    print(f"API informou: {total_de_paginas_api} página(s) e {total_de_registros_api} registro(s) no total.")
                    if total_de_registros_api == 0:
                        print("Nenhum registro encontrado na API.")
                        # Define total_de_paginas_api para 0 ou pagina_atual para sair do loop principal
                        total_de_paginas_api = 0 
                        break # Sai do loop de tentativas, e o loop principal também sairá

                if sucesso_pagina: # Procede apenas se a requisição foi bem sucedida
                    clientes_da_pagina = dados_api_pagina.get('clientes_cadastro', [])
                    if clientes_da_pagina:
                        todos_os_clientes_lista.extend(clientes_da_pagina)
                        print(f"  {len(clientes_da_pagina)} clientes adicionados da página {pagina_atual}. Total acumulado: {len(todos_os_clientes_lista)}")
                    else:
                        print(f"  Nenhum cliente encontrado na página {pagina_atual} (resposta da API).")
                
            except (requests.exceptions.ConnectionError, requests.exceptions.Timeout, requests.exceptions.HTTPError) as e:
                print(f"  Erro ao buscar página {pagina_atual} (tentativa {tentativa_atual}): {e}")
                if tentativa_atual < max_tentativas_por_pagina:
                    delay_atual = delay_entre_tentativas_base * (2 ** (tentativa_atual - 1)) # Backoff exponencial
                    print(f"  Aguardando {delay_atual} segundos antes de tentar novamente...")
                    time.sleep(delay_atual)
                else:
                    print(f"  Máximo de {max_tentativas_por_pagina} tentativas atingido para a página {pagina_atual}. Desistindo desta página.")
                    # Você pode decidir parar todo o script ou pular para a próxima página
                    # Para parar tudo: raise Exception(f"Falha ao buscar página {pagina_atual} após {max_tentativas_por_pagina} tentativas.")
                    break # Sai do loop de tentativas para esta página

        if not sucesso_pagina and total_de_paginas_api > 0 : # Se não conseguiu buscar a página e não é o caso de 0 registros
             print(f"AVISO: Falha ao processar a página {pagina_atual} após todas as tentativas. Alguns dados podem estar faltando.")
             # Decide se quer continuar para a próxima página ou parar. Por padrão, continua.

        if pagina_atual < total_de_paginas_api and total_de_paginas_api > 0:
            time.sleep(1) # Aumentado o delay padrão entre páginas para 1 segundo
        
        if total_de_paginas_api == 0: # Se a API informou 0 registros na primeira página
            break

        pagina_atual += 1

    print("\nExtração de todas as páginas concluída (ou tentativas esgotadas).")

    # --- INÍCIO DA LIMPEZA E TRANSFORMAÇÃO ---
    if not todos_os_clientes_lista:
        print("Nenhum cliente foi extraído da API para processar.")
    else:
        print(f"Total de {len(todos_os_clientes_lista)} registros de clientes extraídos para processamento.")

        df_clientes = pd.json_normalize(todos_os_clientes_lista)
        
        print("\n--- DataFrame Inicial (após json_normalize) ---")
        print(df_clientes.head())

        if 'tags' in df_clientes.columns:
            df_clientes['tags_combinadas'] = df_clientes['tags'].apply(
                lambda lista_de_dicts_tag: ', '.join(
                    [d['tag'] for d in lista_de_dicts_tag if isinstance(d, dict) and 'tag' in d]
                ) if isinstance(lista_de_dicts_tag, list) else ''
            )
        else:
            print("Coluna 'tags' não encontrada no DataFrame.")
            df_clientes['tags_combinadas'] = ''

        if 'enderecoEntrega' in df_clientes.columns:
            # Se 'enderecoEntrega' não gerou subcolunas (ex: 'enderecoEntrega.rua'),
            # significa que provavelmente são dicionários vazios ou todos os valores são nulos.
            # Nesse caso, a coluna original 'enderecoEntrega' (que é do tipo objeto) pode ser removida.
            if not any(col.startswith('enderecoEntrega.') for col in df_clientes.columns):
                if df_clientes['enderecoEntrega'].dtype == 'object':
                     print("Removendo coluna 'enderecoEntrega' original pois não gerou subcampos.")
                     df_clientes = df_clientes.drop(columns=['enderecoEntrega'])
            # Se subcampos foram criados (ex: 'enderecoEntrega.rua'),
            # a coluna original 'enderecoEntrega' (que contém o dict) também pode ser removida.
            elif df_clientes['enderecoEntrega'].dtype == 'object':
                 print("Removendo coluna 'enderecoEntrega' original pois subcampos foram gerados.")
                 df_clientes = df_clientes.drop(columns=['enderecoEntrega'])


        print("\n--- DataFrame após processar 'tags' e 'enderecoEntrega' (se aplicável) ---")
        print(df_clientes.head())

        df_processado = df_clientes.replace('', pd.NA)
        df_processado = df_processado.fillna('N/A')

        print("\n--- DataFrame após preencher valores nulos/vazios ---")
        print(df_processado.head())

        print("\n--- Colunas Disponíveis para Seleção/Renomeação ---")
        colunas_disponiveis = df_processado.columns.tolist()
        for col_name in colunas_disponiveis:
            print(f"- {col_name}")

        mapa_renomeacao_e_selecao = {
            'codigo_cliente_omie': 'ID Cliente Omie',
            'cnpj_cpf': 'CNPJ/CPF',
            'razao_social': 'Razão Social',
            'nome_fantasia': 'Nome Fantasia',
            'endereco': 'Endereço Principal',
            'endereco_numero': 'Número End.',
            'bairro': 'Bairro',
            'cidade': 'Cidade',
            'estado': 'UF',
            'cep': 'CEP',
            'telefone1_ddd': 'DDD Tel Principal',
            'telefone1_numero': 'Número Tel Principal',
            'email': 'E-mail Principal',
            'info.dInc': 'Data Inclusão',
            'info.hInc': 'Hora Inclusão',
            'info.dAlt': 'Data Alteração',
            'info.hAlt': 'Hora Alteração',
            'tags_combinadas': 'Tags',
            'inativo': 'Inativo (S/N)',
            'dadosBancarios.codigo_banco': 'Cód. Banco', # Exemplo de campo aninhado
            'recomendacoes.gerar_boletos': 'Gera Boletos (S/N)' # Exemplo de campo aninhado
            # Adicione ou remova colunas conforme a lista de "Colunas Disponíveis" e sua necessidade
        }

        colunas_para_manter_renomeadas = {
            antigo: novo for antigo, novo in mapa_renomeacao_e_selecao.items() if antigo in df_processado.columns
        }
        colunas_antigas_selecionadas = list(colunas_para_manter_renomeadas.keys())

        if not colunas_antigas_selecionadas:
            print("\nAVISO: Nenhuma coluna do 'mapa_renomeacao_e_selecao' foi encontrada no DataFrame.")
            df_final = df_processado.copy() 
        else:
            df_final = df_processado[colunas_antigas_selecionadas].rename(columns=colunas_para_manter_renomeadas)

            ordem_desejada = [
                'ID Cliente Omie', 'CNPJ/CPF', 'Razão Social', 'Nome Fantasia',
                'E-mail Principal', 'DDD Tel Principal', 'Número Tel Principal',
                'Endereço Principal', 'Número End.', 'Bairro', 'Cidade', 'UF', 'CEP',
                'Data Inclusão', 'Hora Inclusão', 'Data Alteração', 'Hora Alteração',
                'Tags', 'Inativo (S/N)', 'Cód. Banco', 'Gera Boletos (S/N)'
            ]
            ordem_existente = [col for col in ordem_desejada if col in df_final.columns]
            # Adicionar colunas do df_final que não estão na ordem_desejada, para não perdê-las
            for col in df_final.columns:
                if col not in ordem_existente:
                    ordem_existente.append(col)
            df_final = df_final[ordem_existente]

        print("\n--- DataFrame Final (Limpado, Selecionado e Renomeado) ---")
        print(df_final.head())
        print(f"\nTotal de {len(df_final)} clientes no DataFrame final.")

        # Descomente a linha abaixo para salvar em Excel
        # nome_arquivo_saida_excel = 'omie_clientes_limpos_todos.xlsx'
        # df_final.to_excel(nome_arquivo_saida_excel, index=False)
        # print(f"\nDataFrame final salvo em: {nome_arquivo_saida_excel}")

        # Salvar em arquivo CSV
        nome_arquivo_saida_csv = 'omie_clientes_limpos_todos.csv'
        df_final.to_csv(nome_arquivo_saida_csv, index=False, sep=';', encoding='utf-8-sig')
        print(f"\nDataFrame final salvo em: {nome_arquivo_saida_csv}")

except requests.exceptions.HTTPError as http_err:
    print(f"Erro HTTP ocorreu durante a chamada à API: {http_err}")
    if 'response' in locals() and response is not None: # Verifica se response existe
        print(f"Conteúdo da resposta: {response.text}")
except requests.exceptions.ConnectionError as conn_err: # Captura ConnectionError especificamente
    print(f"Erro de Conexão ocorreu durante a chamada à API: {conn_err}")
except requests.exceptions.Timeout as timeout_err: # Captura Timeout especificamente
    print(f"Timeout ocorreu durante a chamada à API: {timeout_err}")
except requests.exceptions.RequestException as req_err: # Captura outras exceções de requests
    print(f"Erro de Requisição ocorreu durante a chamada à API: {req_err}")
except json.JSONDecodeError as json_err:
    print(f"Erro ao decodificar JSON da resposta da API: {json_err}")
    if 'response' in locals() and response is not None: # Verifica se response existe
        print(f"Conteúdo da resposta que causou o erro: {response.text}")
except Exception as e:
    print(f"Um erro inesperado ocorreu: {e}")

Iniciando extração de clientes da API Omie. Registros por página: 50
Buscando página 1 de ... (Tentativa 1/3)...
API informou: 60 página(s) e 2996 registro(s) no total.
  50 clientes adicionados da página 1. Total acumulado: 50
Buscando página 2 de 60 (Tentativa 1/3)...
  50 clientes adicionados da página 2. Total acumulado: 100
Buscando página 3 de 60 (Tentativa 1/3)...
  50 clientes adicionados da página 3. Total acumulado: 150
Buscando página 4 de 60 (Tentativa 1/3)...
  50 clientes adicionados da página 4. Total acumulado: 200
Buscando página 5 de 60 (Tentativa 1/3)...
  50 clientes adicionados da página 5. Total acumulado: 250
Buscando página 6 de 60 (Tentativa 1/3)...
  50 clientes adicionados da página 6. Total acumulado: 300
Buscando página 7 de 60 (Tentativa 1/3)...
  50 clientes adicionados da página 7. Total acumulado: 350
Buscando página 8 de 60 (Tentativa 1/3)...
  50 clientes adicionados da página 8. Total acumulado: 400
Buscando página 9 de 60 (Tentativa 1/3)...
  50 cli

In [7]:
endpoint = "projetos"

base_url = f"https://app.omie.com.br/api/v1/geral/projetos/" 

headers = {
    'Content-Type': 'application/json'
}

body = {
    "call": 'ListarProjetos', 
    "app_key": APP_KEY,
    "app_secret": APP_SECRET,
    "param": [
        {
            "pagina": 1,
            "registros_por_pagina": 50,
            "apenas_importado_api": "N"
        }
    ]
}


try:
    response = requests.post(base_url, headers=headers, data=json.dumps(body))

    # Check if the request was successful
    if response.status_code == 200:
        print("Dados extraídos com sucesso!")
        # Attempt to parse the JSON response
        try:
            print(json.dumps(response.json(), indent=2, ensure_ascii=False)) # Added ensure_ascii=False for better display of special characters
        except json.JSONDecodeError:
            print("Não foi possível decodificar a resposta JSON:")
            print(response.text)
    elif response.status_code == 401:
        print(f"Erro de autenticação (Unauthorized): {response.status_code}")
        print("Verifique se suas credenciais (APP_KEY e APP_SECRET) estão corretas e ativas.")
        print("Detalhes:", response.text)
    elif response.status_code == 500:
        print(f"Erro interno do servidor Omie: {response.status_code}")
        print("Detalhes:", response.text)
        print("Consulte a documentação da API Omie para o faultstring/faultcode específico.")
        # Example of how you might parse Omie's specific error structure if it's JSON
        try:
            error_details = response.json()
            if "faultstring" in error_details:
                print(f"Faultstring: {error_details['faultstring']}")
            if "faultcode" in error_details:
                print(f"Faultcode: {error_details['faultcode']}")
        except json.JSONDecodeError:
            pass # Already printed the raw text
    else:
        print(f"Erro na requisição: {response.status_code}")
        print(response.text)

except requests.exceptions.RequestException as e:
    print(f"Erro na conexão com a API: {e}")



Dados extraídos com sucesso!
{
  "pagina": 1,
  "total_de_paginas": 1,
  "registros": 14,
  "total_de_registros": 14,
  "cadastro": [
    {
      "codInt": "",
      "codigo": 679094336,
      "inativo": "N",
      "info": {
        "data_alt": "01/04/2019",
        "data_inc": "07/01/2019",
        "hora_alt": "11:58:14",
        "hora_inc": "12:29:24",
        "user_alt": "P000090237",
        "user_inc": "P000090237"
      },
      "nome": "OVER HEAD CORPORATIVO"
    },
    {
      "codInt": "",
      "codigo": 679099556,
      "inativo": "N",
      "info": {
        "data_alt": "07/01/2019",
        "data_inc": "07/01/2019",
        "hora_alt": "15:28:02",
        "hora_inc": "12:40:10",
        "user_alt": "P000102075",
        "user_inc": "P000090237"
      },
      "nome": "PROJETOS"
    },
    {
      "codInt": "",
      "codigo": 679099741,
      "inativo": "N",
      "info": {
        "data_alt": "07/01/2019",
        "data_inc": "07/01/2019",
        "hora_alt": "15:28:18",


In [8]:

APP_KEY = "596450736882" # Mantendo as mesmas chaves
APP_SECRET = "37a10edb420d8cd6c7afddcaaffcac05" # Mantendo as mesmas chaves

# --- CONFIGURAÇÕES ESPECÍFICAS PARA O ENDPOINT DE PROJETOS ---
endpoint = "projetos"
api_call_name = "ListarProjetos"
# A chave no JSON da API que contém a lista de itens (projetos)
chave_lista_itens_api = "cadastro"
# Nome do arquivo de saída
nome_arquivo_saida_csv = 'omie_projetos_limpos_todos.csv'
nome_arquivo_saida_excel = 'omie_projetos_limpos_todos.xlsx'
# -----------------------------------------------------------------

versao = "v1" # Geralmente v1 para endpoints gerais da Omie
base_url = f"https://app.omie.com.br/api/{versao}/geral/{endpoint}/"

headers = {
    'Content-Type': 'application/json'
}

# Parâmetros iniciais
pagina_atual = 1
registros_por_pagina = 50
max_tentativas_por_pagina = 3
delay_entre_tentativas_base = 5

print(f"Iniciando extração de '{endpoint}' da API Omie. Registros por página: {registros_por_pagina}")

todos_os_itens_lista = []
total_de_paginas_api = 1

try:
    while pagina_atual <= total_de_paginas_api:
        body = {
            'call': api_call_name,
            'app_key': APP_KEY,
            'app_secret': APP_SECRET,
            'param': [
                {
                    "pagina": pagina_atual,
                    "registros_por_pagina": registros_por_pagina,
                    "apenas_importado_api": "N" # Parâmetro específico do ListarProjetos
                    # Adicione outros parâmetros específicos do 'ListarProjetos' se necessário
                }
            ]
        }

        tentativa_atual = 0
        sucesso_pagina = False
        while tentativa_atual < max_tentativas_por_pagina and not sucesso_pagina:
            tentativa_atual += 1
            print(f"Buscando página {pagina_atual} de {total_de_paginas_api if total_de_paginas_api > 1 else '...' } para '{endpoint}' (Tentativa {tentativa_atual}/{max_tentativas_por_pagina})...")
            
            try:
                response = requests.post(base_url, headers=headers, data=json.dumps(body), timeout=30)
                response.raise_for_status()
                
                dados_api_pagina = response.json()
                sucesso_pagina = True

                if pagina_atual == 1 and tentativa_atual == 1:
                    total_de_paginas_api = dados_api_pagina.get('total_de_paginas', 1)
                    total_de_registros_api = dados_api_pagina.get('total_de_registros', 0)
                    print(f"API informou: {total_de_paginas_api} página(s) e {total_de_registros_api} registro(s) no total para '{endpoint}'.")
                    if total_de_registros_api == 0:
                        print(f"Nenhum registro encontrado na API para '{endpoint}'.")
                        total_de_paginas_api = 0 
                        break 

                if sucesso_pagina:
                    itens_da_pagina = dados_api_pagina.get(chave_lista_itens_api, [])
                    if itens_da_pagina:
                        todos_os_itens_lista.extend(itens_da_pagina)
                        print(f"  {len(itens_da_pagina)} itens adicionados da página {pagina_atual}. Total acumulado: {len(todos_os_itens_lista)}")
                    else:
                        print(f"  Nenhum item encontrado na página {pagina_atual} (resposta da API para '{chave_lista_itens_api}').")
                
            except (requests.exceptions.ConnectionError, requests.exceptions.Timeout, requests.exceptions.HTTPError) as e:
                print(f"  Erro ao buscar página {pagina_atual} (tentativa {tentativa_atual}): {e}")
                if tentativa_atual < max_tentativas_por_pagina:
                    delay_atual = delay_entre_tentativas_base * (2 ** (tentativa_atual - 1))
                    print(f"  Aguardando {delay_atual} segundos antes de tentar novamente...")
                    time.sleep(delay_atual)
                else:
                    print(f"  Máximo de {max_tentativas_por_pagina} tentativas atingido para a página {pagina_atual}. Desistindo desta página.")
                    break 

        if not sucesso_pagina and total_de_paginas_api > 0 :
             print(f"AVISO: Falha ao processar a página {pagina_atual} após todas as tentativas. Alguns dados podem estar faltando.")

        if pagina_atual < total_de_paginas_api and total_de_paginas_api > 0:
            time.sleep(1) 
        
        if total_de_paginas_api == 0: 
            break

        pagina_atual += 1

    print(f"\nExtração de todas as páginas para '{endpoint}' concluída (ou tentativas esgotadas).")

    # --- INÍCIO DA LIMPEZA E TRANSFORMAÇÃO ---
    if not todos_os_itens_lista:
        print(f"Nenhum item de '{endpoint}' foi extraído da API para processar.")
    else:
        print(f"Total de {len(todos_os_itens_lista)} registros de '{endpoint}' extraídos para processamento.")

        df_itens = pd.json_normalize(todos_os_itens_lista)
        
        print("\n--- DataFrame Inicial (após json_normalize) ---")
        print(df_itens.head())
        
        # Para "projetos", não temos 'tags' ou 'enderecoEntrega' da mesma forma que clientes.
        # A estrutura 'info' parece ser comum. json_normalize já achata campos como 'info.data_alt'.

        print("\n--- DataFrame após processamento inicial (se houver) ---")
        print(df_itens.head())

        df_processado = df_itens.replace('', pd.NA)
        df_processado = df_processado.fillna('N/A') # Preenche todos os NaNs com 'N/A'

        print("\n--- DataFrame após preencher valores nulos/vazios ---")
        print(df_processado.head())

        print("\n--- Colunas Disponíveis para Seleção/Renomeação ---")
        colunas_disponiveis = df_processado.columns.tolist()
        for col_name in colunas_disponiveis:
            print(f"- {col_name}")

        # --- AJUSTE AQUI: Defina as colunas que quer manter e os novos nomes para PROJETOS ---
        mapa_renomeacao_e_selecao = {
            # Baseado na sua amostra de "projetos":
            'codigo': 'ID Projeto Omie',
            'codInt': 'Código Integração Projeto',
            'nome': 'Nome do Projeto',
            'inativo': 'Inativo (S/N)',
            'info.data_inc': 'Data Inclusão',
            'info.hora_inc': 'Hora Inclusão',
            'info.user_inc': 'Usuário Inclusão',
            'info.data_alt': 'Data Alteração',
            'info.hora_alt': 'Hora Alteração',
            'info.user_alt': 'Usuário Alteração',
            # Adicione outras colunas que você identificar na saída de "Colunas Disponíveis"
            # e que sejam relevantes para "projetos".
        }

        colunas_para_manter_renomeadas = {
            antigo: novo for antigo, novo in mapa_renomeacao_e_selecao.items() if antigo in df_processado.columns
        }
        colunas_antigas_selecionadas = list(colunas_para_manter_renomeadas.keys())

        if not colunas_antigas_selecionadas:
            print(f"\nAVISO: Nenhuma coluna do 'mapa_renomeacao_e_selecao' para '{endpoint}' foi encontrada no DataFrame.")
            df_final = df_processado.copy() 
        else:
            df_final = df_processado[colunas_antigas_selecionadas].rename(columns=colunas_para_manter_renomeadas)

            # --- AJUSTE AQUI: Defina a ordem desejada para as colunas de PROJETOS ---
            ordem_desejada = [
                'ID Projeto Omie',
                'Nome do Projeto',
                'Código Integração Projeto',
                'Inativo (S/N)',
                'Data Inclusão',
                'Hora Inclusão',
                'Usuário Inclusão',
                'Data Alteração',
                'Hora Alteração',
                'Usuário Alteração'
                # Adicione os outros NOVOS NOMES na ordem que preferir
            ]
            ordem_existente = [col for col in ordem_desejada if col in df_final.columns]
            for col in df_final.columns: # Adiciona colunas não listadas na ordem_desejada ao final
                if col not in ordem_existente:
                    ordem_existente.append(col)
            df_final = df_final[ordem_existente]

        print("\n--- DataFrame Final (Limpado, Selecionado e Renomeado) ---")
        print(df_final.head())
        print(f"\nTotal de {len(df_final)} itens de '{endpoint}' no DataFrame final.")

        # Salvar em arquivo CSV
        df_final.to_csv(nome_arquivo_saida_csv, index=False, sep=';', encoding='utf-8-sig')
        print(f"\nDataFrame final salvo em: {nome_arquivo_saida_csv}")

        # Opcional: Salvar em arquivo Excel
        # df_final.to_excel(nome_arquivo_saida_excel, index=False)
        # print(f"\nDataFrame final salvo em: {nome_arquivo_saida_excel}")

except requests.exceptions.HTTPError as http_err:
    print(f"Erro HTTP ocorreu durante a chamada à API para '{endpoint}': {http_err}")
    if 'response' in locals() and response is not None:
        print(f"Conteúdo da resposta: {response.text}")
except requests.exceptions.ConnectionError as conn_err:
    print(f"Erro de Conexão ocorreu durante a chamada à API para '{endpoint}': {conn_err}")
except requests.exceptions.Timeout as timeout_err:
    print(f"Timeout ocorreu durante a chamada à API para '{endpoint}': {timeout_err}")
except requests.exceptions.RequestException as req_err:
    print(f"Erro de Requisição ocorreu durante a chamada à API para '{endpoint}': {req_err}")
except json.JSONDecodeError as json_err:
    print(f"Erro ao decodificar JSON da resposta da API para '{endpoint}': {json_err}")
    if 'response' in locals() and response is not None:
        print(f"Conteúdo da resposta que causou o erro: {response.text}")
except Exception as e:
    print(f"Um erro inesperado ocorreu durante o processamento de '{endpoint}': {e}")

Iniciando extração de 'projetos' da API Omie. Registros por página: 50
Buscando página 1 de ... para 'projetos' (Tentativa 1/3)...
API informou: 1 página(s) e 14 registro(s) no total para 'projetos'.
  14 itens adicionados da página 1. Total acumulado: 14

Extração de todas as páginas para 'projetos' concluída (ou tentativas esgotadas).
Total de 14 registros de 'projetos' extraídos para processamento.

--- DataFrame Inicial (após json_normalize) ---
  codInt     codigo inativo                   nome info.data_alt  \
0         679094336       N  OVER HEAD CORPORATIVO    01/04/2019   
1         679099556       N               PROJETOS    07/01/2019   
2         679099741       N             SUPERVISÃO    07/01/2019   
3         679158073       S  OPERAÇÕES RODOVIÁRIAS    08/08/2023   
4         680028235       N              COMERCIAL    15/01/2019   

  info.data_inc info.hora_alt info.hora_inc info.user_alt info.user_inc  
0    07/01/2019      11:58:14      12:29:24    P000090237    P0

In [9]:

# Suas credenciais da API Omie
APP_KEY = "596450736882"
APP_SECRET = "37a10edb420d8cd6c7afddcaaffcac05"

# --- CONFIGURAÇÃO PARA EXTRAÇÃO DE EMPRESAS ---
endpoint_name = "Empresas" # Nome da categoria para exibição e uso no DataFrame
base_module = "geral"      # Módulo para Empresas na Omie API, conforme sua URL
api_path = "empresas"      # Parte final da URL do endpoint
call_method = "ListarEmpresas" # Nome do método de chamada da API
# Chave que contém a lista de empresas na resposta JSON (pode variar, verificar docs Omie)
# Tentaremos 'empresa_cadastro' ou 'empresas' ou 'listaRegistros'
json_data_keys_to_try = ["empresa_cadastro", "empresas", "listaRegistros"]


base_url = f"https://app.omie.com.br/api/v1/{base_module}/{api_path}/" 

headers = {
    'Content-Type': 'application/json'
}

# Parâmetros da requisição para Empresas
body = {
    "call": call_method,
    "app_key": APP_KEY,
    "app_secret": APP_SECRET,
    "param": [
        {
            "pagina": 1,
            "registros_por_pagina": 100, # Você especificou 100 aqui
            "apenas_importado_api": "N"
        }
    ]
}

print(f"Iniciando extração de dados para: {endpoint_name}")
print(f"URL da Requisição: {base_url}")
print(f"Método da Requisição (call): {call_method}")
print("-" * 50)

df_empresas = pd.DataFrame() # DataFrame vazio para armazenar os dados de empresas

try:
    response = requests.post(base_url, headers=headers, data=json.dumps(body))
    response.raise_for_status() # Lança uma exceção para códigos de status de erro (4xx ou 5xx)

    print(f"Dados de {endpoint_name} extraídos com sucesso!")
    
    data = response.json()

    # Tentar extrair a lista de empresas da resposta JSON usando as chaves prováveis
    found_key = None
    for key_attempt in json_data_keys_to_try:
        if key_attempt in data and isinstance(data.get(key_attempt), list):
            df_empresas = pd.DataFrame(data[key_attempt])
            found_key = key_attempt
            break # Sair do loop assim que a chave for encontrada

    if found_key:
        print(f"Chave de dados JSON identificada: '{found_key}'")
        print(f"\n--- Análise Exploratória para {endpoint_name} ---")
        print(f"Shape do DataFrame: {df_empresas.shape}")
        print("\nPrimeiras 5 linhas:")
        print(df_empresas.head())

        print("\nInformações do DataFrame (tipos de dados e contagem de não-nulos):")
        buffer = io.StringIO()
        df_empresas.info(buf=buffer, verbose=True, show_counts=True)
        print(buffer.getvalue())

        print("\nContagem de Valores Nulos por Coluna:")
        print(df_empresas.isnull().sum())
        print("-" * 50)
    else:
        print(f"Atenção: Não foi possível identificar a chave dos dados para '{endpoint_name}' na resposta da API.")
        print("Resposta completa da API (para depuração):", json.dumps(data, indent=2, ensure_ascii=False))
        print("-" * 50)

except requests.exceptions.HTTPError as http_err:
    print(f"Erro HTTP ao extrair {endpoint_name}: {http_err}")
    print(f"Resposta da API: {response.text}")
    print("-" * 50)
except requests.exceptions.ConnectionError as conn_err:
    print(f"Erro de Conexão ao extrair {endpoint_name}: {conn_err}")
    print("Verifique sua conexão com a internet ou o status da API Omie.")
    print("-" * 50)
except requests.exceptions.Timeout as timeout_err:
    print(f"Erro de Timeout ao extrair {endpoint_name}: {timeout_err}")
    print("A requisição demorou muito para responder.")
    print("-" * 50)
except requests.exceptions.RequestException as req_err:
    print(f"Erro inesperado ao extrair {endpoint_name}: {req_err}")
    print("-" * 50)
except json.JSONDecodeError:
    print(f"Erro ao decodificar JSON para {endpoint_name}. Resposta não é JSON válido.")
    print(f"Resposta da API: {response.text}")
    print("-" * 50)
except Exception as e:
    print(f"Ocorreu um erro geral ao processar {endpoint_name}: {e}")
    print("-" * 50)

print("\n--- Extração de Empresas concluída! ---")

# Agora você tem o DataFrame 'df_empresas' com os dados extraídos.
# Você pode fazer mais análises ou salvá-lo:
# df_empresas.to_csv('empresas_omie.csv', index=False, encoding='utf-8')
# df_empresas.to_excel('empresas_omie.xlsx', index=False)

Iniciando extração de dados para: Empresas
URL da Requisição: https://app.omie.com.br/api/v1/geral/empresas/
Método da Requisição (call): ListarEmpresas
--------------------------------------------------
Dados de Empresas extraídos com sucesso!
Atenção: Não foi possível identificar a chave dos dados para 'Empresas' na resposta da API.
Resposta completa da API (para depuração): {
  "pagina": 1,
  "total_de_paginas": 1,
  "registros": 1,
  "total_de_registros": 1,
  "empresas_cadastro": [
    {
      "bairro": "CERQUEIRA CESAR",
      "cep": "01415-002",
      "cidade": "SAO PAULO (SP)",
      "cnae": "7112000",
      "cnae_municipal": "",
      "cnpj": "42.161.372/0001-40",
      "codigo_empresa": "671524551",
      "codigo_pais": "1058",
      "complemento": "9 ANDAR",
      "csc_homologacao": "",
      "csc_id_homologacao": "",
      "csc_id_producao": "",
      "csc_producao": "",
      "ecd_codigo_cadastral": "",
      "ecd_codigo_instituicao_responsavel": "",
      "efd_atividade_i

In [10]:

APP_KEY = "596450736882"
APP_SECRET = "37a10edb420d8cd6c7afddcaaffcac05"

# --- CONFIGURAÇÕES ESPECÍFICAS PARA O ENDPOINT DE EMPRESAS ---
endpoint_display_name = "Empresas" # Nome para exibição
base_module = "geral"
api_path = "empresas"
api_call_name = "ListarEmpresas"

# Chaves prováveis no JSON da API que contêm a lista de itens (empresas)
# O script tentará estas chaves em ordem.
json_data_keys_to_try = ["empresa_cadastro", "empresas", "lista_empresas", "cadastro", "registros"]

# Nome do arquivo de saída
nome_arquivo_saida_csv = 'omie_empresas_limpas_todas.csv'
nome_arquivo_saida_excel = 'omie_empresas_limpas_todas.xlsx'
# -----------------------------------------------------------------

versao = "v1" # Geralmente v1 para endpoints gerais da Omie
base_url = f"https://app.omie.com.br/api/{versao}/{base_module}/{api_path}/"

headers = {
    'Content-Type': 'application/json'
}

# Parâmetros iniciais
pagina_atual = 1
registros_por_pagina = 100 # Conforme seu exemplo para empresas
max_tentativas_por_pagina = 3
delay_entre_tentativas_base = 5

print(f"Iniciando extração de '{endpoint_display_name}' da API Omie. Registros por página: {registros_por_pagina}")
print(f"URL Base: {base_url}")

todos_os_itens_lista = []
total_de_paginas_api = 1
chave_lista_itens_api_identificada = None

try:
    while pagina_atual <= total_de_paginas_api:
        body = {
            'call': api_call_name,
            'app_key': APP_KEY,
            'app_secret': APP_SECRET,
            'param': [
                {
                    "pagina": pagina_atual,
                    "registros_por_pagina": registros_por_pagina,
                    "apenas_importado_api": "N" # Parâmetro comum, mantenha se aplicável a Empresas
                }
            ]
        }

        tentativa_atual = 0
        sucesso_pagina = False
        dados_api_pagina = None # Inicializa para o escopo do loop de tentativas

        while tentativa_atual < max_tentativas_por_pagina and not sucesso_pagina:
            tentativa_atual += 1
            print(f"Buscando página {pagina_atual} de {total_de_paginas_api if total_de_paginas_api > 1 else '...' } para '{endpoint_display_name}' (Tentativa {tentativa_atual}/{max_tentativas_por_pagina})...")
            
            try:
                response = requests.post(base_url, headers=headers, data=json.dumps(body), timeout=30)
                response.raise_for_status()
                
                dados_api_pagina = response.json()
                sucesso_pagina = True

                if pagina_atual == 1 and tentativa_atual == 1: # Apenas na primeira tentativa bem-sucedida da primeira página
                    total_de_paginas_api = dados_api_pagina.get('total_de_paginas', 1)
                    total_de_registros_api = dados_api_pagina.get('total_de_registros', 0)
                    print(f"API informou: {total_de_paginas_api} página(s) e {total_de_registros_api} registro(s) no total para '{endpoint_display_name}'.")
                    
                    if total_de_registros_api == 0:
                        print(f"Nenhum registro encontrado na API para '{endpoint_display_name}'.")
                        total_de_paginas_api = 0 
                        break # Sai do loop de tentativas, e o loop principal também sairá
                    
                    # Tenta identificar a chave correta para a lista de itens
                    if not chave_lista_itens_api_identificada:
                        for key_attempt in json_data_keys_to_try:
                            if key_attempt in dados_api_pagina and isinstance(dados_api_pagina.get(key_attempt), list):
                                chave_lista_itens_api_identificada = key_attempt
                                print(f"Chave de dados JSON identificada para a lista de itens: '{chave_lista_itens_api_identificada}'")
                                break
                        if not chave_lista_itens_api_identificada:
                            print(f"ATENÇÃO: Não foi possível identificar a chave da lista de itens para '{endpoint_display_name}' na resposta da API.")
                            print("Resposta da API (primeira página):", json.dumps(dados_api_pagina, indent=2, ensure_ascii=False))
                            # Decide se quer parar ou continuar assumindo uma chave padrão (pode falhar)
                            # Por segurança, vamos parar se a chave não for encontrada.
                            total_de_paginas_api = 0 # Para o loop principal
                            break # Sai do loop de tentativas

                if sucesso_pagina and chave_lista_itens_api_identificada:
                    itens_da_pagina = dados_api_pagina.get(chave_lista_itens_api_identificada, [])
                    if itens_da_pagina:
                        todos_os_itens_lista.extend(itens_da_pagina)
                        print(f"  {len(itens_da_pagina)} itens adicionados da página {pagina_atual}. Total acumulado: {len(todos_os_itens_lista)}")
                    else:
                        print(f"  Nenhum item encontrado na página {pagina_atual} (usando chave '{chave_lista_itens_api_identificada}').")
                elif sucesso_pagina and not chave_lista_itens_api_identificada and total_de_registros_api > 0:
                    print(f"  Sucesso ao buscar página {pagina_atual}, mas a chave da lista de itens ainda não foi identificada. Isso não deveria acontecer.")
                    # Pode ser um erro de lógica, parar por segurança
                    total_de_paginas_api = 0
                    break


            except (requests.exceptions.ConnectionError, requests.exceptions.Timeout, requests.exceptions.HTTPError) as e:
                print(f"  Erro ao buscar página {pagina_atual} (tentativa {tentativa_atual}): {e}")
                if tentativa_atual < max_tentativas_por_pagina:
                    delay_atual = delay_entre_tentativas_base * (2 ** (tentativa_atual - 1))
                    print(f"  Aguardando {delay_atual} segundos antes de tentar novamente...")
                    time.sleep(delay_atual)
                else:
                    print(f"  Máximo de {max_tentativas_por_pagina} tentativas atingido para a página {pagina_atual}. Desistindo desta página.")
                    break # Sai do loop de tentativas para esta página

        if not sucesso_pagina and total_de_paginas_api > 0 :
             print(f"AVISO: Falha ao processar a página {pagina_atual} após todas as tentativas. Alguns dados podem estar faltando.")

        if pagina_atual < total_de_paginas_api and total_de_paginas_api > 0:
            time.sleep(1) 
        
        if total_de_paginas_api == 0: # Se a API informou 0 registros ou a chave não foi encontrada
            break

        pagina_atual += 1

    print(f"\nExtração de todas as páginas para '{endpoint_display_name}' concluída (ou tentativas esgotadas).")

    # --- INÍCIO DA LIMPEZA E TRANSFORMAÇÃO ---
    if not todos_os_itens_lista:
        print(f"Nenhum item de '{endpoint_display_name}' foi extraído da API para processar.")
    else:
        print(f"Total de {len(todos_os_itens_lista)} registros de '{endpoint_display_name}' extraídos para processamento.")

        # pd.json_normalize irá achatar estruturas aninhadas (ex: 'info.campo')
        df_itens = pd.json_normalize(todos_os_itens_lista)
        
        print("\n--- DataFrame Inicial (após json_normalize) ---")
        if not df_itens.empty:
            print(df_itens.head())
        else:
            print("DataFrame inicial está vazio.")
        
        df_processado = df_itens.replace('', pd.NA)
        df_processado = df_processado.fillna('N/A') # Preenche todos os NaNs com 'N/A'

        print("\n--- DataFrame após preencher valores nulos/vazios ---")
        if not df_processado.empty:
            print(df_processado.head())
        else:
            print("DataFrame processado está vazio.")


        print("\n--- Colunas Disponíveis para Seleção/Renomeação ---")
        if not df_processado.empty:
            colunas_disponiveis = df_processado.columns.tolist()
            for col_name in colunas_disponiveis:
                print(f"- {col_name}")
        else:
            print("Nenhuma coluna disponível, DataFrame está vazio.")

        # --- AJUSTE AQUI: Defina as colunas que quer manter e os novos nomes para EMPRESAS ---
        # Este é um EXEMPLO. Você PRECISARÁ ajustar com base na saída de "Colunas Disponíveis"
        # que o script gerar com os dados reais de "Empresas".
        mapa_renomeacao_e_selecao = {
            # Exemplo de colunas comuns (VERIFIQUE A SAÍDA REAL DA API PARA 'Empresas'):
            'codigo_empresa': 'ID Empresa Omie', # Nome hipotético, verifique o real
            'cnpj': 'CNPJ Empresa',             # Nome hipotético
            'razao_social': 'Razão Social',
            'nome_fantasia': 'Nome Fantasia',
            'cidade': 'Cidade Sede',
            'estado': 'UF Sede',
            'email': 'E-mail Contato',
            'data_abertura': 'Data Abertura Empresa', # Nome hipotético
            'info.dInc': 'Data Inclusão Registro', # Se houver um campo 'info'
            'info.dAlt': 'Data Alteração Registro'
            # Adicione outras colunas que você identificar na saída de "Colunas Disponíveis"
            # e que sejam relevantes para "Empresas".
        }

        df_final = pd.DataFrame() # Inicializa df_final vazio
        if not df_processado.empty:
            colunas_para_manter_renomeadas = {
                antigo: novo for antigo, novo in mapa_renomeacao_e_selecao.items() if antigo in df_processado.columns
            }
            colunas_antigas_selecionadas = list(colunas_para_manter_renomeadas.keys())

            if not colunas_antigas_selecionadas:
                print(f"\nAVISO: Nenhuma coluna do 'mapa_renomeacao_e_selecao' para '{endpoint_display_name}' foi encontrada no DataFrame.")
                print("O DataFrame final conterá todas as colunas originais (após preenchimento de nulos).")
                df_final = df_processado.copy() 
            else:
                df_final = df_processado[colunas_antigas_selecionadas].rename(columns=colunas_para_manter_renomeadas)

                # --- AJUSTE AQUI: Defina a ordem desejada para as colunas de EMPRESAS ---
                # Exemplo de ordem (use os NOVOS NOMES definidos no mapa acima)
                ordem_desejada = [
                    'ID Empresa Omie', 'CNPJ Empresa', 'Razão Social', 'Nome Fantasia',
                    'Cidade Sede', 'UF Sede', 'E-mail Contato', 'Data Abertura Empresa',
                    'Data Inclusão Registro', 'Data Alteração Registro'
                    # Adicione os outros NOVOS NOMES na ordem que preferir
                ]
                ordem_existente = [col for col in ordem_desejada if col in df_final.columns]
                for col in df_final.columns: # Adiciona colunas não listadas na ordem_desejada ao final
                    if col not in ordem_existente:
                        ordem_existente.append(col)
                if ordem_existente: # Apenas reordena se houver colunas
                    df_final = df_final[ordem_existente]
        else:
            print("DataFrame processado está vazio, não é possível criar df_final com colunas selecionadas.")


        print("\n--- DataFrame Final (Limpado, Selecionado e Renomeado) ---")
        if not df_final.empty:
            print(df_final.head())
            print(f"\nTotal de {len(df_final)} itens de '{endpoint_display_name}' no DataFrame final.")

            # Salvar em arquivo CSV
            df_final.to_csv(nome_arquivo_saida_csv, index=False, sep=';', encoding='utf-8-sig')
            print(f"\nDataFrame final salvo em: {nome_arquivo_saida_csv}")

            # Opcional: Salvar em arquivo Excel
            # df_final.to_excel(nome_arquivo_saida_excel, index=False)
            # print(f"\nDataFrame final salvo em: {nome_arquivo_saida_excel}")
        else:
            print("DataFrame final está vazio. Nenhum arquivo CSV será gerado.")


except requests.exceptions.HTTPError as http_err:
    print(f"Erro HTTP ocorreu durante a chamada à API para '{endpoint_display_name}': {http_err}")
    if 'response' in locals() and response is not None:
        print(f"Conteúdo da resposta: {response.text}")
except requests.exceptions.ConnectionError as conn_err:
    print(f"Erro de Conexão ocorreu durante a chamada à API para '{endpoint_display_name}': {conn_err}")
except requests.exceptions.Timeout as timeout_err:
    print(f"Timeout ocorreu durante a chamada à API para '{endpoint_display_name}': {timeout_err}")
except requests.exceptions.RequestException as req_err:
    print(f"Erro de Requisição ocorreu durante a chamada à API para '{endpoint_display_name}': {req_err}")
except json.JSONDecodeError as json_err:
    print(f"Erro ao decodificar JSON da resposta da API para '{endpoint_display_name}': {json_err}")
    if 'response' in locals() and response is not None:
        print(f"Conteúdo da resposta que causou o erro: {response.text}")
except Exception as e:
    print(f"Um erro inesperado ocorreu durante o processamento de '{endpoint_display_name}': {e}")

Iniciando extração de 'Empresas' da API Omie. Registros por página: 100
URL Base: https://app.omie.com.br/api/v1/geral/empresas/
Buscando página 1 de ... para 'Empresas' (Tentativa 1/3)...
API informou: 1 página(s) e 1 registro(s) no total para 'Empresas'.
ATENÇÃO: Não foi possível identificar a chave da lista de itens para 'Empresas' na resposta da API.
Resposta da API (primeira página): {
  "pagina": 1,
  "total_de_paginas": 1,
  "registros": 1,
  "total_de_registros": 1,
  "empresas_cadastro": [
    {
      "bairro": "CERQUEIRA CESAR",
      "cep": "01415-002",
      "cidade": "SAO PAULO (SP)",
      "cnae": "7112000",
      "cnae_municipal": "",
      "cnpj": "42.161.372/0001-40",
      "codigo_empresa": "671524551",
      "codigo_pais": "1058",
      "complemento": "9 ANDAR",
      "csc_homologacao": "",
      "csc_id_homologacao": "",
      "csc_id_producao": "",
      "csc_producao": "",
      "ecd_codigo_cadastral": "",
      "ecd_codigo_instituicao_responsavel": "",
      "efd

In [11]:

# Suas credenciais da API Omie
APP_KEY = "596450736882"
APP_SECRET = "37a10edb420d8cd6c7afddcaaffcac05"

# --- CONFIGURAÇÃO PARA EXTRAÇÃO DE DEPARTAMENTOS ---
endpoint_name = "Departamentos" # Nome da categoria para exibição e uso no DataFrame
base_module = "geral"           # Módulo para Departamentos na Omie API, conforme sua URL
api_path = "departamentos"      # Parte final da URL do endpoint
call_method = "ListarDepartamentos" # Nome do método de chamada da API
# Chave que contém a lista de departamentos na resposta JSON (pode variar, verificar docs Omie)
# Tentaremos 'departamentos_cadastro', 'listaDepartamentos', 'listaRegistros' ou 'departamentos'
json_data_keys_to_try = ["departamentos_cadastro", "listaDepartamentos", "listaRegistros", "departamentos"]


base_url = f"https://app.omie.com.br/api/v1/{base_module}/{api_path}/" 

headers = {
    'Content-Type': 'application/json'
}

# Parâmetros da requisição para Departamentos
body = {
    "call": call_method,
    "app_key": APP_KEY,
    "app_secret": APP_SECRET,
    "param": [
        {
            "pagina": 1,
            "registros_por_pagina": 50 # Você especificou 50 aqui
        }
    ]
}

print(f"Iniciando extração de dados para: {endpoint_name}")
print(f"URL da Requisição: {base_url}")
print(f"Método da Requisição (call): {call_method}")
print("-" * 50)

df_departamentos = pd.DataFrame() # DataFrame vazio para armazenar os dados de departamentos

try:
    response = requests.post(base_url, headers=headers, data=json.dumps(body))
    response.raise_for_status() # Lança uma exceção para códigos de status de erro (4xx ou 5xx)

    print(f"Dados de {endpoint_name} extraídos com sucesso!")
    
    data = response.json()

    # Tentar extrair a lista de departamentos da resposta JSON usando as chaves prováveis
    found_key = None
    for key_attempt in json_data_keys_to_try:
        if key_attempt in data and isinstance(data.get(key_attempt), list):
            df_departamentos = pd.DataFrame(data[key_attempt])
            found_key = key_attempt
            break # Sair do loop assim que a chave for encontrada

    if found_key:
        print(f"Chave de dados JSON identificada: '{found_key}'")
        print(f"\n--- Análise Exploratória para {endpoint_name} ---")
        print(f"Shape do DataFrame: {df_departamentos.shape}")
        print("\nPrimeiras 5 linhas:")
        print(df_departamentos.head())

        print("\nInformações do DataFrame (tipos de dados e contagem de não-nulos):")
        buffer = io.StringIO()
        df_departamentos.info(buf=buffer, verbose=True, show_counts=True)
        print(buffer.getvalue())

        print("\nContagem de Valores Nulos por Coluna:")
        print(df_departamentos.isnull().sum())
        print("-" * 50)
    else:
        print(f"Atenção: Não foi possível identificar a chave dos dados para '{endpoint_name}' na resposta da API.")
        print("Resposta completa da API (para depuração):", json.dumps(data, indent=2, ensure_ascii=False))
        print("-" * 50)

except requests.exceptions.HTTPError as http_err:
    print(f"Erro HTTP ao extrair {endpoint_name}: {http_err}")
    print(f"Resposta da API: {response.text}")
    print("-" * 50)
except requests.exceptions.ConnectionError as conn_err:
    print(f"Erro de Conexão ao extrair {endpoint_name}: {conn_err}")
    print("Verifique sua conexão com a internet ou o status da API Omie.")
    print("-" * 50)
except requests.exceptions.Timeout as timeout_err:
    print(f"Erro de Timeout ao extrair {endpoint_name}: {timeout_err}")
    print("A requisição demorou muito para responder.")
    print("-" * 50)
except requests.exceptions.RequestException as req_err:
    print(f"Erro inesperado ao extrair {endpoint_name}: {req_err}")
    print("-" * 50)
except json.JSONDecodeError:
    print(f"Erro ao decodificar JSON para {endpoint_name}. Resposta não é JSON válido.")
    print(f"Resposta da API: {response.text}")
    print("-" * 50)
except Exception as e:
    print(f"Ocorreu um erro geral ao processar {endpoint_name}: {e}")
    print("-" * 50)

print("\n--- Extração de Departamentos concluída! ---")

# Agora você tem o DataFrame 'df_departamentos' com os dados extraídos.
# Você pode fazer mais análises ou salvá-lo:
# df_departamentos.to_csv('departamentos_omie.csv', index=False, encoding='utf-8')
# df_departamentos.to_excel('departamentos_omie.xlsx', index=False)

Iniciando extração de dados para: Departamentos
URL da Requisição: https://app.omie.com.br/api/v1/geral/departamentos/
Método da Requisição (call): ListarDepartamentos
--------------------------------------------------
Dados de Departamentos extraídos com sucesso!
Chave de dados JSON identificada: 'departamentos'

--- Análise Exploratória para Departamentos ---
Shape do DataFrame: (50, 4)

Primeiras 5 linhas:
      codigo                     descricao    estrutura inativo
0  671524591        OVER HEAD COORPORATIVO      001.001       N
1  671524592                    SUPERVISÃO      001.002       N
2  671524593              ECR -CONSOLIDADO          001       N
3  671524594                      PROJETOS      001.003       N
4  679097416  120_ECR/EURO ESTUDIOS -BID 3  001.002.001       S

Informações do DataFrame (tipos de dados e contagem de não-nulos):
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50 entries, 0 to 49
Data columns (total 4 columns):
 #   Column     Non-Null Count  D

In [12]:

APP_KEY = "596450736882"
APP_SECRET = "37a10edb420d8cd6c7afddcaaffcac05"

# --- CONFIGURAÇÕES ESPECÍFICAS PARA O ENDPOINT DE DEPARTAMENTOS ---
endpoint_display_name = "Departamentos" # Nome para exibição
base_module = "geral"
api_path = "departamentos"
api_call_name = "ListarDepartamentos"

# Chaves prováveis no JSON da API que contêm a lista de itens (departamentos)
# O script tentará estas chaves em ordem.
json_data_keys_to_try = ["departamentos_cadastro", "listaDepartamentos", "listaRegistros", "departamentos", "cadastro"]

# Nome do arquivo de saída
nome_arquivo_saida_csv = 'omie_departamentos_limpos_todos.csv'
nome_arquivo_saida_excel = 'omie_departamentos_limpos_todos.xlsx'
# -----------------------------------------------------------------

versao = "v1" # Geralmente v1 para endpoints gerais da Omie
base_url = f"https://app.omie.com.br/api/{versao}/{base_module}/{api_path}/"

headers = {
    'Content-Type': 'application/json'
}

# Parâmetros iniciais
pagina_atual = 1
registros_por_pagina = 50 # Conforme seu exemplo para departamentos
max_tentativas_por_pagina = 3
delay_entre_tentativas_base = 5

print(f"Iniciando extração de '{endpoint_display_name}' da API Omie. Registros por página: {registros_por_pagina}")
print(f"URL Base: {base_url}")

todos_os_itens_lista = []
total_de_paginas_api = 1
chave_lista_itens_api_identificada = None

try:
    while pagina_atual <= total_de_paginas_api:
        body = {
            'call': api_call_name,
            'app_key': APP_KEY,
            'app_secret': APP_SECRET,
            'param': [
                {
                    "pagina": pagina_atual,
                    "registros_por_pagina": registros_por_pagina
                    # "apenas_importado_api": "N" # Removido, verificar se é aplicável a ListarDepartamentos
                }
            ]
        }

        tentativa_atual = 0
        sucesso_pagina = False
        dados_api_pagina = None # Inicializa para o escopo do loop de tentativas

        while tentativa_atual < max_tentativas_por_pagina and not sucesso_pagina:
            tentativa_atual += 1
            print(f"Buscando página {pagina_atual} de {total_de_paginas_api if total_de_paginas_api > 1 else '...' } para '{endpoint_display_name}' (Tentativa {tentativa_atual}/{max_tentativas_por_pagina})...")
            
            try:
                response = requests.post(base_url, headers=headers, data=json.dumps(body), timeout=30)
                response.raise_for_status()
                
                dados_api_pagina = response.json()
                sucesso_pagina = True

                if pagina_atual == 1 and tentativa_atual == 1: # Apenas na primeira tentativa bem-sucedida da primeira página
                    total_de_paginas_api = dados_api_pagina.get('total_de_paginas', 1)
                    total_de_registros_api = dados_api_pagina.get('total_de_registros', 0)
                    print(f"API informou: {total_de_paginas_api} página(s) e {total_de_registros_api} registro(s) no total para '{endpoint_display_name}'.")
                    
                    if total_de_registros_api == 0:
                        print(f"Nenhum registro encontrado na API para '{endpoint_display_name}'.")
                        total_de_paginas_api = 0 
                        break # Sai do loop de tentativas, e o loop principal também sairá
                    
                    # Tenta identificar a chave correta para a lista de itens
                    if not chave_lista_itens_api_identificada:
                        for key_attempt in json_data_keys_to_try:
                            if key_attempt in dados_api_pagina and isinstance(dados_api_pagina.get(key_attempt), list):
                                chave_lista_itens_api_identificada = key_attempt
                                print(f"Chave de dados JSON identificada para a lista de itens: '{chave_lista_itens_api_identificada}'")
                                break
                        if not chave_lista_itens_api_identificada:
                            print(f"ATENÇÃO: Não foi possível identificar a chave da lista de itens para '{endpoint_display_name}' na resposta da API.")
                            print("Resposta da API (primeira página):", json.dumps(dados_api_pagina, indent=2, ensure_ascii=False))
                            total_de_paginas_api = 0 
                            break 

                if sucesso_pagina and chave_lista_itens_api_identificada:
                    itens_da_pagina = dados_api_pagina.get(chave_lista_itens_api_identificada, [])
                    if itens_da_pagina:
                        todos_os_itens_lista.extend(itens_da_pagina)
                        print(f"  {len(itens_da_pagina)} itens adicionados da página {pagina_atual}. Total acumulado: {len(todos_os_itens_lista)}")
                    else:
                        print(f"  Nenhum item encontrado na página {pagina_atual} (usando chave '{chave_lista_itens_api_identificada}').")
                elif sucesso_pagina and not chave_lista_itens_api_identificada and total_de_registros_api > 0:
                    print(f"  Sucesso ao buscar página {pagina_atual}, mas a chave da lista de itens ainda não foi identificada. Isso não deveria acontecer.")
                    total_de_paginas_api = 0
                    break


            except (requests.exceptions.ConnectionError, requests.exceptions.Timeout, requests.exceptions.HTTPError) as e:
                print(f"  Erro ao buscar página {pagina_atual} (tentativa {tentativa_atual}): {e}")
                if tentativa_atual < max_tentativas_por_pagina:
                    delay_atual = delay_entre_tentativas_base * (2 ** (tentativa_atual - 1))
                    print(f"  Aguardando {delay_atual} segundos antes de tentar novamente...")
                    time.sleep(delay_atual)
                else:
                    print(f"  Máximo de {max_tentativas_por_pagina} tentativas atingido para a página {pagina_atual}. Desistindo desta página.")
                    break 

        if not sucesso_pagina and total_de_paginas_api > 0 :
             print(f"AVISO: Falha ao processar a página {pagina_atual} após todas as tentativas. Alguns dados podem estar faltando.")

        if pagina_atual < total_de_paginas_api and total_de_paginas_api > 0:
            time.sleep(1) 
        
        if total_de_paginas_api == 0: 
            break

        pagina_atual += 1

    print(f"\nExtração de todas as páginas para '{endpoint_display_name}' concluída (ou tentativas esgotadas).")

    # --- INÍCIO DA LIMPEZA E TRANSFORMAÇÃO ---
    if not todos_os_itens_lista:
        print(f"Nenhum item de '{endpoint_display_name}' foi extraído da API para processar.")
    else:
        print(f"Total de {len(todos_os_itens_lista)} registros de '{endpoint_display_name}' extraídos para processamento.")

        df_itens = pd.json_normalize(todos_os_itens_lista)
        
        print("\n--- DataFrame Inicial (após json_normalize) ---")
        if not df_itens.empty:
            print(df_itens.head())
        else:
            print("DataFrame inicial está vazio.")
        
        df_processado = df_itens.replace('', pd.NA)
        df_processado = df_processado.fillna('N/A')

        print("\n--- DataFrame após preencher valores nulos/vazios ---")
        if not df_processado.empty:
            print(df_processado.head())
        else:
            print("DataFrame processado está vazio.")


        print("\n--- Colunas Disponíveis para Seleção/Renomeação ---")
        if not df_processado.empty:
            colunas_disponiveis = df_processado.columns.tolist()
            for col_name in colunas_disponiveis:
                print(f"- {col_name}")
        else:
            print("Nenhuma coluna disponível, DataFrame está vazio.")

        # --- AJUSTE AQUI: Defina as colunas que quer manter e os novos nomes para DEPARTAMENTOS ---
        # Este é um EXEMPLO. Você PRECISARÁ ajustar com base na saída de "Colunas Disponíveis"
        # que o script gerar com os dados reais de "Departamentos".
        mapa_renomeacao_e_selecao = {
            # Exemplo de colunas comuns para Departamentos (VERIFIQUE A SAÍDA REAL DA API):
            'codigo': 'ID Dept Omie',          # Nome hipotético, verifique o real
            'descricao': 'Descrição Dept',    # Nome hipotético
            'cSimples': 'Simples Nacional (S/N Dept)', # Nome hipotético
            'inativo': 'Inativo (S/N Dept)',   # Nome hipotético
            'estrutura': 'Estrutura Dept',     # Nome hipotético
            # Se houver um campo 'info' ou similar para datas de inclusão/alteração:
            # 'info.dInc': 'Data Inclusão Registro Dept',
            # 'info.dAlt': 'Data Alteração Registro Dept'
        }

        df_final = pd.DataFrame() 
        if not df_processado.empty:
            colunas_para_manter_renomeadas = {
                antigo: novo for antigo, novo in mapa_renomeacao_e_selecao.items() if antigo in df_processado.columns
            }
            colunas_antigas_selecionadas = list(colunas_para_manter_renomeadas.keys())

            if not colunas_antigas_selecionadas:
                print(f"\nAVISO: Nenhuma coluna do 'mapa_renomeacao_e_selecao' para '{endpoint_display_name}' foi encontrada no DataFrame.")
                print("O DataFrame final conterá todas as colunas originais (após preenchimento de nulos).")
                df_final = df_processado.copy() 
            else:
                df_final = df_processado[colunas_antigas_selecionadas].rename(columns=colunas_para_manter_renomeadas)

                # --- AJUSTE AQUI: Defina a ordem desejada para as colunas de DEPARTAMENTOS ---
                # Exemplo de ordem (use os NOVOS NOMES definidos no mapa acima)
                ordem_desejada = [
                    'ID Dept Omie', 'Descrição Dept', 'Estrutura Dept',
                    'Simples Nacional (S/N Dept)', 'Inativo (S/N Dept)',
                    # 'Data Inclusão Registro Dept', 'Data Alteração Registro Dept'
                ]
                ordem_existente = [col for col in ordem_desejada if col in df_final.columns]
                for col in df_final.columns: 
                    if col not in ordem_existente:
                        ordem_existente.append(col)
                if ordem_existente: 
                    df_final = df_final[ordem_existente]
        else:
            print("DataFrame processado está vazio, não é possível criar df_final com colunas selecionadas.")


        print("\n--- DataFrame Final (Limpado, Selecionado e Renomeado) ---")
        if not df_final.empty:
            print(df_final.head())
            print(f"\nTotal de {len(df_final)} itens de '{endpoint_display_name}' no DataFrame final.")

            # Salvar em arquivo CSV
            df_final.to_csv(nome_arquivo_saida_csv, index=False, sep=';', encoding='utf-8-sig')
            print(f"\nDataFrame final salvo em: {nome_arquivo_saida_csv}")

            # Opcional: Salvar em arquivo Excel
            # df_final.to_excel(nome_arquivo_saida_excel, index=False)
            # print(f"\nDataFrame final salvo em: {nome_arquivo_saida_excel}")
        else:
            print("DataFrame final está vazio. Nenhum arquivo CSV será gerado.")


except requests.exceptions.HTTPError as http_err:
    print(f"Erro HTTP ocorreu durante a chamada à API para '{endpoint_display_name}': {http_err}")
    if 'response' in locals() and response is not None:
        print(f"Conteúdo da resposta: {response.text}")
except requests.exceptions.ConnectionError as conn_err:
    print(f"Erro de Conexão ocorreu durante a chamada à API para '{endpoint_display_name}': {conn_err}")
except requests.exceptions.Timeout as timeout_err:
    print(f"Timeout ocorreu durante a chamada à API para '{endpoint_display_name}': {timeout_err}")
except requests.exceptions.RequestException as req_err:
    print(f"Erro de Requisição ocorreu durante a chamada à API para '{endpoint_display_name}': {req_err}")
except json.JSONDecodeError as json_err:
    print(f"Erro ao decodificar JSON da resposta da API para '{endpoint_display_name}': {json_err}")
    if 'response' in locals() and response is not None:
        print(f"Conteúdo da resposta que causou o erro: {response.text}")
except Exception as e:
    print(f"Um erro inesperado ocorreu durante o processamento de '{endpoint_display_name}': {e}")

Iniciando extração de 'Departamentos' da API Omie. Registros por página: 50
URL Base: https://app.omie.com.br/api/v1/geral/departamentos/
Buscando página 1 de ... para 'Departamentos' (Tentativa 1/3)...
API informou: 8 página(s) e 371 registro(s) no total para 'Departamentos'.
Chave de dados JSON identificada para a lista de itens: 'departamentos'
  50 itens adicionados da página 1. Total acumulado: 50
Buscando página 2 de 8 para 'Departamentos' (Tentativa 1/3)...
  50 itens adicionados da página 2. Total acumulado: 100
Buscando página 3 de 8 para 'Departamentos' (Tentativa 1/3)...
  50 itens adicionados da página 3. Total acumulado: 150
Buscando página 4 de 8 para 'Departamentos' (Tentativa 1/3)...
  50 itens adicionados da página 4. Total acumulado: 200
Buscando página 5 de 8 para 'Departamentos' (Tentativa 1/3)...
  50 itens adicionados da página 5. Total acumulado: 250
Buscando página 6 de 8 para 'Departamentos' (Tentativa 1/3)...
  50 itens adicionados da página 6. Total acumulado

In [13]:

APP_KEY = "596450736882"
APP_SECRET = "37a10edb420d8cd6c7afddcaaffcac05"

# --- CONFIGURAÇÃO PARA EXTRAÇÃO DE CATEGORIAS ---
endpoint_name = "Categorias" # Nome da categoria para exibição e uso no DataFrame
base_module = "geral"        # Módulo para Categorias na Omie API, conforme sua URL
api_path = "categorias"      # Parte final da URL do endpoint
call_method = "ListarCategorias" # Nome do método de chamada da API
# Chave que contém a lista de categorias na resposta JSON (pode variar, verificar docs Omie)
# Tentaremos 'categorias_cadastro', 'listaCategorias', 'listaRegistros' ou 'categorias'
json_data_keys_to_try = ["categorias_cadastro", "listaCategorias", "listaRegistros", "categorias"]


base_url = f"https://app.omie.com.br/api/v1/{base_module}/{api_path}/" 

headers = {
    'Content-Type': 'application/json'
}

# Parâmetros da requisição para Categorias
body = {
    "call": call_method,
    "app_key": APP_KEY,
    "app_secret": APP_SECRET,
    "param": [
        {
            "pagina": 1,
            "registros_por_pagina": 50 # Você especificou 50 aqui
        }
    ]
}

print(f"Iniciando extração de dados para: {endpoint_name}")
print(f"URL da Requisição: {base_url}")
print(f"Método da Requisição (call): {call_method}")
print("-" * 50)

df_categorias = pd.DataFrame() # DataFrame vazio para armazenar os dados de categorias

try:
    response = requests.post(base_url, headers=headers, data=json.dumps(body))
    response.raise_for_status() # Lança uma exceção para códigos de status de erro (4xx ou 5xx)

    print(f"Dados de {endpoint_name} extraídos com sucesso!")
    
    data = response.json()

    # Tentar extrair a lista de categorias da resposta JSON usando as chaves prováveis
    found_key = None
    for key_attempt in json_data_keys_to_try:
        if key_attempt in data and isinstance(data.get(key_attempt), list):
            df_categorias = pd.DataFrame(data[key_attempt])
            found_key = key_attempt
            break # Sair do loop assim que a chave for encontrada

    if found_key:
        print(f"Chave de dados JSON identificada: '{found_key}'")
        print(f"\n--- Análise Exploratória para {endpoint_name} ---")
        print(f"Shape do DataFrame: {df_categorias.shape}")
        print("\nPrimeiras 5 linhas:")
        print(df_categorias.head())

        print("\nInformações do DataFrame (tipos de dados e contagem de não-nulos):")
        buffer = io.StringIO()
        df_categorias.info(buf=buffer, verbose=True, show_counts=True)
        print(buffer.getvalue())

        print("\nContagem de Valores Nulos por Coluna:")
        print(df_categorias.isnull().sum())
        print("-" * 50)
    else:
        print(f"Atenção: Não foi possível identificar a chave dos dados para '{endpoint_name}' na resposta da API.")
        print("Resposta completa da API (para depuração):", json.dumps(data, indent=2, ensure_ascii=False))
        print("-" * 50)

except requests.exceptions.HTTPError as http_err:
    print(f"Erro HTTP ao extrair {endpoint_name}: {http_err}")
    print(f"Resposta da API: {response.text}")
    print("-" * 50)
except requests.exceptions.ConnectionError as conn_err:
    print(f"Erro de Conexão ao extrair {endpoint_name}: {conn_err}")
    print("Verifique sua conexão com a internet ou o status da API Omie.")
    print("-" * 50)
except requests.exceptions.Timeout as timeout_err:
    print(f"Erro de Timeout ao extrair {endpoint_name}: {timeout_err}")
    print("A requisição demorou muito para responder.")
    print("-" * 50)
except requests.exceptions.RequestException as req_err:
    print(f"Erro inesperado ao extrair {endpoint_name}: {req_err}")
    print("-" * 50)
except json.JSONDecodeError:
    print(f"Erro ao decodificar JSON para {endpoint_name}. Resposta não é JSON válido.")
    print(f"Resposta da API: {response.text}")
    print("-" * 50)
except Exception as e:
    print(f"Ocorreu um erro geral ao processar {endpoint_name}: {e}")
    print("-" * 50)

print("\n--- Extração de Categorias concluída! ---")

# Agora você tem o DataFrame 'df_categorias' com os dados extraídos.
# Você pode fazer mais análises ou salvá-lo:
# df_categorias.to_csv('categorias_omie.csv', index=False, encoding='utf-8')
# df_categorias.to_excel('categorias_omie.xlsx', index=False)

Iniciando extração de dados para: Categorias
URL da Requisição: https://app.omie.com.br/api/v1/geral/categorias/
Método da Requisição (call): ListarCategorias
--------------------------------------------------
Dados de Categorias extraídos com sucesso!
Atenção: Não foi possível identificar a chave dos dados para 'Categorias' na resposta da API.
Resposta completa da API (para depuração): {
  "pagina": 1,
  "total_de_paginas": 6,
  "registros": 50,
  "total_de_registros": 256,
  "categoria_cadastro": [
    {
      "categoria_superior": "0",
      "codigo": "0.01",
      "codigo_dre": "",
      "conta_despesa": "N",
      "conta_inativa": "N",
      "conta_receita": "N",
      "dadosDRE": {},
      "definida_pelo_usuario": "N",
      "descricao": "Transferência",
      "descricao_padrao": "Transferência",
      "id_conta_contabil": "",
      "nao_exibir": "S",
      "natureza": "",
      "tag_conta_contabil": "",
      "tipo_categoria": "",
      "totalizadora": "S",
      "transferencia"

In [14]:

APP_KEY = "596450736882"
APP_SECRET = "37a10edb420d8cd6c7afddcaaffcac05"

# --- CONFIGURAÇÕES ESPECÍFICAS PARA O ENDPOINT DE CATEGORIAS ---
endpoint_display_name = "Categorias" # Nome para exibição
base_module = "geral"
api_path = "categorias"
api_call_name = "ListarCategorias"

# Chaves prováveis no JSON da API que contêm a lista de itens (categorias)
# O script tentará estas chaves em ordem.
json_data_keys_to_try = ["categorias_cadastro", "listaCategorias", "listaRegistros", "categorias", "cadastro"] # Adicionado "cadastro" como uma possibilidade

# Nome do arquivo de saída
nome_arquivo_saida_csv = 'omie_categorias_limpas_todas.csv'
nome_arquivo_saida_excel = 'omie_categorias_limpas_todas.xlsx'
# -----------------------------------------------------------------

versao = "v1" # Geralmente v1 para endpoints gerais da Omie
base_url = f"https://app.omie.com.br/api/{versao}/{base_module}/{api_path}/"

headers = {
    'Content-Type': 'application/json'
}

# Parâmetros iniciais
pagina_atual = 1
registros_por_pagina = 50 # Conforme seu exemplo para categorias
max_tentativas_por_pagina = 3
delay_entre_tentativas_base = 5

print(f"Iniciando extração de '{endpoint_display_name}' da API Omie. Registros por página: {registros_por_pagina}")
print(f"URL Base: {base_url}")

todos_os_itens_lista = []
total_de_paginas_api = 1
chave_lista_itens_api_identificada = None

try:
    while pagina_atual <= total_de_paginas_api:
        body = {
            'call': api_call_name,
            'app_key': APP_KEY,
            'app_secret': APP_SECRET,
            'param': [
                {
                    "pagina": pagina_atual,
                    "registros_por_pagina": registros_por_pagina
                    # Verificar se "apenas_importado_api": "N" é aplicável/necessário para ListarCategorias
                }
            ]
        }

        tentativa_atual = 0
        sucesso_pagina = False
        dados_api_pagina = None # Inicializa para o escopo do loop de tentativas

        while tentativa_atual < max_tentativas_por_pagina and not sucesso_pagina:
            tentativa_atual += 1
            print(f"Buscando página {pagina_atual} de {total_de_paginas_api if total_de_paginas_api > 1 else '...' } para '{endpoint_display_name}' (Tentativa {tentativa_atual}/{max_tentativas_por_pagina})...")
            
            try:
                response = requests.post(base_url, headers=headers, data=json.dumps(body), timeout=30)
                response.raise_for_status()
                
                dados_api_pagina = response.json()
                sucesso_pagina = True

                if pagina_atual == 1 and tentativa_atual == 1: # Apenas na primeira tentativa bem-sucedida da primeira página
                    total_de_paginas_api = dados_api_pagina.get('total_de_paginas', 1)
                    total_de_registros_api = dados_api_pagina.get('total_de_registros', 0)
                    print(f"API informou: {total_de_paginas_api} página(s) e {total_de_registros_api} registro(s) no total para '{endpoint_display_name}'.")
                    
                    if total_de_registros_api == 0:
                        print(f"Nenhum registro encontrado na API para '{endpoint_display_name}'.")
                        total_de_paginas_api = 0 
                        break # Sai do loop de tentativas, e o loop principal também sairá
                    
                    # Tenta identificar a chave correta para a lista de itens
                    if not chave_lista_itens_api_identificada:
                        for key_attempt in json_data_keys_to_try:
                            if key_attempt in dados_api_pagina and isinstance(dados_api_pagina.get(key_attempt), list):
                                chave_lista_itens_api_identificada = key_attempt
                                print(f"Chave de dados JSON identificada para a lista de itens: '{chave_lista_itens_api_identificada}'")
                                break
                        if not chave_lista_itens_api_identificada:
                            print(f"ATENÇÃO: Não foi possível identificar a chave da lista de itens para '{endpoint_display_name}' na resposta da API.")
                            print("Resposta da API (primeira página):", json.dumps(dados_api_pagina, indent=2, ensure_ascii=False))
                            total_de_paginas_api = 0 
                            break 

                if sucesso_pagina and chave_lista_itens_api_identificada:
                    itens_da_pagina = dados_api_pagina.get(chave_lista_itens_api_identificada, [])
                    if itens_da_pagina:
                        todos_os_itens_lista.extend(itens_da_pagina)
                        print(f"  {len(itens_da_pagina)} itens adicionados da página {pagina_atual}. Total acumulado: {len(todos_os_itens_lista)}")
                    else:
                        print(f"  Nenhum item encontrado na página {pagina_atual} (usando chave '{chave_lista_itens_api_identificada}').")
                elif sucesso_pagina and not chave_lista_itens_api_identificada and total_de_registros_api > 0:
                    print(f"  Sucesso ao buscar página {pagina_atual}, mas a chave da lista de itens ainda não foi identificada. Isso não deveria acontecer.")
                    total_de_paginas_api = 0
                    break


            except (requests.exceptions.ConnectionError, requests.exceptions.Timeout, requests.exceptions.HTTPError) as e:
                print(f"  Erro ao buscar página {pagina_atual} (tentativa {tentativa_atual}): {e}")
                if tentativa_atual < max_tentativas_por_pagina:
                    delay_atual = delay_entre_tentativas_base * (2 ** (tentativa_atual - 1))
                    print(f"  Aguardando {delay_atual} segundos antes de tentar novamente...")
                    time.sleep(delay_atual)
                else:
                    print(f"  Máximo de {max_tentativas_por_pagina} tentativas atingido para a página {pagina_atual}. Desistindo desta página.")
                    break 

        if not sucesso_pagina and total_de_paginas_api > 0 :
             print(f"AVISO: Falha ao processar a página {pagina_atual} após todas as tentativas. Alguns dados podem estar faltando.")

        if pagina_atual < total_de_paginas_api and total_de_paginas_api > 0:
            time.sleep(1) 
        
        if total_de_paginas_api == 0: 
            break

        pagina_atual += 1

    print(f"\nExtração de todas as páginas para '{endpoint_display_name}' concluída (ou tentativas esgotadas).")

    # --- INÍCIO DA LIMPEZA E TRANSFORMAÇÃO ---
    if not todos_os_itens_lista:
        print(f"Nenhum item de '{endpoint_display_name}' foi extraído da API para processar.")
    else:
        print(f"Total de {len(todos_os_itens_lista)} registros de '{endpoint_display_name}' extraídos para processamento.")

        df_itens = pd.json_normalize(todos_os_itens_lista)
        
        print("\n--- DataFrame Inicial (após json_normalize) ---")
        if not df_itens.empty:
            print(df_itens.head())
        else:
            print("DataFrame inicial está vazio.")
        
        df_processado = df_itens.replace('', pd.NA)
        df_processado = df_processado.fillna('N/A')

        print("\n--- DataFrame após preencher valores nulos/vazios ---")
        if not df_processado.empty:
            print(df_processado.head())
        else:
            print("DataFrame processado está vazio.")


        print("\n--- Colunas Disponíveis para Seleção/Renomeação ---")
        if not df_processado.empty:
            colunas_disponiveis = df_processado.columns.tolist()
            for col_name in colunas_disponiveis:
                print(f"- {col_name}")
        else:
            print("Nenhuma coluna disponível, DataFrame está vazio.")

        # --- AJUSTE AQUI: Defina as colunas que quer manter e os novos nomes para CATEGORIAS ---
        # Este é um EXEMPLO. Você PRECISARÁ ajustar com base na saída de "Colunas Disponíveis"
        # que o script gerar com os dados reais de "Categorias".
        mapa_renomeacao_e_selecao = {
            # Exemplo de colunas comuns para Categorias (VERIFIQUE A SAÍDA REAL DA API):
            'codigo': 'ID Categoria Omie',      # Nome hipotético
            'descricao': 'Descrição Categoria', # Nome hipotético
            'tipo': 'Tipo Categoria',           # Nome hipotético (ex: Receita, Despesa)
            'natureza': 'Natureza Categoria',   # Nome hipotético
            'conta_contabil': 'Conta Contábil Categoria', # Nome hipotético
            'inativo': 'Inativo (S/N Categoria)', # Nome hipotético
            # Se houver um campo 'info' ou similar para datas de inclusão/alteração:
            # 'info.dInc': 'Data Inclusão Registro Categoria',
            # 'info.dAlt': 'Data Alteração Registro Categoria'
        }

        df_final = pd.DataFrame() 
        if not df_processado.empty:
            colunas_para_manter_renomeadas = {
                antigo: novo for antigo, novo in mapa_renomeacao_e_selecao.items() if antigo in df_processado.columns
            }
            colunas_antigas_selecionadas = list(colunas_para_manter_renomeadas.keys())

            if not colunas_antigas_selecionadas:
                print(f"\nAVISO: Nenhuma coluna do 'mapa_renomeacao_e_selecao' para '{endpoint_display_name}' foi encontrada no DataFrame.")
                print("O DataFrame final conterá todas as colunas originais (após preenchimento de nulos).")
                df_final = df_processado.copy() 
            else:
                df_final = df_processado[colunas_antigas_selecionadas].rename(columns=colunas_para_manter_renomeadas)

                # --- AJUSTE AQUI: Defina a ordem desejada para as colunas de CATEGORIAS ---
                # Exemplo de ordem (use os NOVOS NOMES definidos no mapa acima)
                ordem_desejada = [
                    'ID Categoria Omie', 'Descrição Categoria', 'Tipo Categoria', 'Natureza Categoria',
                    'Conta Contábil Categoria', 'Inativo (S/N Categoria)',
                    # 'Data Inclusão Registro Categoria', 'Data Alteração Registro Categoria'
                ]
                ordem_existente = [col for col in ordem_desejada if col in df_final.columns]
                for col in df_final.columns: 
                    if col not in ordem_existente:
                        ordem_existente.append(col)
                if ordem_existente: 
                    df_final = df_final[ordem_existente]
        else:
            print("DataFrame processado está vazio, não é possível criar df_final com colunas selecionadas.")


        print("\n--- DataFrame Final (Limpado, Selecionado e Renomeado) ---")
        if not df_final.empty:
            print(df_final.head())
            print(f"\nTotal de {len(df_final)} itens de '{endpoint_display_name}' no DataFrame final.")

            # Salvar em arquivo CSV
            df_final.to_csv(nome_arquivo_saida_csv, index=False, sep=';', encoding='utf-8-sig')
            print(f"\nDataFrame final salvo em: {nome_arquivo_saida_csv}")

            # Opcional: Salvar em arquivo Excel
            # df_final.to_excel(nome_arquivo_saida_excel, index=False)
            # print(f"\nDataFrame final salvo em: {nome_arquivo_saida_excel}")
        else:
            print("DataFrame final está vazio. Nenhum arquivo CSV será gerado.")


except requests.exceptions.HTTPError as http_err:
    print(f"Erro HTTP ocorreu durante a chamada à API para '{endpoint_display_name}': {http_err}")
    if 'response' in locals() and response is not None:
        print(f"Conteúdo da resposta: {response.text}")
except requests.exceptions.ConnectionError as conn_err:
    print(f"Erro de Conexão ocorreu durante a chamada à API para '{endpoint_display_name}': {conn_err}")
except requests.exceptions.Timeout as timeout_err:
    print(f"Timeout ocorreu durante a chamada à API para '{endpoint_display_name}': {timeout_err}")
except requests.exceptions.RequestException as req_err:
    print(f"Erro de Requisição ocorreu durante a chamada à API para '{endpoint_display_name}': {req_err}")
except json.JSONDecodeError as json_err:
    print(f"Erro ao decodificar JSON da resposta da API para '{endpoint_display_name}': {json_err}")
    if 'response' in locals() and response is not None:
        print(f"Conteúdo da resposta que causou o erro: {response.text}")
except Exception as e:
    print(f"Um erro inesperado ocorreu durante o processamento de '{endpoint_display_name}': {e}")


Iniciando extração de 'Categorias' da API Omie. Registros por página: 50
URL Base: https://app.omie.com.br/api/v1/geral/categorias/
Buscando página 1 de ... para 'Categorias' (Tentativa 1/3)...
API informou: 6 página(s) e 256 registro(s) no total para 'Categorias'.
ATENÇÃO: Não foi possível identificar a chave da lista de itens para 'Categorias' na resposta da API.
Resposta da API (primeira página): {
  "pagina": 1,
  "total_de_paginas": 6,
  "registros": 50,
  "total_de_registros": 256,
  "categoria_cadastro": [
    {
      "categoria_superior": "0",
      "codigo": "0.01",
      "codigo_dre": "",
      "conta_despesa": "N",
      "conta_inativa": "N",
      "conta_receita": "N",
      "dadosDRE": {},
      "definida_pelo_usuario": "N",
      "descricao": "Transferência",
      "descricao_padrao": "Transferência",
      "id_conta_contabil": "",
      "nao_exibir": "S",
      "natureza": "",
      "tag_conta_contabil": "",
      "tipo_categoria": "",
      "totalizadora": "S",
      "t

In [15]:

APP_KEY = "596450736882"
APP_SECRET = "37a10edb420d8cd6c7afddcaaffcac05"

# --- CONFIGURAÇÃO PARA EXTRAÇÃO DE CONTAS CORRENTES ---
endpoint_name = "Contas Correntes" # Nome da categoria para exibição e uso no DataFrame
base_module = "geral"            # Módulo para Contas Correntes na Omie API, conforme sua URL
api_path = "contacorrente"       # Parte final da URL do endpoint
call_method = "ListarContasCorrentes" # Nome do método de chamada da API
# Chave que contém a lista de contas correntes na resposta JSON (pode variar, verificar docs Omie)
# Tentaremos 'contaCorrenteCadastro', 'listaContasCorrentes', 'listaRegistros' ou 'contasCorrentes'
json_data_keys_to_try = ["contaCorrenteCadastro", "listaContasCorrentes", "listaRegistros", "contasCorrentes"]


base_url = f"https://app.omie.com.br/api/v1/{base_module}/{api_path}/" 

headers = {
    'Content-Type': 'application/json'
}

# Parâmetros da requisição para Contas Correntes
body = {
    "call": call_method,
    "app_key": APP_KEY,
    "app_secret": APP_SECRET,
    "param": [
        {
            "pagina": 1,
            "registros_por_pagina": 100, # Você especificou 100 aqui
            "apenas_importado_api": "N"
        }
    ]
}

print(f"Iniciando extração de dados para: {endpoint_name}")
print(f"URL da Requisição: {base_url}")
print(f"Método da Requisição (call): {call_method}")
print("-" * 50)

df_contas_correntes = pd.DataFrame() # DataFrame vazio para armazenar os dados de contas correntes

try:
    response = requests.post(base_url, headers=headers, data=json.dumps(body))
    response.raise_for_status() # Lança uma exceção para códigos de status de erro (4xx ou 5xx)

    print(f"Dados de {endpoint_name} extraídos com sucesso!")
    
    data = response.json()

    # Tentar extrair a lista de contas correntes da resposta JSON usando as chaves prováveis
    found_key = None
    for key_attempt in json_data_keys_to_try:
        if key_attempt in data and isinstance(data.get(key_attempt), list):
            df_contas_correntes = pd.DataFrame(data[key_attempt])
            found_key = key_attempt
            break # Sair do loop assim que a chave for encontrada

    if found_key:
        print(f"Chave de dados JSON identificada: '{found_key}'")
        print(f"\n--- Análise Exploratória para {endpoint_name} ---")
        print(f"Shape do DataFrame: {df_contas_correntes.shape}")
        print("\nPrimeiras 5 linhas:")
        print(df_contas_correntes.head())

        print("\nInformações do DataFrame (tipos de dados e contagem de não-nulos):")
        buffer = io.StringIO()
        df_contas_correntes.info(buf=buffer, verbose=True, show_counts=True)
        print(buffer.getvalue())

        print("\nContagem de Valores Nulos por Coluna:")
        print(df_contas_correntes.isnull().sum())
        print("-" * 50)
    else:
        print(f"Atenção: Não foi possível identificar a chave dos dados para '{endpoint_name}' na resposta da API.")
        print("Resposta completa da API (para depuração):", json.dumps(data, indent=2, ensure_ascii=False))
        print("-" * 50)

except requests.exceptions.HTTPError as http_err:
    print(f"Erro HTTP ao extrair {endpoint_name}: {http_err}")
    print(f"Resposta da API: {response.text}")
    print("-" * 50)
except requests.exceptions.ConnectionError as conn_err:
    print(f"Erro de Conexão ao extrair {endpoint_name}: {conn_err}")
    print("Verifique sua conexão com a internet ou o status da API Omie.")
    print("-" * 50)
except requests.exceptions.Timeout as timeout_err:
    print(f"Erro de Timeout ao extrair {endpoint_name}: {timeout_err}")
    print("A requisição demorou muito para responder.")
    print("-" * 50)
except requests.exceptions.RequestException as req_err:
    print(f"Erro inesperado ao extrair {endpoint_name}: {req_err}")
    print("-" * 50)
except json.JSONDecodeError:
    print(f"Erro ao decodificar JSON para {endpoint_name}. Resposta não é JSON válido.")
    print(f"Resposta da API: {response.text}")
    print("-" * 50)
except Exception as e:
    print(f"Ocorreu um erro geral ao processar {endpoint_name}: {e}")
    print("-" * 50)

print("\n--- Extração de Contas Correntes concluída! ---")

# Agora você tem o DataFrame 'df_contas_correntes' com os dados extraídos.
# Você pode fazer mais análises ou salvá-lo:
# df_contas_correntes.to_csv('contas_correntes_omie.csv', index=False, encoding='utf-8')
# df_contas_correntes.to_excel('contas_correntes_omie.xlsx', index=False)

Iniciando extração de dados para: Contas Correntes
URL da Requisição: https://app.omie.com.br/api/v1/geral/contacorrente/
Método da Requisição (call): ListarContasCorrentes
--------------------------------------------------
Dados de Contas Correntes extraídos com sucesso!
Atenção: Não foi possível identificar a chave dos dados para 'Contas Correntes' na resposta da API.
Resposta completa da API (para depuração): {
  "pagina": 1,
  "total_de_paginas": 1,
  "registros": 63,
  "total_de_registros": 63,
  "ListarContasCorrentes": [
    {
      "bairro": "",
      "bloqueado": "N",
      "bol_instr1": "Sr. Caixa, receber até 10 dias após o vencimento.",
      "bol_instr2": "",
      "bol_instr3": "",
      "bol_instr4": "",
      "bol_sn": "N",
      "cCodCCInt": "",
      "cEstabelecimento": "",
      "cTipoCartao": "",
      "cancinstr": "",
      "cep": "",
      "cidade": "",
      "cnab_esp": "",
      "cobr_esp": "",
      "cobr_sn": "N",
      "codigo_agencia": "6613",
      "codigo_

In [16]:

APP_KEY = "596450736882"
APP_SECRET = "37a10edb420d8cd6c7afddcaaffcac05"

# --- CONFIGURAÇÕES ESPECÍFICAS PARA O ENDPOINT DE CONTAS CORRENTES ---
endpoint_display_name = "Contas Correntes" # Nome para exibição
base_module = "geral"
api_path = "contacorrente" # Conforme seu exemplo
api_call_name = "ListarContasCorrentes"

# Chaves prováveis no JSON da API que contêm a lista de itens (contas correntes)
# O script tentará estas chaves em ordem.
# Com base no seu log, "ListarContasCorrentes" parece ser a chave que contém a lista diretamente.
json_data_keys_to_try = [api_call_name, "contaCorrenteCadastro", "listaContasCorrentes", "listaRegistros", "contasCorrentes", "cadastro"]

# Nome do arquivo de saída
nome_arquivo_saida_csv = 'omie_contas_correntes_limpas_todas.csv'
nome_arquivo_saida_excel = 'omie_contas_correntes_limpas_todas.xlsx'
# -----------------------------------------------------------------

versao = "v1" # Geralmente v1 para endpoints gerais da Omie
base_url = f"https://app.omie.com.br/api/{versao}/{base_module}/{api_path}/"

headers = {
    'Content-Type': 'application/json'
}

# Parâmetros iniciais
pagina_atual = 1
registros_por_pagina = 100 # Conforme seu exemplo para Contas Correntes
max_tentativas_por_pagina = 3
delay_entre_tentativas_base = 5

print(f"Iniciando extração de '{endpoint_display_name}' da API Omie. Registros por página: {registros_por_pagina}")
print(f"URL Base: {base_url}")

todos_os_itens_lista = []
total_de_paginas_api = 1
chave_lista_itens_api_identificada = None # Será definida na primeira requisição bem-sucedida da página 1

try:
    while pagina_atual <= total_de_paginas_api:
        body = {
            'call': api_call_name,
            'app_key': APP_KEY,
            'app_secret': APP_SECRET,
            'param': [
                {
                    "pagina": pagina_atual,
                    "registros_por_pagina": registros_por_pagina,
                    "apenas_importado_api": "N" # Conforme seu exemplo para Contas Correntes
                }
            ]
        }

        tentativa_atual = 0
        sucesso_pagina = False
        dados_api_pagina = None 

        while tentativa_atual < max_tentativas_por_pagina and not sucesso_pagina:
            tentativa_atual += 1
            print(f"Buscando página {pagina_atual} de {total_de_paginas_api if total_de_paginas_api > 1 else '...' } para '{endpoint_display_name}' (Tentativa {tentativa_atual}/{max_tentativas_por_pagina})...")
            
            try:
                response = requests.post(base_url, headers=headers, data=json.dumps(body), timeout=30)
                response.raise_for_status()
                
                dados_api_pagina = response.json()
                sucesso_pagina = True # Requisição HTTP foi bem-sucedida

                # Lógica de identificação da chave e total de páginas (APENAS na primeira página BEM-SUCEDIDA)
                if pagina_atual == 1 and chave_lista_itens_api_identificada is None: # Só roda uma vez
                    total_de_paginas_api = dados_api_pagina.get('total_de_paginas', 1)
                    total_de_registros_api = dados_api_pagina.get('total_de_registros', 0)
                    print(f"API informou: {total_de_paginas_api} página(s) e {total_de_registros_api} registro(s) no total para '{endpoint_display_name}'.")
                    
                    if total_de_registros_api == 0:
                        print(f"Nenhum registro encontrado na API para '{endpoint_display_name}'.")
                        total_de_paginas_api = 0 # Para o loop principal
                        break # Sai do loop de tentativas, pois não há dados
                    
                    # Tenta identificar a chave correta para a lista de itens
                    for key_attempt in json_data_keys_to_try:
                        if key_attempt in dados_api_pagina and isinstance(dados_api_pagina.get(key_attempt), list):
                            chave_lista_itens_api_identificada = key_attempt
                            print(f"Chave de dados JSON identificada para a lista de itens: '{chave_lista_itens_api_identificada}'")
                            break
                    
                    if not chave_lista_itens_api_identificada:
                        print(f"ATENÇÃO: Não foi possível identificar a chave da lista de itens para '{endpoint_display_name}' na resposta da API.")
                        print("Resposta da API (primeira página):", json.dumps(dados_api_pagina, indent=2, ensure_ascii=False))
                        total_de_paginas_api = 0 # Para o loop principal
                        break # Sai do loop de tentativas

                # Extração dos itens da página atual
                if sucesso_pagina and chave_lista_itens_api_identificada:
                    itens_da_pagina = dados_api_pagina.get(chave_lista_itens_api_identificada, [])
                    if itens_da_pagina:
                        todos_os_itens_lista.extend(itens_da_pagina)
                        print(f"  {len(itens_da_pagina)} itens adicionados da página {pagina_atual}. Total acumulado: {len(todos_os_itens_lista)}")
                    else:
                        print(f"  Nenhum item encontrado na página {pagina_atual} (usando chave '{chave_lista_itens_api_identificada}').")
                elif sucesso_pagina and not chave_lista_itens_api_identificada and total_de_registros_api > 0 and pagina_atual == 1 :
                    # Esta condição pode ser alcançada se a chave não foi identificada mesmo após a lógica acima
                    print(f"  Sucesso ao buscar página {pagina_atual}, mas a chave da lista de itens não foi identificada. Verifique 'json_data_keys_to_try'.")
                    total_de_paginas_api = 0 # Parar se a chave não for encontrada na primeira página
                    break


            except (requests.exceptions.ConnectionError, requests.exceptions.Timeout, requests.exceptions.HTTPError) as e:
                print(f"  Erro ao buscar página {pagina_atual} (tentativa {tentativa_atual}): {e}")
                if tentativa_atual < max_tentativas_por_pagina:
                    delay_atual = delay_entre_tentativas_base * (2 ** (tentativa_atual - 1))
                    print(f"  Aguardando {delay_atual} segundos antes de tentar novamente...")
                    time.sleep(delay_atual)
                else:
                    print(f"  Máximo de {max_tentativas_por_pagina} tentativas atingido para a página {pagina_atual}. Desistindo desta página.")
                    break # Sai do loop de tentativas para esta página

        if not sucesso_pagina and total_de_paginas_api > 0 : # Se falhou em buscar a página após todas as tentativas
             print(f"AVISO: Falha ao processar a página {pagina_atual} após todas as tentativas. Alguns dados podem estar faltando.")
             # Decide se quer continuar para a próxima página ou parar. Por padrão, continua.

        if pagina_atual < total_de_paginas_api and total_de_paginas_api > 0:
            time.sleep(1) 
        
        if total_de_paginas_api == 0: # Se a API informou 0 registros ou a chave não foi encontrada na primeira página
            break

        pagina_atual += 1

    print(f"\nExtração de todas as páginas para '{endpoint_display_name}' concluída (ou tentativas esgotadas).")

    # --- INÍCIO DA LIMPEZA E TRANSFORMAÇÃO ---
    if not todos_os_itens_lista:
        print(f"Nenhum item de '{endpoint_display_name}' foi extraído da API para processar.")
    else:
        print(f"Total de {len(todos_os_itens_lista)} registros de '{endpoint_display_name}' extraídos para processamento.")

        df_itens = pd.json_normalize(todos_os_itens_lista)
        
        print("\n--- DataFrame Inicial (após json_normalize) ---")
        if not df_itens.empty:
            print(df_itens.head())
        else:
            print("DataFrame inicial está vazio.")
        
        df_processado = df_itens.replace('', pd.NA)
        df_processado = df_processado.fillna('N/A')

        print("\n--- DataFrame após preencher valores nulos/vazios ---")
        if not df_processado.empty:
            print(df_processado.head())
        else:
            print("DataFrame processado está vazio.")


        print("\n--- Colunas Disponíveis para Seleção/Renomeação ---")
        if not df_processado.empty:
            colunas_disponiveis = df_processado.columns.tolist()
            for col_name in colunas_disponiveis:
                print(f"- {col_name}")
        else:
            print("Nenhuma coluna disponível, DataFrame está vazio.")

        # --- AJUSTE AQUI: Defina as colunas que quer manter e os novos nomes para CONTAS CORRENTES ---
        # Este é um EXEMPLO. Você PRECISARÁ ajustar com base na saída de "Colunas Disponíveis"
        # que o script gerar com os dados reais de "Contas Correntes".
        mapa_renomeacao_e_selecao = {
            # Exemplo de colunas comuns para Contas Correntes (VERIFIQUE A SAÍDA REAL DA API):
            'nCodCC': 'ID Conta Omie', # Exemplo: nCodCC pode ser o código da conta
            'descricao': 'Descrição Conta',
            'tipo': 'Tipo Conta', # Exemplo: tipo pode ser 'BM' (Banco Movimento), 'CX' (Caixa)
            'cTipo': 'Tipo Detalhado', # Exemplo: cTipo pode ser 'CC' (Conta Corrente), 'CA' (Caixa)
            'codigo_banco': 'Código Banco',
            'agencia': 'Agência',
            'conta_corrente': 'Número Conta', # Ou 'numero_conta_corrente'
            'saldo_inicial': 'Saldo Inicial',
            'data_saldo': 'Data Saldo Inicial', # Ou 'data_saldo_inicial'
            'inativo': 'Inativo (S/N Conta)', # Ou 'bloqueado'
            'valor_limite': 'Valor Limite',
            'pdv_descricao': 'PDV Descrição',
            # Se houver um campo 'info' ou similar para datas de inclusão/alteração:
            # 'info.dInc': 'Data Inclusão Registro Conta',
            # 'info.dAlt': 'Data Alteração Registro Conta'
        }

        df_final = pd.DataFrame() 
        if not df_processado.empty:
            colunas_para_manter_renomeadas = {
                antigo: novo for antigo, novo in mapa_renomeacao_e_selecao.items() if antigo in df_processado.columns
            }
            colunas_antigas_selecionadas = list(colunas_para_manter_renomeadas.keys())

            if not colunas_antigas_selecionadas:
                print(f"\nAVISO: Nenhuma coluna do 'mapa_renomeacao_e_selecao' para '{endpoint_display_name}' foi encontrada no DataFrame.")
                print("O DataFrame final conterá todas as colunas originais (após preenchimento de nulos).")
                df_final = df_processado.copy() 
            else:
                df_final = df_processado[colunas_antigas_selecionadas].rename(columns=colunas_para_manter_renomeadas)

                # --- AJUSTE AQUI: Defina a ordem desejada para as colunas de CONTAS CORRENTES ---
                # Exemplo de ordem (use os NOVOS NOMES definidos no mapa acima)
                ordem_desejada = [
                    'ID Conta Omie', 'Descrição Conta', 'Tipo Conta', 'Tipo Detalhado', 'Código Banco', 'Agência', 'Número Conta',
                    'Saldo Inicial', 'Data Saldo Inicial', 'Valor Limite', 'PDV Descrição', 'Inativo (S/N Conta)'
                ]
                ordem_existente = [col for col in ordem_desejada if col in df_final.columns]
                for col in df_final.columns: 
                    if col not in ordem_existente:
                        ordem_existente.append(col)
                if ordem_existente: 
                    df_final = df_final[ordem_existente]
        else:
            print("DataFrame processado está vazio, não é possível criar df_final com colunas selecionadas.")


        print("\n--- DataFrame Final (Limpado, Selecionado e Renomeado) ---")
        if not df_final.empty:
            print(df_final.head())
            print(f"\nTotal de {len(df_final)} itens de '{endpoint_display_name}' no DataFrame final.")

            # Salvar em arquivo CSV
            df_final.to_csv(nome_arquivo_saida_csv, index=False, sep=';', encoding='utf-8-sig')
            print(f"\nDataFrame final salvo em: {nome_arquivo_saida_csv}")

            # Opcional: Salvar em arquivo Excel
            # df_final.to_excel(nome_arquivo_saida_excel, index=False)
            # print(f"\nDataFrame final salvo em: {nome_arquivo_saida_excel}")
        else:
            print("DataFrame final está vazio. Nenhum arquivo CSV será gerado.")


except requests.exceptions.HTTPError as http_err:
    print(f"Erro HTTP ocorreu durante a chamada à API para '{endpoint_display_name}': {http_err}")
    if 'response' in locals() and response is not None:
        print(f"Conteúdo da resposta: {response.text}")
except requests.exceptions.ConnectionError as conn_err: # Captura ConnectionError especificamente
    print(f"Erro de Conexão ocorreu durante a chamada à API para '{endpoint_display_name}': {conn_err}")
except requests.exceptions.Timeout as timeout_err: # Captura Timeout especificamente
    print(f"Timeout ocorreu durante a chamada à API para '{endpoint_display_name}': {timeout_err}")
except requests.exceptions.RequestException as req_err: # Captura outras exceções de requests
    print(f"Erro de Requisição ocorreu durante a chamada à API para '{endpoint_display_name}': {req_err}")
except json.JSONDecodeError as json_err:
    print(f"Erro ao decodificar JSON da resposta da API para '{endpoint_display_name}': {json_err}")
    if 'response' in locals() and response is not None: # Verifica se response existe
        print(f"Conteúdo da resposta que causou o erro: {response.text}")
except Exception as e:
    print(f"Um erro inesperado ocorreu durante o processamento de '{endpoint_display_name}': {e}")

Iniciando extração de 'Contas Correntes' da API Omie. Registros por página: 100
URL Base: https://app.omie.com.br/api/v1/geral/contacorrente/
Buscando página 1 de ... para 'Contas Correntes' (Tentativa 1/3)...
API informou: 1 página(s) e 63 registro(s) no total para 'Contas Correntes'.
Chave de dados JSON identificada para a lista de itens: 'ListarContasCorrentes'
  63 itens adicionados da página 1. Total acumulado: 63

Extração de todas as páginas para 'Contas Correntes' concluída (ou tentativas esgotadas).
Total de 63 registros de 'Contas Correntes' extraídos para processamento.

--- DataFrame Inicial (após json_normalize) ---
         bairro bloqueado                                         bol_instr1  \
0                       N  Sr. Caixa, receber até 10 dias após o vencimento.   
1          Brás         N  Sr. Caixa, receber até 10 dias após o vencimento.   
2  Vila Buarque         N  Sr. Caixa, receber até 10 dias após o vencimento.   
3    Alphaville         N  Sr. Caixa, receb

In [17]:

APP_KEY = "596450736882"
APP_SECRET = "37a10edb420d8cd6c7afddcaaffcac05"

# --- CONFIGURAÇÃO PARA EXTRAÇÃO DE CONTAS A PAGAR ---
endpoint_name = "Contas a Pagar" # Nome da categoria para exibição e uso no DataFrame
base_module = "financas"         # Módulo para Contas a Pagar na Omie API, conforme sua URL
api_path = "contapagar"          # Parte final da URL do endpoint
call_method = "ListarContasPagar" # Nome do método de chamada da API
# Chave que contém a lista de contas a pagar na resposta JSON (pode variar, verificar docs Omie)
# Tentaremos 'titulosPagar', 'listaContasPagar', 'listaRegistros' ou 'contasPagar'
json_data_keys_to_try = ["titulosPagar", "listaContasPagar", "listaRegistros", "contasPagar"]


base_url = f"https://app.omie.com.br/api/v1/{base_module}/{api_path}/" 

headers = {
    'Content-Type': 'application/json'
}

# Parâmetros da requisição para Contas a Pagar
body = {
    "call": call_method,
    "app_key": APP_KEY,
    "app_secret": APP_SECRET,
    "param": [
        {
            "pagina": 1,
            "registros_por_pagina": 20, # Você especificou 20 aqui
            "apenas_importado_api": "N"
        }
    ]
}

print(f"Iniciando extração de dados para: {endpoint_name}")
print(f"URL da Requisição: {base_url}")
print(f"Método da Requisição (call): {call_method}")
print("-" * 50)

df_contas_pagar = pd.DataFrame() # DataFrame vazio para armazenar os dados de contas a pagar

try:
    response = requests.post(base_url, headers=headers, data=json.dumps(body))
    response.raise_for_status() # Lança uma exceção para códigos de status de erro (4xx ou 5xx)

    print(f"Dados de {endpoint_name} extraídos com sucesso!")
    
    data = response.json()

    # Tentar extrair a lista de contas a pagar da resposta JSON usando as chaves prováveis
    found_key = None
    for key_attempt in json_data_keys_to_try:
        if key_attempt in data and isinstance(data.get(key_attempt), list):
            df_contas_pagar = pd.DataFrame(data[key_attempt])
            found_key = key_attempt
            break # Sair do loop assim que a chave for encontrada

    if found_key:
        print(f"Chave de dados JSON identificada: '{found_key}'")
        print(f"\n--- Análise Exploratória para {endpoint_name} ---")
        print(f"Shape do DataFrame: {df_contas_pagar.shape}")
        print("\nPrimeiras 5 linhas:")
        print(df_contas_pagar.head())

        print("\nInformações do DataFrame (tipos de dados e contagem de não-nulos):")
        buffer = io.StringIO()
        df_contas_pagar.info(buf=buffer, verbose=True, show_counts=True)
        print(buffer.getvalue())

        print("\nContagem de Valores Nulos por Coluna:")
        print(df_contas_pagar.isnull().sum())
        print("-" * 50)
    else:
        print(f"Atenção: Não foi possível identificar a chave dos dados para '{endpoint_name}' na resposta da API.")
        print("Resposta completa da API (para depuração):", json.dumps(data, indent=2, ensure_ascii=False))
        print("-" * 50)

except requests.exceptions.HTTPError as http_err:
    print(f"Erro HTTP ao extrair {endpoint_name}: {http_err}")
    print(f"Resposta da API: {response.text}")
    print("-" * 50)
except requests.exceptions.ConnectionError as conn_err:
    print(f"Erro de Conexão ao extrair {endpoint_name}: {conn_err}")
    print("Verifique sua conexão com a internet ou o status da API Omie.")
    print("-" * 50)
except requests.exceptions.Timeout as timeout_err:
    print(f"Erro de Timeout ao extrair {endpoint_name}: {timeout_err}")
    print("A requisição demorou muito para responder.")
    print("-" * 50)
except requests.exceptions.RequestException as req_err:
    print(f"Erro inesperado ao extrair {endpoint_name}: {req_err}")
    print("-" * 50)
except json.JSONDecodeError:
    print(f"Erro ao decodificar JSON para {endpoint_name}. Resposta não é JSON válido.")
    print(f"Resposta da API: {response.text}")
    print("-" * 50)
except Exception as e:
    print(f"Ocorreu um erro geral ao processar {endpoint_name}: {e}")
    print("-" * 50)

print("\n--- Extração de Contas a Pagar concluída! ---")

# Agora você tem o DataFrame 'df_contas_pagar' com os dados extraídos.
# Você pode fazer mais análises ou salvá-lo:
# df_contas_pagar.to_csv('contas_pagar_omie.csv', index=False, encoding='utf-8')
# df_contas_pagar.to_excel('contas_pagar_omie.xlsx', index=False)

Iniciando extração de dados para: Contas a Pagar
URL da Requisição: https://app.omie.com.br/api/v1/financas/contapagar/
Método da Requisição (call): ListarContasPagar
--------------------------------------------------
Dados de Contas a Pagar extraídos com sucesso!
Atenção: Não foi possível identificar a chave dos dados para 'Contas a Pagar' na resposta da API.
Resposta completa da API (para depuração): {
  "pagina": 1,
  "total_de_paginas": 2192,
  "registros": 20,
  "total_de_registros": 43834,
  "conta_pagar_cadastro": [
    {
      "baixa_bloqueada": "N",
      "bloqueado": "N",
      "categorias": [
        {
          "codigo_categoria": "2.11.98",
          "percentual": 100,
          "valor": 1800
        }
      ],
      "codigo_categoria": "2.11.98",
      "codigo_cliente_fornecedor": 679084500,
      "codigo_lancamento_integracao": "",
      "codigo_lancamento_omie": 679094339,
      "codigo_projeto": 679094336,
      "codigo_tipo_documento": "99999",
      "data_emissao": "

In [18]:

APP_KEY = "596450736882"
APP_SECRET = "37a10edb420d8cd6c7afddcaaffcac05"

# --- CONFIGURAÇÕES ESPECÍFICAS PARA O ENDPOINT DE CONTAS A PAGAR ---
endpoint_display_name = "Contas a Pagar" # Nome para exibição
base_module = "financas"
api_path = "contapagar" 
api_call_name = "ListarContasPagar"

# Chaves prováveis no JSON da API que contêm a lista de itens (contas a pagar)
# O script tentará estas chaves em ordem.
json_data_keys_to_try = ["conta_pagar_cadastro", "titulosPagar", "listaContasPagar", "listaRegistros", "contasPagar", "cadastro"]

# Nome do arquivo de saída
nome_arquivo_saida_csv = 'omie_contas_a_pagar_limpas_todas.csv'
nome_arquivo_saida_excel = 'omie_contas_a_pagar_limpas_todas.xlsx'
# -----------------------------------------------------------------

versao = "v1" # Geralmente v1 para endpoints gerais da Omie
base_url = f"https://app.omie.com.br/api/{versao}/{base_module}/{api_path}/"

headers = {
    'Content-Type': 'application/json'
}

# Parâmetros iniciais
pagina_atual = 1
registros_por_pagina = 20 # Conforme seu exemplo para Contas a Pagar
max_tentativas_por_pagina = 3
delay_entre_tentativas_base = 5

print(f"Iniciando extração de '{endpoint_display_name}' da API Omie. Registros por página: {registros_por_pagina}")
print(f"URL Base: {base_url}")

todos_os_itens_lista = []
total_de_paginas_api = 1
chave_lista_itens_api_identificada = None 

try:
    while pagina_atual <= total_de_paginas_api:
        body = {
            'call': api_call_name,
            'app_key': APP_KEY,
            'app_secret': APP_SECRET,
            'param': [
                {
                    "pagina": pagina_atual,
                    "registros_por_pagina": registros_por_pagina,
                    "apenas_importado_api": "N" 
                }
            ]
        }

        tentativa_atual = 0
        sucesso_pagina = False
        dados_api_pagina = None 

        while tentativa_atual < max_tentativas_por_pagina and not sucesso_pagina:
            tentativa_atual += 1
            print(f"Buscando página {pagina_atual} de {total_de_paginas_api if total_de_paginas_api > 1 else '...' } para '{endpoint_display_name}' (Tentativa {tentativa_atual}/{max_tentativas_por_pagina})...")
            
            try:
                response = requests.post(base_url, headers=headers, data=json.dumps(body), timeout=60) # Timeout aumentado para 60s
                response.raise_for_status()
                
                dados_api_pagina = response.json()
                sucesso_pagina = True 

                if pagina_atual == 1 and chave_lista_itens_api_identificada is None: 
                    total_de_paginas_api = dados_api_pagina.get('total_de_paginas', 1)
                    total_de_registros_api = dados_api_pagina.get('total_de_registros', 0)
                    print(f"API informou: {total_de_paginas_api} página(s) e {total_de_registros_api} registro(s) no total para '{endpoint_display_name}'.")
                    
                    if total_de_registros_api == 0:
                        print(f"Nenhum registro encontrado na API para '{endpoint_display_name}'.")
                        total_de_paginas_api = 0 
                        break 
                    
                    for key_attempt in json_data_keys_to_try:
                        if key_attempt in dados_api_pagina and isinstance(dados_api_pagina.get(key_attempt), list):
                            chave_lista_itens_api_identificada = key_attempt
                            print(f"Chave de dados JSON identificada para a lista de itens: '{chave_lista_itens_api_identificada}'")
                            break
                    
                    if not chave_lista_itens_api_identificada:
                        print(f"ATENÇÃO: Não foi possível identificar a chave da lista de itens para '{endpoint_display_name}' na resposta da API.")
                        print("Resposta da API (primeira página):", json.dumps(dados_api_pagina, indent=2, ensure_ascii=False))
                        total_de_paginas_api = 0 
                        break 

                if sucesso_pagina and chave_lista_itens_api_identificada:
                    itens_da_pagina = dados_api_pagina.get(chave_lista_itens_api_identificada, [])
                    if itens_da_pagina:
                        todos_os_itens_lista.extend(itens_da_pagina)
                        print(f"  {len(itens_da_pagina)} itens adicionados da página {pagina_atual}. Total acumulado: {len(todos_os_itens_lista)}")
                    else:
                        print(f"  Nenhum item encontrado na página {pagina_atual} (usando chave '{chave_lista_itens_api_identificada}').")
                elif sucesso_pagina and not chave_lista_itens_api_identificada and total_de_registros_api > 0 and pagina_atual == 1 :
                    print(f"  Sucesso ao buscar página {pagina_atual}, mas a chave da lista de itens não foi identificada. Verifique 'json_data_keys_to_try'.")
                    total_de_paginas_api = 0 
                    break


            except (requests.exceptions.ConnectionError, requests.exceptions.Timeout, requests.exceptions.HTTPError) as e:
                print(f"  Erro ao buscar página {pagina_atual} (tentativa {tentativa_atual}): {e}")
                if tentativa_atual < max_tentativas_por_pagina:
                    delay_atual = delay_entre_tentativas_base * (2 ** (tentativa_atual - 1))
                    print(f"  Aguardando {delay_atual} segundos antes de tentar novamente...")
                    time.sleep(delay_atual)
                else:
                    print(f"  Máximo de {max_tentativas_por_pagina} tentativas atingido para a página {pagina_atual}. Desistindo desta página.")
                    break 

        if not sucesso_pagina and total_de_paginas_api > 0 : 
             print(f"AVISO: Falha ao processar a página {pagina_atual} após todas as tentativas. Alguns dados podem estar faltando.")

        if pagina_atual < total_de_paginas_api and total_de_paginas_api > 0:
            time.sleep(1) 
        
        if total_de_paginas_api == 0: 
            break

        pagina_atual += 1

    print(f"\nExtração de todas as páginas para '{endpoint_display_name}' concluída (ou tentativas esgotadas).")

    if not todos_os_itens_lista:
        print(f"Nenhum item de '{endpoint_display_name}' foi extraído da API para processar.")
    else:
        print(f"Total de {len(todos_os_itens_lista)} registros de '{endpoint_display_name}' extraídos para processamento.")

        df_itens = pd.json_normalize(todos_os_itens_lista)
        
        print("\n--- DataFrame Inicial (após json_normalize) ---")
        if not df_itens.empty:
            print(df_itens.head())
        else:
            print("DataFrame inicial está vazio.")

        # Processamento específico para o campo 'categorias' (lista de dicionários)
        if 'categorias' in df_itens.columns:
            # Extrai os códigos das categorias e junta-os numa string
            df_itens['categorias_codigos'] = df_itens['categorias'].apply(
                lambda lista_cats: ', '.join(
                    [cat.get('codigo_categoria', '') for cat in lista_cats if isinstance(cat, dict)]
                ) if isinstance(lista_cats, list) else ''
            )
            # Extrai os valores das categorias e soma-os (se houver apenas um valor por lançamento)
            # Ou concatena se houver múltiplos valores/percentuais e você precisar deles separados.
            # Aqui, para simplificar, vamos pegar o primeiro valor encontrado ou a soma se for simples.
            df_itens['categorias_valores'] = df_itens['categorias'].apply(
                lambda lista_cats: sum(
                    [cat.get('valor', 0) for cat in lista_cats if isinstance(cat, dict) and isinstance(cat.get('valor'), (int, float))]
                ) if isinstance(lista_cats, list) else 0
            )
            # Você pode querer remover a coluna 'categorias' original se não for mais útil
            # df_itens = df_itens.drop(columns=['categorias'])
        else:
            print("Coluna 'categorias' não encontrada no DataFrame para processamento específico.")
            if not df_itens.empty: # Adiciona colunas vazias se o df não estiver vazio mas a coluna faltar
                df_itens['categorias_codigos'] = ''
                df_itens['categorias_valores'] = 0.0


        df_processado = df_itens.replace('', pd.NA)
        df_processado = df_processado.fillna('N/A')

        print("\n--- DataFrame após preencher valores nulos/vazios e processar categorias ---")
        if not df_processado.empty:
            print(df_processado.head())
        else:
            print("DataFrame processado está vazio.")


        print("\n--- Colunas Disponíveis para Seleção/Renomeação ---")
        if not df_processado.empty:
            colunas_disponiveis = df_processado.columns.tolist()
            for col_name in colunas_disponiveis:
                print(f"- {col_name}")
        else:
            print("Nenhuma coluna disponível, DataFrame está vazio.")

        # --- AJUSTE AQUI: Defina as colunas que quer manter e os novos nomes para CONTAS A PAGAR ---
        mapa_renomeacao_e_selecao = {
            'codigo_lancamento_omie': 'ID Lançamento Omie', # Exemplo: geralmente um ID único do lançamento
            'codigo_lancamento_integracao': 'ID Lançamento Integração',
            'codigo_cliente_fornecedor': 'ID Cliente/Fornecedor',
            'data_vencimento': 'Data Vencimento',
            'valor_documento': 'Valor Documento',
            'numero_documento_fiscal': 'NF',
            'data_emissao': 'Data Emissão NF',
            'codigo_categoria': 'Código Categoria Principal', # Esta é a categoria principal listada, não a da lista aninhada
            'categorias_codigos': 'Códigos Categorias (Todas)', # Nova coluna do processamento
            'categorias_valores': 'Valor Total Categorias', # Nova coluna do processamento
            'data_pagamento': 'Data Pagamento',
            'valor_pago': 'Valor Pago',
            'status_titulo': 'Status Título', # Ex: PAGO, A PAGAR
            'observacao': 'Observação',
            'codigo_conta_corrente': 'ID Conta Corrente',
            # Campos do detalhe (info) se achatados por json_normalize
            'detalhes.dInc': 'Data Inclusão Lanç.', # Exemplo se houver um objeto 'detalhes'
            'detalhes.dAlt': 'Data Alteração Lanç.',
            # Adicione outras colunas da lista "Colunas Disponíveis"
        }

        df_final = pd.DataFrame() 
        if not df_processado.empty:
            colunas_para_manter_renomeadas = {
                antigo: novo for antigo, novo in mapa_renomeacao_e_selecao.items() if antigo in df_processado.columns
            }
            colunas_antigas_selecionadas = list(colunas_para_manter_renomeadas.keys())

            if not colunas_antigas_selecionadas:
                print(f"\nAVISO: Nenhuma coluna do 'mapa_renomeacao_e_selecao' para '{endpoint_display_name}' foi encontrada no DataFrame.")
                df_final = df_processado.copy() 
            else:
                df_final = df_processado[colunas_antigas_selecionadas].rename(columns=colunas_para_manter_renomeadas)

                ordem_desejada = [
                    'ID Lançamento Omie', 'ID Cliente/Fornecedor', 'NF', 'Data Emissão NF',
                    'Data Vencimento', 'Valor Documento', 'Status Título',
                    'Código Categoria Principal', 'Códigos Categorias (Todas)', 'Valor Total Categorias',
                    'Data Pagamento', 'Valor Pago', 'ID Conta Corrente', 'Observação',
                    'Data Inclusão Lanç.', 'Data Alteração Lanç.'
                ]
                ordem_existente = [col for col in ordem_desejada if col in df_final.columns]
                for col in df_final.columns: 
                    if col not in ordem_existente:
                        ordem_existente.append(col)
                if ordem_existente: 
                    df_final = df_final[ordem_existente]
        else:
            print("DataFrame processado está vazio, não é possível criar df_final com colunas selecionadas.")


        print("\n--- DataFrame Final (Limpado, Selecionado e Renomeado) ---")
        if not df_final.empty:
            print(df_final.head())
            print(f"\nTotal de {len(df_final)} itens de '{endpoint_display_name}' no DataFrame final.")

            df_final.to_csv(nome_arquivo_saida_csv, index=False, sep=';', encoding='utf-8-sig')
            print(f"\nDataFrame final salvo em: {nome_arquivo_saida_csv}")
        else:
            print("DataFrame final está vazio. Nenhum arquivo CSV será gerado.")

except requests.exceptions.HTTPError as http_err:
    print(f"Erro HTTP ocorreu durante a chamada à API para '{endpoint_display_name}': {http_err}")
    if 'response' in locals() and response is not None:
        print(f"Conteúdo da resposta: {response.text}")
except requests.exceptions.ConnectionError as conn_err: 
    print(f"Erro de Conexão ocorreu durante a chamada à API para '{endpoint_display_name}': {conn_err}")
except requests.exceptions.Timeout as timeout_err: 
    print(f"Timeout ocorreu durante a chamada à API para '{endpoint_display_name}': {timeout_err}")
except requests.exceptions.RequestException as req_err: 
    print(f"Erro de Requisição ocorreu durante a chamada à API para '{endpoint_display_name}': {req_err}")
except json.JSONDecodeError as json_err:
    print(f"Erro ao decodificar JSON da resposta da API para '{endpoint_display_name}': {json_err}")
    if 'response' in locals() and response is not None: 
        print(f"Conteúdo da resposta que causou o erro: {response.text}")
except Exception as e:
    print(f"Um erro inesperado ocorreu durante o processamento de '{endpoint_display_name}': {e}")

Iniciando extração de 'Contas a Pagar' da API Omie. Registros por página: 20
URL Base: https://app.omie.com.br/api/v1/financas/contapagar/
Buscando página 1 de ... para 'Contas a Pagar' (Tentativa 1/3)...
API informou: 2192 página(s) e 43834 registro(s) no total para 'Contas a Pagar'.
Chave de dados JSON identificada para a lista de itens: 'conta_pagar_cadastro'
  20 itens adicionados da página 1. Total acumulado: 20
Buscando página 2 de 2192 para 'Contas a Pagar' (Tentativa 1/3)...
  20 itens adicionados da página 2. Total acumulado: 40
Buscando página 3 de 2192 para 'Contas a Pagar' (Tentativa 1/3)...
  20 itens adicionados da página 3. Total acumulado: 60
Buscando página 4 de 2192 para 'Contas a Pagar' (Tentativa 1/3)...
  20 itens adicionados da página 4. Total acumulado: 80
Buscando página 5 de 2192 para 'Contas a Pagar' (Tentativa 1/3)...
  20 itens adicionados da página 5. Total acumulado: 100
Buscando página 6 de 2192 para 'Contas a Pagar' (Tentativa 1/3)...
  20 itens adiciona

In [19]:

APP_KEY = "596450736882"
APP_SECRET = "37a10edb420d8cd6c7afddcaaffcac05"

# --- CONFIGURAÇÃO PARA EXTRAÇÃO DE PLANO DE CONTAS (DRE) ---
endpoint_name = "Plano de Contas (DRE)" # Nome da categoria para exibição e uso no DataFrame
base_module = "geral"                  # Módulo para DRE na Omie API, conforme sua URL
api_path = "dre"                       # Parte final da URL do endpoint
call_method = "ListarCadastroDRE"      # Nome do método de chamada da API
# Chave que contém a lista de contas DRE na resposta JSON (pode variar, verificar docs Omie)
# Tentaremos 'listaContas', 'listaRegistros' ou 'contasDRE'
json_data_keys_to_try = ["listaContas", "listaRegistros", "contasDRE"]


base_url = f"https://app.omie.com.br/api/v1/{base_module}/{api_path}/" 

headers = {
    'Content-Type': 'application/json'
}

# Parâmetros da requisição para Plano de Contas (DRE)
body = {
    "call": call_method,
    "app_key": APP_KEY,
    "app_secret": APP_SECRET,
    "param": [
        {
            "apenasContasAtivas": "N"
        }
    ]
}

print(f"Iniciando extração de dados para: {endpoint_name}")
print(f"URL da Requisição: {base_url}")
print(f"Método da Requisição (call): {call_method}")
print("-" * 50)

df_plano_contas_dre = pd.DataFrame() # DataFrame vazio para armazenar os dados do plano de contas

try:
    response = requests.post(base_url, headers=headers, data=json.dumps(body))
    response.raise_for_status() # Lança uma exceção para códigos de status de erro (4xx ou 5xx)

    print(f"Dados de {endpoint_name} extraídos com sucesso!")
    
    data = response.json()

    # Tentar extrair a lista de contas DRE da resposta JSON usando as chaves prováveis
    found_key = None
    for key_attempt in json_data_keys_to_try:
        if key_attempt in data and isinstance(data.get(key_attempt), list):
            df_plano_contas_dre = pd.DataFrame(data[key_attempt])
            found_key = key_attempt
            break # Sair do loop assim que a chave for encontrada

    if found_key:
        print(f"Chave de dados JSON identificada: '{found_key}'")
        print(f"\n--- Análise Exploratória para {endpoint_name} ---")
        print(f"Shape do DataFrame: {df_plano_contas_dre.shape}")
        print("\nPrimeiras 5 linhas:")
        print(df_plano_contas_dre.head())

        print("\nInformações do DataFrame (tipos de dados e contagem de não-nulos):")
        buffer = io.StringIO()
        df_plano_contas_dre.info(buf=buffer, verbose=True, show_counts=True)
        print(buffer.getvalue())

        print("\nContagem de Valores Nulos por Coluna:")
        print(df_plano_contas_dre.isnull().sum())
        print("-" * 50)
    else:
        print(f"Atenção: Não foi possível identificar a chave dos dados para '{endpoint_name}' na resposta da API.")
        print("Resposta completa da API (para depuração):", json.dumps(data, indent=2, ensure_ascii=False))
        print("-" * 50)

except requests.exceptions.HTTPError as http_err:
    print(f"Erro HTTP ao extrair {endpoint_name}: {http_err}")
    print(f"Resposta da API: {response.text}")
    print("-" * 50)
except requests.exceptions.ConnectionError as conn_err:
    print(f"Erro de Conexão ao extrair {endpoint_name}: {conn_err}")
    print("Verifique sua conexão com a internet ou o status da API Omie.")
    print("-" * 50)
except requests.exceptions.Timeout as timeout_err:
    print(f"Erro de Timeout ao extrair {endpoint_name}: {timeout_err}")
    print("A requisição demorou muito para responder.")
    print("-" * 50)
except requests.exceptions.RequestException as req_err:
    print(f"Erro inesperado ao extrair {endpoint_name}: {req_err}")
    print("-" * 50)
except json.JSONDecodeError:
    print(f"Erro ao decodificar JSON para {endpoint_name}. Resposta não é JSON válido.")
    print(f"Resposta da API: {response.text}")
    print("-" * 50)
except Exception as e:
    print(f"Ocorreu um erro geral ao processar {endpoint_name}: {e}")
    print("-" * 50)

print("\n--- Extração de Plano de Contas (DRE) concluída! ---")

# Agora você tem o DataFrame 'df_plano_contas_dre' com os dados extraídos.
# Você pode fazer mais análises ou salvá-lo:
# df_plano_contas_dre.to_csv('plano_contas_dre_omie.csv', index=False, encoding='utf-8')
# df_plano_contas_dre.to_excel('plano_contas_dre_omie.xlsx', index=False)

Iniciando extração de dados para: Plano de Contas (DRE)
URL da Requisição: https://app.omie.com.br/api/v1/geral/dre/
Método da Requisição (call): ListarCadastroDRE
--------------------------------------------------
Dados de Plano de Contas (DRE) extraídos com sucesso!
Atenção: Não foi possível identificar a chave dos dados para 'Plano de Contas (DRE)' na resposta da API.
Resposta completa da API (para depuração): {
  "totalRegistros": 28,
  "dreLista": [
    {
      "codigoDRE": "1",
      "descricaoDRE": "Lucro Bruto",
      "naoExibirDRE": "N",
      "nivelDRE": 1,
      "sinalDRE": "",
      "totalizaDRE": "S"
    },
    {
      "codigoDRE": "1.01",
      "descricaoDRE": "Receita Líquida Operacional",
      "naoExibirDRE": "N",
      "nivelDRE": 2,
      "sinalDRE": "",
      "totalizaDRE": "S"
    },
    {
      "codigoDRE": "1.01.01",
      "descricaoDRE": "Receita Bruta de Vendas",
      "naoExibirDRE": "N",
      "nivelDRE": 3,
      "sinalDRE": "+",
      "totalizaDRE": "N"
   

In [20]:

APP_KEY = "596450736882"
APP_SECRET = "37a10edb420d8cd6c7afddcaaffcac05"

# --- CONFIGURAÇÕES ESPECÍFICAS PARA O ENDPOINT DE PLANO DE CONTAS (DRE) ---
endpoint_display_name = "Plano de Contas (DRE)" # Nome para exibição
base_module = "geral"
api_path = "dre" 
api_call_name = "ListarCadastroDRE"

# Chaves prováveis no JSON da API que contêm a lista de itens (contas DRE)
# O script tentará estas chaves em ordem.
json_data_keys_to_try = ["listaContas", "listaRegistros", "contasDRE", "cadastro"]

# Nome do arquivo de saída
nome_arquivo_saida_csv = 'omie_plano_contas_dre_limpos_todos.csv'
nome_arquivo_saida_excel = 'omie_plano_contas_dre_limpos_todos.xlsx'
# -----------------------------------------------------------------

versao = "v1" # Geralmente v1 para endpoints gerais da Omie
base_url = f"https://app.omie.com.br/api/{versao}/{base_module}/{api_path}/"

headers = {
    'Content-Type': 'application/json'
}

# Parâmetros iniciais
# Para ListarCadastroDRE, a paginação pode não ser aplicável ou diferente.
# O script tentará buscar a primeira página. Se total_de_paginas for 1 (ou não existir), o loop rodará uma vez.
pagina_atual = 1 
# registros_por_pagina não é usado por ListarCadastroDRE tipicamente, mas mantemos a variável para a estrutura do loop.
registros_por_pagina = 500 # Um valor alto, já que esperamos todos os registros de uma vez.
max_tentativas_por_pagina = 3
delay_entre_tentativas_base = 5

print(f"Iniciando extração de '{endpoint_display_name}' da API Omie.")
print(f"URL Base: {base_url}")

todos_os_itens_lista = []
total_de_paginas_api = 1 # Assume 1 página por padrão para endpoints não pagináveis ou com resposta única
chave_lista_itens_api_identificada = None 

try:
    # Para ListarCadastroDRE, o loop de paginação provavelmente rodará apenas uma vez.
    while pagina_atual <= total_de_paginas_api:
        # Corpo da requisição específico para ListarCadastroDRE
        body = {
            'call': api_call_name,
            'app_key': APP_KEY,
            'app_secret': APP_SECRET,
            'param': [
                {
                    "apenasContasAtivas": "N" # Parâmetro específico do ListarCadastroDRE
                }
            ]
        }
        # Se ListarCadastroDRE suportar paginação e você quiser usá-la, adicione os parâmetros de página aqui.
        # No entanto, geralmente este endpoint retorna tudo. Se ele tiver 'pagina' e 'registros_por_pagina',
        # você pode adicionar ao param:
        # "pagina": pagina_atual,
        # "registros_por_pagina": registros_por_pagina

        tentativa_atual = 0
        sucesso_pagina = False
        dados_api_pagina = None 

        while tentativa_atual < max_tentativas_por_pagina and not sucesso_pagina:
            tentativa_atual += 1
            print(f"Buscando dados para '{endpoint_display_name}' (Página {pagina_atual} de {total_de_paginas_api}, Tentativa {tentativa_atual}/{max_tentativas_por_pagina})...")
            
            try:
                response = requests.post(base_url, headers=headers, data=json.dumps(body), timeout=60) 
                response.raise_for_status()
                
                dados_api_pagina = response.json()
                sucesso_pagina = True 

                # Se for um endpoint não paginável que retorna tudo na "página 1"
                # ou se a API retornar total_de_paginas.
                if chave_lista_itens_api_identificada is None: # Tenta identificar a chave apenas uma vez
                    # Este endpoint pode não retornar 'total_de_paginas' ou 'total_de_registros'
                    # Se não retornar, total_de_paginas_api continuará 1 e o loop rodará uma vez.
                    total_de_paginas_api = dados_api_pagina.get('total_de_paginas', 1) 
                    total_de_registros_api = dados_api_pagina.get('total_de_registros', 0) # Pega se existir
                    
                    if total_de_registros_api > 0 : # Se existe a contagem de registros
                         print(f"API informou: {total_de_paginas_api} página(s) e {total_de_registros_api} registro(s) no total para '{endpoint_display_name}'.")
                    elif not dados_api_pagina.get(json_data_keys_to_try[0], []) and not any(key in dados_api_pagina for key in json_data_keys_to_try if isinstance(dados_api_pagina.get(key), list)):
                        # Verifica se a primeira chave provável está vazia e se nenhuma outra chave da lista existe como lista
                        print(f"Nenhum registro principal encontrado na API para '{endpoint_display_name}' na primeira tentativa de chave, ou estrutura de resposta inesperada.")
                        total_de_paginas_api = 0 # Para o loop
                        break

                    for key_attempt in json_data_keys_to_try:
                        if key_attempt in dados_api_pagina and isinstance(dados_api_pagina.get(key_attempt), list):
                            chave_lista_itens_api_identificada = key_attempt
                            print(f"Chave de dados JSON identificada para a lista de itens: '{chave_lista_itens_api_identificada}'")
                            # Se a chave foi encontrada e não há 'total_de_registros', inferimos o total pela lista
                            if total_de_registros_api == 0 and len(dados_api_pagina[key_attempt]) > 0:
                                total_de_registros_api = len(dados_api_pagina[key_attempt])
                                print(f"Inferido {total_de_registros_api} registro(s) com base no conteúdo da chave '{chave_lista_itens_api_identificada}'.")
                            break 
                    
                    if not chave_lista_itens_api_identificada:
                        print(f"ATENÇÃO: Não foi possível identificar a chave da lista de itens para '{endpoint_display_name}' na resposta da API.")
                        print("Resposta da API:", json.dumps(dados_api_pagina, indent=2, ensure_ascii=False))
                        total_de_paginas_api = 0 
                        break 
                    
                    if total_de_registros_api == 0 and chave_lista_itens_api_identificada and not dados_api_pagina.get(chave_lista_itens_api_identificada):
                        print(f"Chave '{chave_lista_itens_api_identificada}' encontrada, mas está vazia. Nenhum registro.")
                        total_de_paginas_api = 0
                        break


                if sucesso_pagina and chave_lista_itens_api_identificada:
                    itens_da_pagina = dados_api_pagina.get(chave_lista_itens_api_identificada, [])
                    if itens_da_pagina:
                        todos_os_itens_lista.extend(itens_da_pagina)
                        print(f"  {len(itens_da_pagina)} itens adicionados. Total acumulado: {len(todos_os_itens_lista)}")
                    else:
                        print(f"  Nenhum item encontrado (usando chave '{chave_lista_itens_api_identificada}').")
                elif sucesso_pagina and not chave_lista_itens_api_identificada and total_de_registros_api > 0 : # total_de_registros_api pode não ser relevante aqui
                    print(f"  Sucesso ao buscar dados, mas a chave da lista de itens não foi identificada. Verifique 'json_data_keys_to_try'.")
                    total_de_paginas_api = 0 
                    break


            except (requests.exceptions.ConnectionError, requests.exceptions.Timeout, requests.exceptions.HTTPError) as e:
                print(f"  Erro ao buscar dados (tentativa {tentativa_atual}): {e}")
                if tentativa_atual < max_tentativas_por_pagina:
                    delay_atual = delay_entre_tentativas_base * (2 ** (tentativa_atual - 1))
                    print(f"  Aguardando {delay_atual} segundos antes de tentar novamente...")
                    time.sleep(delay_atual)
                else:
                    print(f"  Máximo de {max_tentativas_por_pagina} tentativas atingido. Desistindo.")
                    break 

        if not sucesso_pagina and total_de_paginas_api > 0 : 
             print(f"AVISO: Falha ao processar os dados após todas as tentativas.")

        # Como ListarCadastroDRE geralmente não é paginado, quebramos o loop após a primeira tentativa (ou falha).
        break # Remove esta linha se ListarCadastroDRE suportar paginação e você a implementou acima.
        
        # pagina_atual += 1 # Desnecessário se não houver paginação real.

    print(f"\nExtração para '{endpoint_display_name}' concluída (ou tentativas esgotadas).")

    if not todos_os_itens_lista:
        print(f"Nenhum item de '{endpoint_display_name}' foi extraído da API para processar.")
    else:
        print(f"Total de {len(todos_os_itens_lista)} registros de '{endpoint_display_name}' extraídos para processamento.")

        df_itens = pd.json_normalize(todos_os_itens_lista)
        
        print("\n--- DataFrame Inicial (após json_normalize) ---")
        if not df_itens.empty:
            print(df_itens.head())
        else:
            print("DataFrame inicial está vazio.")
        
        df_processado = df_itens.replace('', pd.NA)
        df_processado = df_processado.fillna('N/A')

        print("\n--- DataFrame após preencher valores nulos/vazios ---")
        if not df_processado.empty:
            print(df_processado.head())
        else:
            print("DataFrame processado está vazio.")


        print("\n--- Colunas Disponíveis para Seleção/Renomeação ---")
        if not df_processado.empty:
            colunas_disponiveis = df_processado.columns.tolist()
            for col_name in colunas_disponiveis:
                print(f"- {col_name}")
        else:
            print("Nenhuma coluna disponível, DataFrame está vazio.")

        # --- AJUSTE AQUI: Defina as colunas que quer manter e os novos nomes para PLANO DE CONTAS (DRE) ---
        mapa_renomeacao_e_selecao = {
            # Exemplo de colunas comuns para Plano de Contas (DRE) (VERIFIQUE A SAÍDA REAL DA API):
            'ccodigo': 'Código Conta DRE',          # Nome hipotético
            'cdescricao': 'Descrição Conta DRE',    # Nome hipotético
            'cnivel': 'Nível Conta DRE',            # Nome hipotético
            'csintetica': 'Sintética (S/N)',        # Nome hipotético (se é conta totalizadora)
            'cgrupo': 'Grupo Conta DRE',            # Nome hipotético (ex: R, D, C)
            'corigem': 'Origem Conta DRE',          # Nome hipotético
            'csaldo': 'Tipo Saldo DRE',             # Nome hipotético (ex: D, C)
            'cinativa': 'Inativa (S/N)',            # Nome hipotético
            # Adicione outras colunas que você identificar na saída de "Colunas Disponíveis"
        }

        df_final = pd.DataFrame() 
        if not df_processado.empty:
            colunas_para_manter_renomeadas = {
                antigo: novo for antigo, novo in mapa_renomeacao_e_selecao.items() if antigo in df_processado.columns
            }
            colunas_antigas_selecionadas = list(colunas_para_manter_renomeadas.keys())

            if not colunas_antigas_selecionadas:
                print(f"\nAVISO: Nenhuma coluna do 'mapa_renomeacao_e_selecao' para '{endpoint_display_name}' foi encontrada no DataFrame.")
                df_final = df_processado.copy() 
            else:
                df_final = df_processado[colunas_antigas_selecionadas].rename(columns=colunas_para_manter_renomeadas)

                # --- AJUSTE AQUI: Defina a ordem desejada para as colunas de PLANO DE CONTAS (DRE) ---
                ordem_desejada = [
                    'Código Conta DRE', 'Descrição Conta DRE', 'Nível Conta DRE', 
                    'Sintética (S/N)', 'Grupo Conta DRE', 'Origem Conta DRE', 
                    'Tipo Saldo DRE', 'Inativa (S/N)'
                ]
                ordem_existente = [col for col in ordem_desejada if col in df_final.columns]
                for col in df_final.columns: 
                    if col not in ordem_existente:
                        ordem_existente.append(col)
                if ordem_existente: 
                    df_final = df_final[ordem_existente]
        else:
            print("DataFrame processado está vazio, não é possível criar df_final com colunas selecionadas.")


        print("\n--- DataFrame Final (Limpado, Selecionado e Renomeado) ---")
        if not df_final.empty:
            print(df_final.head())
            print(f"\nTotal de {len(df_final)} itens de '{endpoint_display_name}' no DataFrame final.")

            df_final.to_csv(nome_arquivo_saida_csv, index=False, sep=';', encoding='utf-8-sig')
            print(f"\nDataFrame final salvo em: {nome_arquivo_saida_csv}")
        else:
            print("DataFrame final está vazio. Nenhum arquivo CSV será gerado.")


except requests.exceptions.HTTPError as http_err:
    print(f"Erro HTTP ocorreu durante a chamada à API para '{endpoint_display_name}': {http_err}")
    if 'response' in locals() and response is not None:
        print(f"Conteúdo da resposta: {response.text}")
except requests.exceptions.ConnectionError as conn_err: 
    print(f"Erro de Conexão ocorreu durante a chamada à API para '{endpoint_display_name}': {conn_err}")
except requests.exceptions.Timeout as timeout_err: 
    print(f"Timeout ocorreu durante a chamada à API para '{endpoint_display_name}': {timeout_err}")
except requests.exceptions.RequestException as req_err: 
    print(f"Erro de Requisição ocorreu durante a chamada à API para '{endpoint_display_name}': {req_err}")
except json.JSONDecodeError as json_err:
    print(f"Erro ao decodificar JSON da resposta da API para '{endpoint_display_name}': {json_err}")
    if 'response' in locals() and response is not None: 
        print(f"Conteúdo da resposta que causou o erro: {response.text}")
except Exception as e:
    print(f"Um erro inesperado ocorreu durante o processamento de '{endpoint_display_name}': {e}")


Iniciando extração de 'Plano de Contas (DRE)' da API Omie.
URL Base: https://app.omie.com.br/api/v1/geral/dre/
Buscando dados para 'Plano de Contas (DRE)' (Página 1 de 1, Tentativa 1/3)...
Nenhum registro principal encontrado na API para 'Plano de Contas (DRE)' na primeira tentativa de chave, ou estrutura de resposta inesperada.

Extração para 'Plano de Contas (DRE)' concluída (ou tentativas esgotadas).
Nenhum item de 'Plano de Contas (DRE)' foi extraído da API para processar.


In [21]:

APP_KEY = "596450736882"
APP_SECRET = "37a10edb420d8cd6c7afddcaaffcac05"

# --- CONFIGURAÇÃO PARA EXTRAÇÃO DE NF-e (CONSULTAS) ---
endpoint_name = "NF-e (Consultas)" # Nome da categoria para exibição e uso no DataFrame
base_module = "produtos"         # Módulo para NF-e na Omie API, conforme sua URL
api_path = "nfconsultar"         # Parte final da URL do endpoint
call_method = "ListarNF"         # Nome do método de chamada da API
# Chave que contém a lista de NF-e na resposta JSON (pode variar, verificar docs Omie)
# Tentaremos 'nf_cadastro', 'listaNF', 'notasFiscais', 'listaRegistros' ou 'nfList'
json_data_keys_to_try = ["nf_cadastro", "listaNF", "notasFiscais", "listaRegistros", "nfList"]


base_url = f"https://app.omie.com.br/api/v1/{base_module}/{api_path}/" 

headers = {
    'Content-Type': 'application/json'
}

# Parâmetros da requisição para NF-e (Consultas)
body = {
    "call": call_method,
    "app_key": APP_KEY,
    "app_secret": APP_SECRET,
    "param": [
        {
            "pagina": 1,
            "registros_por_pagina": 20, # Você especificou 20 aqui
            "apenas_importado_api": "N",
            "ordenar_por": "CODIGO"
        }
    ]
}

print(f"Iniciando extração de dados para: {endpoint_name}")
print(f"URL da Requisição: {base_url}")
print(f"Método da Requisição (call): {call_method}")
print("-" * 50)

df_nfe_consultas = pd.DataFrame() # DataFrame vazio para armazenar os dados de NF-e

try:
    response = requests.post(base_url, headers=headers, data=json.dumps(body))
    response.raise_for_status() # Lança uma exceção para códigos de status de erro (4xx ou 5xx)

    print(f"Dados de {endpoint_name} extraídos com sucesso!")
    
    data = response.json()

    # Tentar extrair a lista de NF-e da resposta JSON usando as chaves prováveis
    found_key = None
    for key_attempt in json_data_keys_to_try:
        if key_attempt in data and isinstance(data.get(key_attempt), list):
            df_nfe_consultas = pd.DataFrame(data[key_attempt])
            found_key = key_attempt
            break # Sair do loop assim que a chave for encontrada

    if found_key:
        print(f"Chave de dados JSON identificada: '{found_key}'")
        print(f"\n--- Análise Exploratória para {endpoint_name} ---")
        print(f"Shape do DataFrame: {df_nfe_consultas.shape}")
        print("\nPrimeiras 5 linhas:")
        print(df_nfe_consultas.head())

        print("\nInformações do DataFrame (tipos de dados e contagem de não-nulos):")
        buffer = io.StringIO()
        df_nfe_consultas.info(buf=buffer, verbose=True, show_counts=True)
        print(buffer.getvalue())

        print("\nContagem de Valores Nulos por Coluna:")
        print(df_nfe_consultas.isnull().sum())
        print("-" * 50)
    else:
        print(f"Atenção: Não foi possível identificar a chave dos dados para '{endpoint_name}' na resposta da API.")
        print("Resposta completa da API (para depuração):", json.dumps(data, indent=2, ensure_ascii=False))
        print("-" * 50)

except requests.exceptions.HTTPError as http_err:
    print(f"Erro HTTP ao extrair {endpoint_name}: {http_err}")
    print(f"Resposta da API: {response.text}")
    print("-" * 50)
except requests.exceptions.ConnectionError as conn_err:
    print(f"Erro de Conexão ao extrair {endpoint_name}: {conn_err}")
    print("Verifique sua conexão com a internet ou o status da API Omie.")
    print("-" * 50)
except requests.exceptions.Timeout as timeout_err:
    print(f"Erro de Timeout ao extrair {endpoint_name}: {timeout_err}")
    print("A requisição demorou muito para responder.")
    print("-" * 50)
except requests.exceptions.RequestException as req_err:
    print(f"Erro inesperado ao extrair {endpoint_name}: {req_err}")
    print("-" * 50)
except json.JSONDecodeError:
    print(f"Erro ao decodificar JSON para {endpoint_name}. Resposta não é JSON válido.")
    print(f"Resposta da API: {response.text}")
    print("-" * 50)
except Exception as e:
    print(f"Ocorreu um erro geral ao processar {endpoint_name}: {e}")
    print("-" * 50)

print("\n--- Extração de NF-e (Consultas) concluída! ---")

# Agora você tem o DataFrame 'df_nfe_consultas' com os dados extraídos.
# Você pode fazer mais análises ou salvá-lo:
# df_nfe_consultas.to_csv('nfe_consultas_omie.csv', index=False, encoding='utf-8')
# df_nfe_consultas.to_excel('nfe_consultas_omie.xlsx', index=False)

Iniciando extração de dados para: NF-e (Consultas)
URL da Requisição: https://app.omie.com.br/api/v1/produtos/nfconsultar/
Método da Requisição (call): ListarNF
--------------------------------------------------
Dados de NF-e (Consultas) extraídos com sucesso!
Atenção: Não foi possível identificar a chave dos dados para 'NF-e (Consultas)' na resposta da API.
Resposta completa da API (para depuração): {
  "pagina": 1,
  "total_de_paginas": 103,
  "registros": 20,
  "total_de_registros": 2060,
  "nfCadastro": [
    {
      "compl": {
        "cChaveNFe": "35181243283811013995550010001945931286166664",
        "cCodCateg": "2.04.06",
        "cModFrete": "9",
        "nIdNF": 679597940,
        "nIdPedido": 0,
        "nIdReceb": 679596110,
        "nIdTransp": 0
      },
      "det": [
        {
          "nfProdInt": {
            "cCodItemInt": "",
            "cCodProdInt": "",
            "nCodItem": 679597944,
            "nCodProd": 679597943
          },
          "prod": {
      

In [22]:

APP_KEY = "596450736882"
APP_SECRET = "37a10edb420d8cd6c7afddcaaffcac05"

# --- CONFIGURAÇÕES ESPECÍFICAS PARA O ENDPOINT DE NF-e (CONSULTAS) ---
endpoint_display_name = "NF-e (Consultas)" # Nome para exibição
base_module = "produtos"
api_path = "nfconsultar" 
api_call_name = "ListarNF"

# Chaves prováveis no JSON da API que contêm a lista de itens (NF-e)
# O script tentará estas chaves em ordem.
json_data_keys_to_try = ["nfCadastro", "nf_cadastro", "listaNF", "notasFiscais", "listaRegistros", "nfList", "cadastro"] # Adicionado nfCadastro

# Nome do arquivo de saída
nome_arquivo_saida_csv = 'omie_nfe_consultas_limpas_todas.csv'
nome_arquivo_saida_excel = 'omie_nfe_consultas_limpas_todas.xlsx'
# -----------------------------------------------------------------

versao = "v1" # Geralmente v1 para endpoints gerais da Omie
base_url = f"https://app.omie.com.br/api/{versao}/{base_module}/{api_path}/"

headers = {
    'Content-Type': 'application/json'
}

# Parâmetros iniciais
pagina_atual = 1 
registros_por_pagina = 20 # Conforme seu exemplo para NF-e
max_tentativas_por_pagina = 3
delay_entre_tentativas_base = 5

print(f"Iniciando extração de '{endpoint_display_name}' da API Omie.")
print(f"URL Base: {base_url}")

todos_os_itens_lista = []
total_de_paginas_api = 1 
chave_lista_itens_api_identificada = None 

try:
    while pagina_atual <= total_de_paginas_api:
        # Corpo da requisição específico para ListarNF
        body = {
            'call': api_call_name,
            'app_key': APP_KEY,
            'app_secret': APP_SECRET,
            'param': [
                {
                    "pagina": pagina_atual,
                    "registros_por_pagina": registros_por_pagina,
                    "apenas_importado_api": "N",
                    "ordenar_por": "CODIGO" # Parâmetro específico do ListarNF
                    # Adicione outros filtros se necessário, ex: "filtrar_por_data_de", "filtrar_por_data_ate"
                }
            ]
        }

        tentativa_atual = 0
        sucesso_pagina = False
        dados_api_pagina = None 

        while tentativa_atual < max_tentativas_por_pagina and not sucesso_pagina:
            tentativa_atual += 1
            print(f"Buscando dados para '{endpoint_display_name}' (Página {pagina_atual} de {total_de_paginas_api if total_de_paginas_api > 1 else '...'}, Tentativa {tentativa_atual}/{max_tentativas_por_pagina})...")
            
            try:
                response = requests.post(base_url, headers=headers, data=json.dumps(body), timeout=60) 
                response.raise_for_status()
                
                dados_api_pagina = response.json()
                sucesso_pagina = True 

                if pagina_atual == 1 and chave_lista_itens_api_identificada is None: 
                    total_de_paginas_api = dados_api_pagina.get('total_de_paginas', 1) 
                    total_de_registros_api = dados_api_pagina.get('total_de_registros', 0) 
                    
                    print(f"API informou: {total_de_paginas_api} página(s) e {total_de_registros_api} registro(s) no total para '{endpoint_display_name}'.")
                    
                    if total_de_registros_api == 0:
                        print(f"Nenhum registro encontrado na API para '{endpoint_display_name}'.")
                        total_de_paginas_api = 0 
                        break

                    for key_attempt in json_data_keys_to_try:
                        if key_attempt in dados_api_pagina and isinstance(dados_api_pagina.get(key_attempt), list):
                            chave_lista_itens_api_identificada = key_attempt
                            print(f"Chave de dados JSON identificada para a lista de itens: '{chave_lista_itens_api_identificada}'")
                            break 
                    
                    if not chave_lista_itens_api_identificada:
                        print(f"ATENÇÃO: Não foi possível identificar a chave da lista de itens para '{endpoint_display_name}' na resposta da API.")
                        print("Resposta da API (primeira página):", json.dumps(dados_api_pagina, indent=2, ensure_ascii=False))
                        total_de_paginas_api = 0 
                        break 
                    
                    if total_de_registros_api == 0 and chave_lista_itens_api_identificada and not dados_api_pagina.get(chave_lista_itens_api_identificada):
                        print(f"Chave '{chave_lista_itens_api_identificada}' encontrada, mas está vazia. Nenhum registro.")
                        total_de_paginas_api = 0
                        break

                if sucesso_pagina and chave_lista_itens_api_identificada:
                    itens_da_pagina = dados_api_pagina.get(chave_lista_itens_api_identificada, [])
                    if itens_da_pagina:
                        todos_os_itens_lista.extend(itens_da_pagina)
                        print(f"  {len(itens_da_pagina)} itens adicionados. Total acumulado: {len(todos_os_itens_lista)}")
                    else:
                        print(f"  Nenhum item encontrado na página {pagina_atual} (usando chave '{chave_lista_itens_api_identificada}').")
                elif sucesso_pagina and not chave_lista_itens_api_identificada and total_de_registros_api > 0 and pagina_atual == 1 :
                    print(f"  Sucesso ao buscar dados, mas a chave da lista de itens não foi identificada. Verifique 'json_data_keys_to_try'.")
                    total_de_paginas_api = 0 
                    break

            except (requests.exceptions.ConnectionError, requests.exceptions.Timeout, requests.exceptions.HTTPError) as e:
                print(f"  Erro ao buscar dados (tentativa {tentativa_atual}): {e}")
                if tentativa_atual < max_tentativas_por_pagina:
                    delay_atual = delay_entre_tentativas_base * (2 ** (tentativa_atual - 1))
                    print(f"  Aguardando {delay_atual} segundos antes de tentar novamente...")
                    time.sleep(delay_atual)
                else:
                    print(f"  Máximo de {max_tentativas_por_pagina} tentativas atingido. Desistindo.")
                    break 

        if not sucesso_pagina and total_de_paginas_api > 0 : 
             print(f"AVISO: Falha ao processar a página {pagina_atual} após todas as tentativas.")
        
        if pagina_atual < total_de_paginas_api and total_de_paginas_api > 0:
            time.sleep(1) # Delay entre páginas bem-sucedidas
        
        if total_de_paginas_api == 0: 
            break
        
        pagina_atual += 1

    print(f"\nExtração para '{endpoint_display_name}' concluída (ou tentativas esgotadas).")

    if not todos_os_itens_lista:
        print(f"Nenhum item de '{endpoint_display_name}' foi extraído da API para processar.")
    else:
        print(f"Total de {len(todos_os_itens_lista)} registros de '{endpoint_display_name}' extraídos para processamento.")

        # NF-e pode ter estruturas muito aninhadas, especialmente para itens (detalhe).
        # pd.json_normalize achatará o primeiro nível.
        # Para listas de itens (ex: 'ide.det', 'detalhes.produtos'), pode ser necessário um segundo passo de normalização
        # ou usar 'record_path' se você quiser uma linha por item da nota.
        df_itens = pd.json_normalize(todos_os_itens_lista, errors='ignore') # 'ignore' para campos que podem não existir em todos os registros
        
        print("\n--- DataFrame Inicial (após json_normalize) ---")
        if not df_itens.empty:
            print(df_itens.head())
        else:
            print("DataFrame inicial está vazio.")
        
        # Exemplo de como processar uma coluna que contém uma lista de itens (ex: 'detalhes.produtos')
        # Se 'detalhes.produtos' for uma coluna no df_itens contendo listas de dicionários:
        # if 'detalhes.produtos' in df_itens.columns:
        #     # Cria um novo DataFrame com uma linha por produto, mantendo o ID da NF-e
        #     df_produtos_nfe = pd.json_normalize(
        #         data=todos_os_itens_lista, # Usa a lista original para ter acesso ao ID da NF
        #         record_path=['detalhes', 'produtos'], # Caminho para a lista de produtos
        #         meta=[['ide','nNF'], ['ide','serie'], ['emitente','cnpj']], # Campos da NF-e para repetir em cada linha de produto
        #         meta_prefix='nf_',
        #         errors='ignore'
        #     )
        #     print("\n--- DataFrame de Itens da NF-e (Exemplo) ---")
        #     print(df_produtos_nfe.head())
            # Você poderia então juntar (merge) df_produtos_nfe com df_itens ou usá-lo separadamente.

        df_processado = df_itens.replace('', pd.NA)
        df_processado = df_processado.fillna('N/A')

        print("\n--- DataFrame após preencher valores nulos/vazios ---")
        if not df_processado.empty:
            print(df_processado.head())
        else:
            print("DataFrame processado está vazio.")


        print("\n--- Colunas Disponíveis para Seleção/Renomeação ---")
        if not df_processado.empty:
            colunas_disponiveis = df_processado.columns.tolist()
            for col_name in colunas_disponiveis:
                print(f"- {col_name}")
        else:
            print("Nenhuma coluna disponível, DataFrame está vazio.")

        # --- AJUSTE AQUI: Defina as colunas que quer manter e os novos nomes para NF-e (Consultas) ---
        # Este é um EXEMPLO. Você PRECISARÁ ajustar com base na saída de "Colunas Disponíveis".
        # A estrutura da NF-e é complexa, então os nomes das colunas podem ser como 'ide.nNF', 'emit.CNPJ', 'dest.xNome', 'total.ICMSTot.vNF' etc.
        mapa_renomeacao_e_selecao = {
            # Campos da Identificação da NF-e (ide)
            'ide.nNF': 'Número NF',
            'ide.serie': 'Série NF',
            'ide.dEmi': 'Data Emissão',
            'ide.dhEmi': 'Data/Hora Emissão',
            'ide.natOp': 'Natureza Operação',
            'ide.tpNF': 'Tipo NF (0=Entrada, 1=Saída)',
            # Campos do Emitente (emit)
            'emit.CNPJ': 'CNPJ Emitente',
            'emit.xNome': 'Nome Emitente',
            # Campos do Destinatário (dest)
            'dest.CNPJ': 'CNPJ Destinatário', # Ou 'dest.CPF'
            'dest.xNome': 'Nome Destinatário',
            # Campos de Totais (total)
            'total.ICMSTot.vProd': 'Valor Produtos',
            'total.ICMSTot.vFrete': 'Valor Frete',
            'total.ICMSTot.vSeg': 'Valor Seguro',
            'total.ICMSTot.vDesc': 'Valor Desconto',
            'total.ICMSTot.vOutro': 'Outras Despesas',
            'total.ICMSTot.vNF': 'Valor Total NF',
            # Campos de Informações Adicionais (infAdic)
            'infAdic.infCpl': 'Informações Complementares',
            # Campos de Status da NF-e (geralmente no nível raiz ou em um grupo específico)
            'compl.cStat': 'Status Sefaz', # Exemplo, pode variar
            'compl.xMotivo': 'Motivo Sefaz', # Exemplo
            'nfCancelada': 'NF Cancelada (S/N)', # Exemplo, se houver um campo direto

            # Se você normalizou os itens separadamente, pode não precisar de colunas de item aqui.
            # Se os itens foram achatados diretamente (ex: 'det.0.prod.cProd', 'det.1.prod.cProd'),
            # você teria que listá-los aqui se quisesse mantê-los.
        }

        df_final = pd.DataFrame() 
        if not df_processado.empty:
            colunas_para_manter_renomeadas = {
                antigo: novo for antigo, novo in mapa_renomeacao_e_selecao.items() if antigo in df_processado.columns
            }
            colunas_antigas_selecionadas = list(colunas_para_manter_renomeadas.keys())

            if not colunas_antigas_selecionadas:
                print(f"\nAVISO: Nenhuma coluna do 'mapa_renomeacao_e_selecao' para '{endpoint_display_name}' foi encontrada no DataFrame.")
                df_final = df_processado.copy() 
            else:
                df_final = df_processado[colunas_antigas_selecionadas].rename(columns=colunas_para_manter_renomeadas)

                # --- AJUSTE AQUI: Defina a ordem desejada para as colunas de NF-e ---
                ordem_desejada = [
                    'Número NF', 'Série NF', 'Data Emissão', 'Data/Hora Emissão', 
                    'Tipo NF (0=Entrada, 1=Saída)', 'Natureza Operação',
                    'CNPJ Emitente', 'Nome Emitente',
                    'CNPJ Destinatário', 'Nome Destinatário',
                    'Valor Produtos', 'Valor Frete', 'Valor Seguro', 'Valor Desconto', 
                    'Outras Despesas', 'Valor Total NF',
                    'Status Sefaz', 'Motivo Sefaz', 'NF Cancelada (S/N)',
                    'Informações Complementares'
                ]
                ordem_existente = [col for col in ordem_desejada if col in df_final.columns]
                for col in df_final.columns: 
                    if col not in ordem_existente:
                        ordem_existente.append(col)
                if ordem_existente: 
                    df_final = df_final[ordem_existente]
        else:
            print("DataFrame processado está vazio, não é possível criar df_final com colunas selecionadas.")


        print("\n--- DataFrame Final (Limpado, Selecionado e Renomeado) ---")
        if not df_final.empty:
            print(df_final.head())
            print(f"\nTotal de {len(df_final)} itens de '{endpoint_display_name}' no DataFrame final.")

            df_final.to_csv(nome_arquivo_saida_csv, index=False, sep=';', encoding='utf-8-sig')
            print(f"\nDataFrame final salvo em: {nome_arquivo_saida_csv}")
        else:
            print("DataFrame final está vazio. Nenhum arquivo CSV será gerado.")

except requests.exceptions.HTTPError as http_err:
    print(f"Erro HTTP ocorreu durante a chamada à API para '{endpoint_display_name}': {http_err}")
    if 'response' in locals() and response is not None:
        print(f"Conteúdo da resposta: {response.text}")
except requests.exceptions.ConnectionError as conn_err: 
    print(f"Erro de Conexão ocorreu durante a chamada à API para '{endpoint_display_name}': {conn_err}")
except requests.exceptions.Timeout as timeout_err: 
    print(f"Timeout ocorreu durante a chamada à API para '{endpoint_display_name}': {timeout_err}")
except requests.exceptions.RequestException as req_err: 
    print(f"Erro de Requisição ocorreu durante a chamada à API para '{endpoint_display_name}': {req_err}")
except json.JSONDecodeError as json_err:
    print(f"Erro ao decodificar JSON da resposta da API para '{endpoint_display_name}': {json_err}")
    if 'response' in locals() and response is not None: 
        print(f"Conteúdo da resposta que causou o erro: {response.text}")
except Exception as e:
    print(f"Um erro inesperado ocorreu durante o processamento de '{endpoint_display_name}': {e}")

Iniciando extração de 'NF-e (Consultas)' da API Omie.
URL Base: https://app.omie.com.br/api/v1/produtos/nfconsultar/
Buscando dados para 'NF-e (Consultas)' (Página 1 de ..., Tentativa 1/3)...
API informou: 103 página(s) e 2060 registro(s) no total para 'NF-e (Consultas)'.
Chave de dados JSON identificada para a lista de itens: 'nfCadastro'
  20 itens adicionados. Total acumulado: 20
Buscando dados para 'NF-e (Consultas)' (Página 2 de 103, Tentativa 1/3)...
  20 itens adicionados. Total acumulado: 40
Buscando dados para 'NF-e (Consultas)' (Página 3 de 103, Tentativa 1/3)...
  20 itens adicionados. Total acumulado: 60
Buscando dados para 'NF-e (Consultas)' (Página 4 de 103, Tentativa 1/3)...
  20 itens adicionados. Total acumulado: 80
Buscando dados para 'NF-e (Consultas)' (Página 5 de 103, Tentativa 1/3)...
  20 itens adicionados. Total acumulado: 100
Buscando dados para 'NF-e (Consultas)' (Página 6 de 103, Tentativa 1/3)...
  20 itens adicionados. Total acumulado: 120
Buscando dados pa

In [23]:

APP_KEY = "596450736882"
APP_SECRET = "37a10edb420d8cd6c7afddcaaffcac05"

# --- CONFIGURAÇÃO PARA EXTRAÇÃO DE SERVIÇOS ---
endpoint_name = "Serviços" # Nome da categoria para exibição e uso no DataFrame
base_module = "servicos"   # Módulo para Serviços na Omie API, conforme sua URL
api_path = "servico"       # Parte final da URL do endpoint
call_method = "ListarCadastroServico" # Nome do método de chamada da API
# Chave que contém a lista de serviços na resposta JSON (pode variar, verificar docs Omie)
# Tentaremos 'servicoCadastro', 'listaServicos', 'listaRegistros' ou 'servicos'
json_data_keys_to_try = ["servicoCadastro", "listaServicos", "listaRegistros", "servicos"]


base_url = f"https://app.omie.com.br/api/v1/{base_module}/{api_path}/" 

headers = {
    'Content-Type': 'application/json'
}

# Parâmetros da requisição para Serviços
body = {
    "call": call_method,
    "app_key": APP_KEY,
    "app_secret": APP_SECRET,
    "param": [
        {
            "nPagina": 1,
            "nRegPorPagina": 20 # Você especificou 20 aqui
        }
    ]
}

print(f"Iniciando extração de dados para: {endpoint_name}")
print(f"URL da Requisição: {base_url}")
print(f"Método da Requisição (call): {call_method}")
print("-" * 50)

df_servicos = pd.DataFrame() # DataFrame vazio para armazenar os dados de serviços

try:
    response = requests.post(base_url, headers=headers, data=json.dumps(body))
    response.raise_for_status() # Lança uma exceção para códigos de status de erro (4xx ou 5xx)

    print(f"Dados de {endpoint_name} extraídos com sucesso!")
    
    data = response.json()

    # Tentar extrair a lista de serviços da resposta JSON usando as chaves prováveis
    found_key = None
    for key_attempt in json_data_keys_to_try:
        if key_attempt in data and isinstance(data.get(key_attempt), list):
            df_servicos = pd.DataFrame(data[key_attempt])
            found_key = key_attempt
            break # Sair do loop assim que a chave for encontrada

    if found_key:
        print(f"Chave de dados JSON identificada: '{found_key}'")
        print(f"\n--- Análise Exploratória para {endpoint_name} ---")
        print(f"Shape do DataFrame: {df_servicos.shape}")
        print("\nPrimeiras 5 linhas:")
        print(df_servicos.head())

        print("\nInformações do DataFrame (tipos de dados e contagem de não-nulos):")
        buffer = io.StringIO()
        df_servicos.info(buf=buffer, verbose=True, show_counts=True)
        print(buffer.getvalue())

        print("\nContagem de Valores Nulos por Coluna:")
        print(df_servicos.isnull().sum())
        print("-" * 50)
    else:
        print(f"Atenção: Não foi possível identificar a chave dos dados para '{endpoint_name}' na resposta da API.")
        print("Resposta completa da API (para depuração):", json.dumps(data, indent=2, ensure_ascii=False))
        print("-" * 50)

except requests.exceptions.HTTPError as http_err:
    print(f"Erro HTTP ao extrair {endpoint_name}: {http_err}")
    print(f"Resposta da API: {response.text}")
    print("-" * 50)
except requests.exceptions.ConnectionError as conn_err:
    print(f"Erro de Conexão ao extrair {endpoint_name}: {conn_err}")
    print("Verifique sua conexão com a internet ou o status da API Omie.")
    print("-" * 50)
except requests.exceptions.Timeout as timeout_err:
    print(f"Erro de Timeout ao extrair {endpoint_name}: {timeout_err}")
    print("A requisição demorou muito para responder.")
    print("-" * 50)
except requests.exceptions.RequestException as req_err:
    print(f"Erro inesperado ao extrair {endpoint_name}: {req_err}")
    print("-" * 50)
except json.JSONDecodeError:
    print(f"Erro ao decodificar JSON para {endpoint_name}. Resposta não é JSON válido.")
    print(f"Resposta da API: {response.text}")
    print("-" * 50)
except Exception as e:
    print(f"Ocorreu um erro geral ao processar {endpoint_name}: {e}")
    print("-" * 50)

print("\n--- Extração de Serviços concluída! ---")

# Agora você tem o DataFrame 'df_servicos' com os dados extraídos.
# Você pode fazer mais análises ou salvá-lo:
# df_servicos.to_csv('servicos_omie.csv', index=False, encoding='utf-8')
# df_servicos.to_excel('servicos_omie.xlsx', index=False)

Iniciando extração de dados para: Serviços
URL da Requisição: https://app.omie.com.br/api/v1/servicos/servico/
Método da Requisição (call): ListarCadastroServico
--------------------------------------------------
Dados de Serviços extraídos com sucesso!
Atenção: Não foi possível identificar a chave dos dados para 'Serviços' na resposta da API.
Resposta completa da API (para depuração): {
  "nPagina": 1,
  "nTotPaginas": 1,
  "nRegistros": 3,
  "nTotRegistros": 3,
  "cadastros": [
    {
      "cabecalho": {
        "cCodCateg": "",
        "cCodLC116": "7.17",
        "cCodServMun": "01805",
        "cCodigo": "SRV00001",
        "cDescricao": "Acompanhamento e fiscalização",
        "cIdTrib": "01",
        "cTipoDesc": "",
        "nAliqDesc": 0,
        "nIdNBS": "",
        "nPrecoUnit": 0,
        "nValorDesc": 0
      },
      "descricao": {
        "cDescrCompleta": "DADOS PARA DEPÓSITO||BANCO DO BRASIL|AGÊNCIA 0303-4|CONTA 106.745-1"
      },
      "impostos": {
        "cRetCOF

In [24]:

APP_KEY = "596450736882"
APP_SECRET = "37a10edb420d8cd6c7afddcaaffcac05"

# --- CONFIGURAÇÕES ESPECÍFICAS PARA O ENDPOINT DE SERVIÇOS ---
endpoint_display_name = "Serviços" # Nome para exibição
base_module = "servicos"
api_path = "servico" 
api_call_name = "ListarCadastroServico"

# Chaves prováveis no JSON da API que contêm a lista de itens (serviços)
# O script tentará estas chaves em ordem.
json_data_keys_to_try = ["servicoCadastro", "listaServicos", "listaRegistros", "servicos", "cadastro"] 

# Nome do arquivo de saída
nome_arquivo_saida_csv = 'omie_servicos_limpos_todos.csv'
nome_arquivo_saida_excel = 'omie_servicos_limpos_todos.xlsx'
# -----------------------------------------------------------------

versao = "v1" # Geralmente v1 para endpoints gerais da Omie
base_url = f"https://app.omie.com.br/api/{versao}/{base_module}/{api_path}/"

headers = {
    'Content-Type': 'application/json'
}

# Parâmetros iniciais
pagina_atual_api = 1 # Usaremos nPagina para a API Omie neste endpoint
registros_por_pagina_api = 20 # Conforme seu exemplo para Serviços
max_tentativas_por_pagina = 3
delay_entre_tentativas_base = 5

print(f"Iniciando extração de '{endpoint_display_name}' da API Omie.")
print(f"URL Base: {base_url}")

todos_os_itens_lista = []
total_de_paginas_api = 1 
chave_lista_itens_api_identificada = None 

try:
    while pagina_atual_api <= total_de_paginas_api:
        # Corpo da requisição específico para ListarCadastroServico
        body = {
            'call': api_call_name,
            'app_key': APP_KEY,
            'app_secret': APP_SECRET,
            'param': [
                {
                    "nPagina": pagina_atual_api, # Omie usa nPagina para este endpoint
                    "nRegPorPagina": registros_por_pagina_api # Omie usa nRegPorPagina
                    # Adicione outros filtros se necessário, ex: "cGrupo", "dAltDe", "dAltAte"
                }
            ]
        }

        tentativa_atual = 0
        sucesso_pagina = False
        dados_api_pagina = None 

        while tentativa_atual < max_tentativas_por_pagina and not sucesso_pagina:
            tentativa_atual += 1
            print(f"Buscando dados para '{endpoint_display_name}' (Página {pagina_atual_api} de {total_de_paginas_api if total_de_paginas_api > 1 else '...'}, Tentativa {tentativa_atual}/{max_tentativas_por_pagina})...")
            
            try:
                response = requests.post(base_url, headers=headers, data=json.dumps(body), timeout=60) 
                response.raise_for_status()
                
                dados_api_pagina = response.json()
                sucesso_pagina = True 

                # A API ListarCadastroServico usa nPagina, nTotalPaginas, nRegistros, nTotalRegistros
                if pagina_atual_api == 1 and chave_lista_itens_api_identificada is None: 
                    total_de_paginas_api = dados_api_pagina.get('nTotalPaginas', 1) 
                    total_de_registros_api = dados_api_pagina.get('nTotalRegistros', 0) 
                    
                    print(f"API informou: {total_de_paginas_api} página(s) e {total_de_registros_api} registro(s) no total para '{endpoint_display_name}'.")
                    
                    if total_de_registros_api == 0:
                        print(f"Nenhum registro encontrado na API para '{endpoint_display_name}'.")
                        total_de_paginas_api = 0 
                        break

                    for key_attempt in json_data_keys_to_try:
                        # ListarCadastroServico geralmente tem a lista dentro de 'servico_cadastro'
                        # mas a resposta da API que você está usando pode ser 'servicoCadastro'
                        if key_attempt in dados_api_pagina and isinstance(dados_api_pagina.get(key_attempt), list):
                            chave_lista_itens_api_identificada = key_attempt
                            print(f"Chave de dados JSON identificada para a lista de itens: '{chave_lista_itens_api_identificada}'")
                            break 
                    
                    if not chave_lista_itens_api_identificada:
                        print(f"ATENÇÃO: Não foi possível identificar a chave da lista de itens para '{endpoint_display_name}' na resposta da API.")
                        print("Resposta da API (primeira página):", json.dumps(dados_api_pagina, indent=2, ensure_ascii=False))
                        total_de_paginas_api = 0 
                        break 
                    
                    if total_de_registros_api == 0 and chave_lista_itens_api_identificada and not dados_api_pagina.get(chave_lista_itens_api_identificada):
                        print(f"Chave '{chave_lista_itens_api_identificada}' encontrada, mas está vazia. Nenhum registro.")
                        total_de_paginas_api = 0
                        break

                if sucesso_pagina and chave_lista_itens_api_identificada:
                    itens_da_pagina = dados_api_pagina.get(chave_lista_itens_api_identificada, [])
                    if itens_da_pagina:
                        todos_os_itens_lista.extend(itens_da_pagina)
                        print(f"  {len(itens_da_pagina)} itens adicionados. Total acumulado: {len(todos_os_itens_lista)}")
                    else:
                        print(f"  Nenhum item encontrado na página {pagina_atual_api} (usando chave '{chave_lista_itens_api_identificada}').")
                elif sucesso_pagina and not chave_lista_itens_api_identificada and total_de_registros_api > 0 and pagina_atual_api == 1 :
                    print(f"  Sucesso ao buscar dados, mas a chave da lista de itens não foi identificada. Verifique 'json_data_keys_to_try'.")
                    total_de_paginas_api = 0 
                    break

            except (requests.exceptions.ConnectionError, requests.exceptions.Timeout, requests.exceptions.HTTPError) as e:
                print(f"  Erro ao buscar dados (tentativa {tentativa_atual}): {e}")
                if tentativa_atual < max_tentativas_por_pagina:
                    delay_atual = delay_entre_tentativas_base * (2 ** (tentativa_atual - 1))
                    print(f"  Aguardando {delay_atual} segundos antes de tentar novamente...")
                    time.sleep(delay_atual)
                else:
                    print(f"  Máximo de {max_tentativas_por_pagina} tentativas atingido. Desistindo.")
                    break 

        if not sucesso_pagina and total_de_paginas_api > 0 : 
             print(f"AVISO: Falha ao processar a página {pagina_atual_api} após todas as tentativas.")
        
        if pagina_atual_api < total_de_paginas_api and total_de_paginas_api > 0:
            time.sleep(1) # Delay entre páginas bem-sucedidas
        
        if total_de_paginas_api == 0: 
            break
        
        pagina_atual_api += 1

    print(f"\nExtração para '{endpoint_display_name}' concluída (ou tentativas esgotadas).")

    if not todos_os_itens_lista:
        print(f"Nenhum item de '{endpoint_display_name}' foi extraído da API para processar.")
    else:
        print(f"Total de {len(todos_os_itens_lista)} registros de '{endpoint_display_name}' extraídos para processamento.")

        df_itens = pd.json_normalize(todos_os_itens_lista, errors='ignore') 
        
        print("\n--- DataFrame Inicial (após json_normalize) ---")
        if not df_itens.empty:
            print(df_itens.head())
        else:
            print("DataFrame inicial está vazio.")
        
        # Exemplo de como processar campos aninhados específicos para serviços, se houver.
        # Por exemplo, 'impostos' pode ser um dicionário aninhado.
        # if 'impostos.pis_pct_aliquota' in df_itens.columns:
        # print("Campo de imposto PIS encontrado.")
        # O json_normalize já deve ter achatado esses campos.

        df_processado = df_itens.replace('', pd.NA)
        df_processado = df_processado.fillna('N/A')

        print("\n--- DataFrame após preencher valores nulos/vazios ---")
        if not df_processado.empty:
            print(df_processado.head())
        else:
            print("DataFrame processado está vazio.")


        print("\n--- Colunas Disponíveis para Seleção/Renomeação ---")
        if not df_processado.empty:
            colunas_disponiveis = df_processado.columns.tolist()
            for col_name in colunas_disponiveis:
                print(f"- {col_name}")
        else:
            print("Nenhuma coluna disponível, DataFrame está vazio.")

        # --- AJUSTE AQUI: Defina as colunas que quer manter e os novos nomes para SERVIÇOS ---
        # Este é um EXEMPLO. Você PRECISARÁ ajustar com base na saída de "Colunas Disponíveis".
        mapa_renomeacao_e_selecao = {
            'cabecalho.codigo_servico': 'ID Servico Omie', # Exemplo, pode ser só 'codigo_servico'
            'cabecalho.descricao': 'Descrição Serviço',
            'cabecalho.codigo_servico_integracao': 'ID Integração Serviço',
            'cabecalho.valor_unitario': 'Valor Unitário',
            'cabecalho.unidade': 'Unidade',
            'cabecalho.inativo': 'Inativo (S/N)',
            'cabecalho.ncm': 'NCM',
            'cabecalho.id_tributacao_municipal': 'ID Trib. Municipal',
            # Campos de impostos (achatados pelo json_normalize)
            'impostos.aliquota_cofins': 'Alíquota COFINS (%)',
            'impostos.aliquota_csll': 'Alíquota CSLL (%)',
            'impostos.aliquota_irpj': 'Alíquota IRPJ (%)',
            'impostos.aliquota_pis': 'Alíquota PIS (%)',
            'impostos.iss_aliquota': 'Alíquota ISS (%)',
            'impostos.iss_retido_fonte': 'ISS Retido Fonte (S/N)',
            # Informações de inclusão/alteração (geralmente dentro de 'int оригинальныйInfo')
            'intInfo.dInc': 'Data Inclusão',
            'intInfo.hInc': 'Hora Inclusão',
            'intInfo.uInc': 'Usuário Inclusão',
            'intInfo.dAlt': 'Data Alteração',
            'intInfo.hAlt': 'Hora Alteração',
            'intInfo.uAlt': 'Usuário Alteração',
        }

        df_final = pd.DataFrame() 
        if not df_processado.empty:
            colunas_para_manter_renomeadas = {
                antigo: novo for antigo, novo in mapa_renomeacao_e_selecao.items() if antigo in df_processado.columns
            }
            colunas_antigas_selecionadas = list(colunas_para_manter_renomeadas.keys())

            if not colunas_antigas_selecionadas:
                print(f"\nAVISO: Nenhuma coluna do 'mapa_renomeacao_e_selecao' para '{endpoint_display_name}' foi encontrada no DataFrame.")
                df_final = df_processado.copy() 
            else:
                df_final = df_processado[colunas_antigas_selecionadas].rename(columns=colunas_para_manter_renomeadas)

                # --- AJUSTE AQUI: Defina a ordem desejada para as colunas de SERVIÇOS ---
                ordem_desejada = [
                    'ID Servico Omie', 'Descrição Serviço', 'ID Integração Serviço', 'Valor Unitário', 'Unidade',
                    'NCM', 'ID Trib. Municipal', 'Inativo (S/N)',
                    'Alíquota PIS (%)', 'Alíquota COFINS (%)', 'Alíquota CSLL (%)', 'Alíquota IRPJ (%)', 
                    'Alíquota ISS (%)', 'ISS Retido Fonte (S/N)',
                    'Data Inclusão', 'Hora Inclusão', 'Usuário Inclusão',
                    'Data Alteração', 'Hora Alteração', 'Usuário Alteração'
                ]
                ordem_existente = [col for col in ordem_desejada if col in df_final.columns]
                for col in df_final.columns: 
                    if col not in ordem_existente:
                        ordem_existente.append(col)
                if ordem_existente: 
                    df_final = df_final[ordem_existente]
        else:
            print("DataFrame processado está vazio, não é possível criar df_final com colunas selecionadas.")


        print("\n--- DataFrame Final (Limpado, Selecionado e Renomeado) ---")
        if not df_final.empty:
            print(df_final.head())
            print(f"\nTotal de {len(df_final)} itens de '{endpoint_display_name}' no DataFrame final.")

            df_final.to_csv(nome_arquivo_saida_csv, index=False, sep=';', encoding='utf-8-sig')
            print(f"\nDataFrame final salvo em: {nome_arquivo_saida_csv}")
        else:
            print("DataFrame final está vazio. Nenhum arquivo CSV será gerado.")

except requests.exceptions.HTTPError as http_err:
    print(f"Erro HTTP ocorreu durante a chamada à API para '{endpoint_display_name}': {http_err}")
    if 'response' in locals() and response is not None:
        print(f"Conteúdo da resposta: {response.text}")
except requests.exceptions.ConnectionError as conn_err: 
    print(f"Erro de Conexão ocorreu durante a chamada à API para '{endpoint_display_name}': {conn_err}")
except requests.exceptions.Timeout as timeout_err: 
    print(f"Timeout ocorreu durante a chamada à API para '{endpoint_display_name}': {timeout_err}")
except requests.exceptions.RequestException as req_err: 
    print(f"Erro de Requisição ocorreu durante a chamada à API para '{endpoint_display_name}': {req_err}")
except json.JSONDecodeError as json_err:
    print(f"Erro ao decodificar JSON da resposta da API para '{endpoint_display_name}': {json_err}")
    if 'response' in locals() and response is not None: 
        print(f"Conteúdo da resposta que causou o erro: {response.text}")
except Exception as e:
    print(f"Um erro inesperado ocorreu durante o processamento de '{endpoint_display_name}': {e}")

Iniciando extração de 'Serviços' da API Omie.
URL Base: https://app.omie.com.br/api/v1/servicos/servico/
Buscando dados para 'Serviços' (Página 1 de ..., Tentativa 1/3)...
API informou: 1 página(s) e 0 registro(s) no total para 'Serviços'.
Nenhum registro encontrado na API para 'Serviços'.

Extração para 'Serviços' concluída (ou tentativas esgotadas).
Nenhum item de 'Serviços' foi extraído da API para processar.


In [25]:

APP_KEY = "596450736882"
APP_SECRET = "37a10edb420d8cd6c7afddcaaffcac05"

# --- CONFIGURAÇÃO PARA EXTRAÇÃO DE ORDENS DE SERVIÇO ---
endpoint_name = "Ordens de Serviço" # Nome da categoria para exibição e uso no DataFrame
base_module = "servicos"             # Módulo para Ordens de Serviço na Omie API, conforme sua URL
api_path = "os"                      # Parte final da URL do endpoint
call_method = "ListarOS"             # Nome do método de chamada da API
# Chave que contém a lista de ordens de serviço na resposta JSON (pode variar, verificar docs Omie)
# Tentaremos 'osCadastro', 'listaOS', 'listaRegistros' ou 'ordensServico'
json_data_keys_to_try = ["osCadastro", "listaOS", "listaRegistros", "ordensServico"]


base_url = f"https://app.omie.com.br/api/v1/{base_module}/{api_path}/" 

headers = {
    'Content-Type': 'application/json'
}

# Parâmetros da requisição para Ordens de Serviço
body = {
    "call": call_method,
    "app_key": APP_KEY,
    "app_secret": APP_SECRET,
    "param": [
        {
            "pagina": 1,
            "registros_por_pagina": 50, # Você especificou 50 aqui
            "apenas_importado_api": "N"
        }
    ]
}

print(f"Iniciando extração de dados para: {endpoint_name}")
print(f"URL da Requisição: {base_url}")
print(f"Método da Requisição (call): {call_method}")
print("-" * 50)

df_ordens_servico = pd.DataFrame() # DataFrame vazio para armazenar os dados de ordens de serviço

try:
    response = requests.post(base_url, headers=headers, data=json.dumps(body))
    response.raise_for_status() # Lança uma exceção para códigos de status de erro (4xx ou 5xx)

    print(f"Dados de {endpoint_name} extraídos com sucesso!")
    
    data = response.json()

    # Tentar extrair a lista de ordens de serviço da resposta JSON usando as chaves prováveis
    found_key = None
    for key_attempt in json_data_keys_to_try:
        if key_attempt in data and isinstance(data.get(key_attempt), list):
            df_ordens_servico = pd.DataFrame(data[key_attempt])
            found_key = key_attempt
            break # Sair do loop assim que a chave for encontrada

    if found_key:
        print(f"Chave de dados JSON identificada: '{found_key}'")
        print(f"\n--- Análise Exploratória para {endpoint_name} ---")
        print(f"Shape do DataFrame: {df_ordens_servico.shape}")
        print("\nPrimeiras 5 linhas:")
        print(df_ordens_servico.head())

        print("\nInformações do DataFrame (tipos de dados e contagem de não-nulos):")
        buffer = io.StringIO()
        df_ordens_servico.info(buf=buffer, verbose=True, show_counts=True)
        print(buffer.getvalue())

        print("\nContagem de Valores Nulos por Coluna:")
        print(df_ordens_servico.isnull().sum())
        print("-" * 50)
    else:
        print(f"Atenção: Não foi possível identificar a chave dos dados para '{endpoint_name}' na resposta da API.")
        print("Resposta completa da API (para depuração):", json.dumps(data, indent=2, ensure_ascii=False))
        print("-" * 50)

except requests.exceptions.HTTPError as http_err:
    print(f"Erro HTTP ao extrair {endpoint_name}: {http_err}")
    print(f"Resposta da API: {response.text}")
    print("-" * 50)
except requests.exceptions.ConnectionError as conn_err:
    print(f"Erro de Conexão ao extrair {endpoint_name}: {conn_err}")
    print("Verifique sua conexão com a internet ou o status da API Omie.")
    print("-" * 50)
except requests.exceptions.Timeout as timeout_err:
    print(f"Erro de Timeout ao extrair {endpoint_name}: {timeout_err}")
    print("A requisição demorou muito para responder.")
    print("-" * 50)
except requests.exceptions.RequestException as req_err:
    print(f"Erro inesperado ao extrair {endpoint_name}: {req_err}")
    print("-" * 50)
except json.JSONDecodeError:
    print(f"Erro ao decodificar JSON para {endpoint_name}. Resposta não é JSON válido.")
    print(f"Resposta da API: {response.text}")
    print("-" * 50)
except Exception as e:
    print(f"Ocorreu um erro geral ao processar {endpoint_name}: {e}")
    print("-" * 50)

print("\n--- Extração de Ordens de Serviço concluída! ---")

# Agora você tem o DataFrame 'df_ordens_servico' com os dados extraídos.
# Você pode fazer mais análises ou salvá-lo:
# df_ordens_servico.to_csv('ordens_servico_omie.csv', index=False, encoding='utf-8')
# df_ordens_servico.to_excel('ordens_servico_omie.xlsx', index=False)

Iniciando extração de dados para: Ordens de Serviço
URL da Requisição: https://app.omie.com.br/api/v1/servicos/os/
Método da Requisição (call): ListarOS
--------------------------------------------------
Dados de Ordens de Serviço extraídos com sucesso!
Chave de dados JSON identificada: 'osCadastro'

--- Análise Exploratória para Ordens de Serviço ---
Shape do DataFrame: (50, 8)

Primeiras 5 linhas:
                                           Cabecalho  \
0  {'cCodIntOS': '', 'cCodParc': '000', 'cEtapa':...   
1  {'cCodIntOS': '', 'cCodParc': '000', 'cEtapa':...   
2  {'cCodIntOS': '', 'cCodParc': '000', 'cEtapa':...   
3  {'cCodIntOS': '', 'cCodParc': '000', 'cEtapa':...   
4  {'cCodIntOS': '', 'cCodParc': '000', 'cEtapa':...   

                                       Departamentos  \
0  [{'cCodDepto': '7280778563', 'nPerc': 100, 'nV...   
1  [{'cCodDepto': '7280778563', 'nPerc': 100, 'nV...   
2  [{'cCodDepto': '7280778563', 'nPerc': 100, 'nV...   
3  [{'cCodDepto': '7280778563', 'nPe

In [26]:

APP_KEY = "596450736882"
APP_SECRET = "37a10edb420d8cd6c7afddcaaffcac05"

# --- CONFIGURAÇÕES ESPECÍFICAS PARA O ENDPOINT DE ORDENS DE SERVIÇO ---
endpoint_display_name = "Ordens de Serviço" # Nome para exibição
base_module = "servicos"
api_path = "os" 
api_call_name = "ListarOS"

# Chaves prováveis no JSON da API que contêm a lista de itens (Ordens de Serviço)
# O script tentará estas chaves em ordem.
json_data_keys_to_try = ["osCadastro", "listaOS", "listaRegistros", "ordensServico", "cadastro"] 

# Nome do arquivo de saída
nome_arquivo_saida_csv = 'omie_ordens_servico_limpas_todas.csv'
nome_arquivo_saida_excel = 'omie_ordens_servico_limpas_todas.xlsx'
# -----------------------------------------------------------------

versao = "v1" # Geralmente v1 para endpoints gerais da Omie
base_url = f"https://app.omie.com.br/api/{versao}/{base_module}/{api_path}/"

headers = {
    'Content-Type': 'application/json'
}

# Parâmetros iniciais
pagina_atual_api = 1 
registros_por_pagina_api = 50 # Conforme seu exemplo para Ordens de Serviço
max_tentativas_por_pagina = 3
delay_entre_tentativas_base = 5

print(f"Iniciando extração de '{endpoint_display_name}' da API Omie.")
print(f"URL Base: {base_url}")

todos_os_itens_lista = []
total_de_paginas_api = 1 
chave_lista_itens_api_identificada = None 

try:
    while pagina_atual_api <= total_de_paginas_api:
        # Corpo da requisição específico para ListarOS
        body = {
            'call': api_call_name,
            'app_key': APP_KEY,
            'app_secret': APP_SECRET,
            'param': [
                {
                    "pagina": pagina_atual_api, # Omie geralmente usa 'pagina' para ListarOS
                    "registros_por_pagina": registros_por_pagina_api,
                    "apenas_importado_api": "N"
                    # Adicione outros filtros se necessário, ex: "dDtDe", "dDtAte", "cStatus"
                }
            ]
        }

        tentativa_atual = 0
        sucesso_pagina = False
        dados_api_pagina = None 

        while tentativa_atual < max_tentativas_por_pagina and not sucesso_pagina:
            tentativa_atual += 1
            print(f"Buscando dados para '{endpoint_display_name}' (Página {pagina_atual_api} de {total_de_paginas_api if total_de_paginas_api > 1 else '...'}, Tentativa {tentativa_atual}/{max_tentativas_por_pagina})...")
            
            try:
                response = requests.post(base_url, headers=headers, data=json.dumps(body), timeout=60) 
                response.raise_for_status()
                
                dados_api_pagina = response.json()
                sucesso_pagina = True 

                # ListarOS geralmente usa 'pagina', 'total_de_paginas', 'registros', 'total_de_registros'
                if pagina_atual_api == 1 and chave_lista_itens_api_identificada is None: 
                    total_de_paginas_api = dados_api_pagina.get('total_de_paginas', dados_api_pagina.get('nTotalPaginas', 1)) 
                    total_de_registros_api = dados_api_pagina.get('total_de_registros', dados_api_pagina.get('nTotalRegistros', 0))
                    
                    print(f"API informou: {total_de_paginas_api} página(s) e {total_de_registros_api} registro(s) no total para '{endpoint_display_name}'.")
                    
                    if total_de_registros_api == 0:
                        print(f"Nenhum registro encontrado na API para '{endpoint_display_name}'.")
                        total_de_paginas_api = 0 
                        break

                    for key_attempt in json_data_keys_to_try:
                        if key_attempt in dados_api_pagina and isinstance(dados_api_pagina.get(key_attempt), list):
                            chave_lista_itens_api_identificada = key_attempt
                            print(f"Chave de dados JSON identificada para a lista de itens: '{chave_lista_itens_api_identificada}'")
                            break 
                    
                    if not chave_lista_itens_api_identificada:
                        print(f"ATENÇÃO: Não foi possível identificar a chave da lista de itens para '{endpoint_display_name}' na resposta da API.")
                        print("Resposta da API (primeira página):", json.dumps(dados_api_pagina, indent=2, ensure_ascii=False))
                        total_de_paginas_api = 0 
                        break 
                    
                    if total_de_registros_api == 0 and chave_lista_itens_api_identificada and not dados_api_pagina.get(chave_lista_itens_api_identificada):
                        print(f"Chave '{chave_lista_itens_api_identificada}' encontrada, mas está vazia. Nenhum registro.")
                        total_de_paginas_api = 0
                        break

                if sucesso_pagina and chave_lista_itens_api_identificada:
                    itens_da_pagina = dados_api_pagina.get(chave_lista_itens_api_identificada, [])
                    if itens_da_pagina:
                        todos_os_itens_lista.extend(itens_da_pagina)
                        print(f"  {len(itens_da_pagina)} itens adicionados. Total acumulado: {len(todos_os_itens_lista)}")
                    else:
                        print(f"  Nenhum item encontrado na página {pagina_atual_api} (usando chave '{chave_lista_itens_api_identificada}').")
                elif sucesso_pagina and not chave_lista_itens_api_identificada and total_de_registros_api > 0 and pagina_atual_api == 1 :
                    print(f"  Sucesso ao buscar dados, mas a chave da lista de itens não foi identificada. Verifique 'json_data_keys_to_try'.")
                    total_de_paginas_api = 0 
                    break

            except (requests.exceptions.ConnectionError, requests.exceptions.Timeout, requests.exceptions.HTTPError) as e:
                print(f"  Erro ao buscar dados (tentativa {tentativa_atual}): {e}")
                if tentativa_atual < max_tentativas_por_pagina:
                    delay_atual = delay_entre_tentativas_base * (2 ** (tentativa_atual - 1))
                    print(f"  Aguardando {delay_atual} segundos antes de tentar novamente...")
                    time.sleep(delay_atual)
                else:
                    print(f"  Máximo de {max_tentativas_por_pagina} tentativas atingido. Desistindo.")
                    break 

        if not sucesso_pagina and total_de_paginas_api > 0 : 
             print(f"AVISO: Falha ao processar a página {pagina_atual_api} após todas as tentativas.")
        
        if pagina_atual_api < total_de_paginas_api and total_de_paginas_api > 0:
            time.sleep(1) # Delay entre páginas bem-sucedidas
        
        if total_de_paginas_api == 0: 
            break
        
        pagina_atual_api += 1

    print(f"\nExtração para '{endpoint_display_name}' concluída (ou tentativas esgotadas).")

    if not todos_os_itens_lista:
        print(f"Nenhum item de '{endpoint_display_name}' foi extraído da API para processar.")
    else:
        print(f"Total de {len(todos_os_itens_lista)} registros de '{endpoint_display_name}' extraídos para processamento.")

        # Ordens de Serviço podem ter muitos campos aninhados (cabecalho, info, email, servicos_produtos, etc.)
        # json_normalize tentará achatar o máximo possível.
        df_itens = pd.json_normalize(todos_os_itens_lista, errors='ignore') 
        
        print("\n--- DataFrame Inicial (após json_normalize) ---")
        if not df_itens.empty:
            print(df_itens.head())
        else:
            print("DataFrame inicial está vazio.")
        
        # Processamento adicional para listas aninhadas dentro de cada OS (ex: serviços/produtos)
        # Se você quiser uma linha por serviço/produto da OS, usaria record_path.
        # Por enquanto, o json_normalize vai criar colunas como 'ServicosProdutos.0.descricao', 'ServicosProdutos.1.descricao'
        # ou pode deixar a coluna 'ServicosProdutos' como uma lista de dicts se não conseguir achatar.
        # Se for uma lista de dicts, você pode querer extrair informações dela:
        if 'ServicosProdutos' in df_itens.columns and not df_itens.empty: # Verifica se a coluna existe e o DF não está vazio
            # Exemplo: Concatenar descrições dos serviços/produtos
            try:
                df_itens['OS_Servicos_Descricoes'] = df_itens['ServicosProdutos'].apply(
                    lambda lista_sp: ', '.join(
                        [sp.get('descricao', '') for sp in lista_sp if isinstance(sp, dict)]
                    ) if isinstance(lista_sp, list) else ''
                )
                df_itens['OS_Servicos_Valores_Total'] = df_itens['ServicosProdutos'].apply(
                    lambda lista_sp: sum(
                        [sp.get('valor_total', 0) for sp in lista_sp if isinstance(sp, dict) and isinstance(sp.get('valor_total'), (int, float))]
                    ) if isinstance(lista_sp, list) else 0.0
                )
            except Exception as e:
                print(f"Erro ao processar coluna 'ServicosProdutos': {e}")
                if 'OS_Servicos_Descricoes' not in df_itens.columns: df_itens['OS_Servicos_Descricoes'] = ''
                if 'OS_Servicos_Valores_Total' not in df_itens.columns: df_itens['OS_Servicos_Valores_Total'] = 0.0

        df_processado = df_itens.replace('', pd.NA)
        df_processado = df_processado.fillna('N/A')

        print("\n--- DataFrame após preencher valores nulos/vazios ---")
        if not df_processado.empty:
            print(df_processado.head())
        else:
            print("DataFrame processado está vazio.")


        print("\n--- Colunas Disponíveis para Seleção/Renomeação ---")
        if not df_processado.empty:
            colunas_disponiveis = df_processado.columns.tolist()
            for col_name in colunas_disponiveis:
                print(f"- {col_name}")
        else:
            print("Nenhuma coluna disponível, DataFrame está vazio.")

        # --- AJUSTE AQUI: Defina as colunas que quer manter e os novos nomes para ORDENS DE SERVIÇO ---
        # Este é um EXEMPLO. Você PRECISARÁ ajustar com base na saída de "Colunas Disponíveis".
        # A estrutura da OS é complexa, então os nomes das colunas podem ser como 'Cabecalho.nCodOS', 'InformacoesAdicionais.cCodCateg', etc.
        mapa_renomeacao_e_selecao = {
            'Cabecalho.nCodOS': 'ID OS Omie',
            'Cabecalho.cNumOS': 'Número OS',
            'Cabecalho.dDtPrevisao': 'Data Previsão',
            'Cabecalho.nCodCli': 'ID Cliente OS',
            'Cabecalho.cCodParc': 'Código Parcela OS', # Se aplicável
            'Cabecalho.cEtapa': 'Etapa OS',
            'Cabecalho.cStatus': 'Status OS',
            'Cabecalho.nValorTotal': 'Valor Total OS',
            'InformacoesAdicionais.cCodCateg': 'Código Categoria OS',
            'InformacoesAdicionais.nCodCC': 'ID Centro Custo OS',
            'Email.cPara': 'E-mail Destinatário OS',
            # Campos de info (geralmente inclusão/alteração da OS)
            'Info.dInc': 'Data Inclusão OS',
            'Info.hInc': 'Hora Inclusão OS',
            'Info.uInc': 'Usuário Inclusão OS',
            'Info.dAlt': 'Data Alteração OS',
            'Info.hAlt': 'Hora Alteração OS',
            'Info.uAlt': 'Usuário Alteração OS',
            # Novas colunas criadas do processamento de ServicosProdutos
            'OS_Servicos_Descricoes': 'Descrições Serviços/Produtos da OS',
            'OS_Servicos_Valores_Total': 'Valor Total Serviços/Produtos da OS',
        }

        df_final = pd.DataFrame() 
        if not df_processado.empty:
            colunas_para_manter_renomeadas = {
                antigo: novo for antigo, novo in mapa_renomeacao_e_selecao.items() if antigo in df_processado.columns
            }
            colunas_antigas_selecionadas = list(colunas_para_manter_renomeadas.keys())

            if not colunas_antigas_selecionadas:
                print(f"\nAVISO: Nenhuma coluna do 'mapa_renomeacao_e_selecao' para '{endpoint_display_name}' foi encontrada no DataFrame.")
                df_final = df_processado.copy() 
            else:
                df_final = df_processado[colunas_antigas_selecionadas].rename(columns=colunas_para_manter_renomeadas)

                # --- AJUSTE AQUI: Defina a ordem desejada para as colunas de ORDENS DE SERVIÇO ---
                ordem_desejada = [
                    'ID OS Omie', 'Número OS', 'ID Cliente OS', 'Data Previsão', 'Etapa OS', 'Status OS', 
                    'Valor Total OS', 'Código Categoria OS', 'ID Centro Custo OS',
                    'Descrições Serviços/Produtos da OS', 'Valor Total Serviços/Produtos da OS',
                    'E-mail Destinatário OS',
                    'Data Inclusão OS', 'Hora Inclusão OS', 'Usuário Inclusão OS',
                    'Data Alteração OS', 'Hora Alteração OS', 'Usuário Alteração OS'
                ]
                ordem_existente = [col for col in ordem_desejada if col in df_final.columns]
                for col in df_final.columns: 
                    if col not in ordem_existente:
                        ordem_existente.append(col)
                if ordem_existente: 
                    df_final = df_final[ordem_existente]
        else:
            print("DataFrame processado está vazio, não é possível criar df_final com colunas selecionadas.")


        print("\n--- DataFrame Final (Limpado, Selecionado e Renomeado) ---")
        if not df_final.empty:
            print(df_final.head())
            print(f"\nTotal de {len(df_final)} itens de '{endpoint_display_name}' no DataFrame final.")

            df_final.to_csv(nome_arquivo_saida_csv, index=False, sep=';', encoding='utf-8-sig')
            print(f"\nDataFrame final salvo em: {nome_arquivo_saida_csv}")
        else:
            print("DataFrame final está vazio. Nenhum arquivo CSV será gerado.")

except requests.exceptions.HTTPError as http_err:
    print(f"Erro HTTP ocorreu durante a chamada à API para '{endpoint_display_name}': {http_err}")
    if 'response' in locals() and response is not None:
        print(f"Conteúdo da resposta: {response.text}")
except requests.exceptions.ConnectionError as conn_err: 
    print(f"Erro de Conexão ocorreu durante a chamada à API para '{endpoint_display_name}': {conn_err}")
except requests.exceptions.Timeout as timeout_err: 
    print(f"Timeout ocorreu durante a chamada à API para '{endpoint_display_name}': {timeout_err}")
except requests.exceptions.RequestException as req_err: 
    print(f"Erro de Requisição ocorreu durante a chamada à API para '{endpoint_display_name}': {req_err}")
except json.JSONDecodeError as json_err:
    print(f"Erro ao decodificar JSON da resposta da API para '{endpoint_display_name}': {json_err}")
    if 'response' in locals() and response is not None: 
        print(f"Conteúdo da resposta que causou o erro: {response.text}")
except Exception as e:
    print(f"Um erro inesperado ocorreu durante o processamento de '{endpoint_display_name}': {e}")

Iniciando extração de 'Ordens de Serviço' da API Omie.
URL Base: https://app.omie.com.br/api/v1/servicos/os/
Buscando dados para 'Ordens de Serviço' (Página 1 de ..., Tentativa 1/3)...
API informou: 86 página(s) e 4263 registro(s) no total para 'Ordens de Serviço'.
Chave de dados JSON identificada para a lista de itens: 'osCadastro'
  50 itens adicionados. Total acumulado: 50
Buscando dados para 'Ordens de Serviço' (Página 2 de 86, Tentativa 1/3)...
  50 itens adicionados. Total acumulado: 100
Buscando dados para 'Ordens de Serviço' (Página 3 de 86, Tentativa 1/3)...
  50 itens adicionados. Total acumulado: 150
Buscando dados para 'Ordens de Serviço' (Página 4 de 86, Tentativa 1/3)...
  50 itens adicionados. Total acumulado: 200
Buscando dados para 'Ordens de Serviço' (Página 5 de 86, Tentativa 1/3)...
  50 itens adicionados. Total acumulado: 250
Buscando dados para 'Ordens de Serviço' (Página 6 de 86, Tentativa 1/3)...
  50 itens adicionados. Total acumulado: 300
Buscando dados para '

In [27]:

APP_KEY = "596450736882"
APP_SECRET = "37a10edb420d8cd6c7afddcaaffcac05"

# --- CONFIGURAÇÃO PARA EXTRAÇÃO DE CONTRATOS DE SERVIÇO ---
endpoint_name = "Contratos de Serviço" # Nome da categoria para exibição e uso no DataFrame
base_module = "servicos"               # Módulo para Contratos de Serviço na Omie API, conforme sua URL
api_path = "contrato"                  # Parte final da URL do endpoint
call_method = "ListarContratos"        # Nome do método de chamada da API
# Chave que contém a lista de contratos de serviço na resposta JSON (pode variar, verificar docs Omie)
# Tentaremos 'contratoCadastro', 'listaContratos', 'listaRegistros' ou 'contratos'
json_data_keys_to_try = ["contratoCadastro", "listaContratos", "listaRegistros", "contratos"]


base_url = f"https://app.omie.com.br/api/v1/{base_module}/{api_path}/" 

headers = {
    'Content-Type': 'application/json'
}

# Parâmetros da requisição para Contratos de Serviço
body = {
    "call": call_method,
    "app_key": APP_KEY,
    "app_secret": APP_SECRET,
    "param": [
        {
            "pagina": 1,
            "registros_por_pagina": 50, # Você especificou 50 aqui
            "apenas_importado_api": "N"
        }
    ]
}

print(f"Iniciando extração de dados para: {endpoint_name}")
print(f"URL da Requisição: {base_url}")
print(f"Método da Requisição (call): {call_method}")
print("-" * 50)

df_contratos_servico = pd.DataFrame() # DataFrame vazio para armazenar os dados de contratos de serviço

try:
    response = requests.post(base_url, headers=headers, data=json.dumps(body))
    response.raise_for_status() # Lança uma exceção para códigos de status de erro (4xx ou 5xx)

    print(f"Dados de {endpoint_name} extraídos com sucesso!")
    
    data = response.json()

    # Tentar extrair a lista de contratos de serviço da resposta JSON usando as chaves prováveis
    found_key = None
    for key_attempt in json_data_keys_to_try:
        if key_attempt in data and isinstance(data.get(key_attempt), list):
            df_contratos_servico = pd.DataFrame(data[key_attempt])
            found_key = key_attempt
            break # Sair do loop assim que a chave for encontrada

    if found_key:
        print(f"Chave de dados JSON identificada: '{found_key}'")
        print(f"\n--- Análise Exploratória para {endpoint_name} ---")
        print(f"Shape do DataFrame: {df_contratos_servico.shape}")
        print("\nPrimeiras 5 linhas:")
        print(df_contratos_servico.head())

        print("\nInformações do DataFrame (tipos de dados e contagem de não-nulos):")
        buffer = io.StringIO()
        df_contratos_servico.info(buf=buffer, verbose=True, show_counts=True)
        print(buffer.getvalue())

        print("\nContagem de Valores Nulos por Coluna:")
        print(df_contratos_servico.isnull().sum())
        print("-" * 50)
    else:
        print(f"Atenção: Não foi possível identificar a chave dos dados para '{endpoint_name}' na resposta da API.")
        print("Resposta completa da API (para depuração):", json.dumps(data, indent=2, ensure_ascii=False))
        print("-" * 50)

except requests.exceptions.HTTPError as http_err:
    print(f"Erro HTTP ao extrair {endpoint_name}: {http_err}")
    print(f"Resposta da API: {response.text}")
    print("-" * 50)
except requests.exceptions.ConnectionError as conn_err:
    print(f"Erro de Conexão ao extrair {endpoint_name}: {conn_err}")
    print("Verifique sua conexão com a internet ou o status da API Omie.")
    print("-" * 50)
except requests.exceptions.Timeout as timeout_err:
    print(f"Erro de Timeout ao extrair {endpoint_name}: {timeout_err}")
    print("A requisição demorou muito para responder.")
    print("-" * 50)
except requests.exceptions.RequestException as req_err:
    print(f"Erro inesperado ao extrair {endpoint_name}: {req_err}")
    print("-" * 50)
except json.JSONDecodeError:
    print(f"Erro ao decodificar JSON para {endpoint_name}. Resposta não é JSON válido.")
    print(f"Resposta da API: {response.text}")
    print("-" * 50)
except Exception as e:
    print(f"Ocorreu um erro geral ao processar {endpoint_name}: {e}")
    print("-" * 50)

print("\n--- Extração de Contratos de Serviço concluída! ---")

# Agora você tem o DataFrame 'df_contratos_servico' com os dados extraídos.
# Você pode fazer mais análises ou salvá-lo:
# df_contratos_servico.to_csv('contratos_servico_omie.csv', index=False, encoding='utf-8')
# df_contratos_servico.to_excel('contratos_servico_omie.xlsx', index=False)

Iniciando extração de dados para: Contratos de Serviço
URL da Requisição: https://app.omie.com.br/api/v1/servicos/contrato/
Método da Requisição (call): ListarContratos
--------------------------------------------------
Dados de Contratos de Serviço extraídos com sucesso!
Chave de dados JSON identificada: 'contratoCadastro'

--- Análise Exploratória para Contratos de Serviço ---
Shape do DataFrame: (50, 7)

Primeiras 5 linhas:
                                           cabecalho  \
0  {'cCodIntCtr': '', 'cCodSit': '10', 'cNumCtr':...   
1  {'cCodIntCtr': '', 'cCodSit': '10', 'cNumCtr':...   
2  {'cCodIntCtr': '', 'cCodSit': '10', 'cNumCtr':...   
3  {'cCodIntCtr': '', 'cCodSit': '10', 'cNumCtr':...   
4  {'cCodIntCtr': '', 'cCodSit': '10', 'cNumCtr':...   

                                       departamentos  \
0  [{'cCodDep': '7280778563', 'cDesDep': '', 'nPe...   
1  [{'cCodDep': '7280778563', 'cDesDep': '', 'nPe...   
2  [{'cCodDep': '7280778563', 'cDesDep': '', 'nPe...   
3  [{'cC

In [28]:

APP_KEY = "596450736882"
APP_SECRET = "37a10edb420d8cd6c7afddcaaffcac05"

# --- CONFIGURAÇÕES ESPECÍFICAS PARA O ENDPOINT DE CONTRATOS DE SERVIÇO ---
endpoint_display_name = "Contratos de Serviço" # Nome para exibição
base_module = "servicos"
api_path = "contrato" 
api_call_name = "ListarContratos"

# Chaves prováveis no JSON da API que contêm a lista de itens (Contratos de Serviço)
# O script tentará estas chaves em ordem.
json_data_keys_to_try = ["contratoCadastro", "listaContratos", "listaRegistros", "contratos", "cadastro"] 

# Nome do arquivo de saída
nome_arquivo_saida_csv = 'omie_contratos_servico_limpos_todos.csv'
nome_arquivo_saida_excel = 'omie_contratos_servico_limpos_todos.xlsx'
# -----------------------------------------------------------------

versao = "v1" # Geralmente v1 para endpoints gerais da Omie
base_url = f"https://app.omie.com.br/api/{versao}/{base_module}/{api_path}/"

headers = {
    'Content-Type': 'application/json'
}

# Parâmetros iniciais
pagina_atual_api = 1 
registros_por_pagina_api = 50 # Conforme seu exemplo para Contratos de Serviço
max_tentativas_por_pagina = 3
delay_entre_tentativas_base = 5

print(f"Iniciando extração de '{endpoint_display_name}' da API Omie.")
print(f"URL Base: {base_url}")

todos_os_itens_lista = []
total_de_paginas_api = 1 
chave_lista_itens_api_identificada = None 

try:
    while pagina_atual_api <= total_de_paginas_api:
        # Corpo da requisição específico para ListarContratos
        body = {
            'call': api_call_name,
            'app_key': APP_KEY,
            'app_secret': APP_SECRET,
            'param': [
                {
                    "pagina": pagina_atual_api, 
                    "registros_por_pagina": registros_por_pagina_api,
                    "apenas_importado_api": "N"
                    # Adicione outros filtros se necessário, ex: "data_cadastro_de", "data_cadastro_ate", "status_contrato"
                }
            ]
        }

        tentativa_atual = 0
        sucesso_pagina = False
        dados_api_pagina = None 

        while tentativa_atual < max_tentativas_por_pagina and not sucesso_pagina:
            tentativa_atual += 1
            print(f"Buscando dados para '{endpoint_display_name}' (Página {pagina_atual_api} de {total_de_paginas_api if total_de_paginas_api > 1 else '...'}, Tentativa {tentativa_atual}/{max_tentativas_por_pagina})...")
            
            try:
                response = requests.post(base_url, headers=headers, data=json.dumps(body), timeout=60) 
                response.raise_for_status()
                
                dados_api_pagina = response.json()
                sucesso_pagina = True 

                # ListarContratos geralmente usa 'pagina', 'total_de_paginas', 'registros', 'total_de_registros'
                if pagina_atual_api == 1 and chave_lista_itens_api_identificada is None: 
                    total_de_paginas_api = dados_api_pagina.get('total_de_paginas', 1) 
                    total_de_registros_api = dados_api_pagina.get('total_de_registros', 0)
                    
                    print(f"API informou: {total_de_paginas_api} página(s) e {total_de_registros_api} registro(s) no total para '{endpoint_display_name}'.")
                    
                    if total_de_registros_api == 0:
                        print(f"Nenhum registro encontrado na API para '{endpoint_display_name}'.")
                        total_de_paginas_api = 0 
                        break

                    for key_attempt in json_data_keys_to_try:
                        if key_attempt in dados_api_pagina and isinstance(dados_api_pagina.get(key_attempt), list):
                            chave_lista_itens_api_identificada = key_attempt
                            print(f"Chave de dados JSON identificada para a lista de itens: '{chave_lista_itens_api_identificada}'")
                            break 
                    
                    if not chave_lista_itens_api_identificada:
                        print(f"ATENÇÃO: Não foi possível identificar a chave da lista de itens para '{endpoint_display_name}' na resposta da API.")
                        print("Resposta da API (primeira página):", json.dumps(dados_api_pagina, indent=2, ensure_ascii=False))
                        total_de_paginas_api = 0 
                        break 
                    
                    if total_de_registros_api == 0 and chave_lista_itens_api_identificada and not dados_api_pagina.get(chave_lista_itens_api_identificada):
                        print(f"Chave '{chave_lista_itens_api_identificada}' encontrada, mas está vazia. Nenhum registro.")
                        total_de_paginas_api = 0
                        break

                if sucesso_pagina and chave_lista_itens_api_identificada:
                    itens_da_pagina = dados_api_pagina.get(chave_lista_itens_api_identificada, [])
                    if itens_da_pagina:
                        todos_os_itens_lista.extend(itens_da_pagina)
                        print(f"  {len(itens_da_pagina)} itens adicionados. Total acumulado: {len(todos_os_itens_lista)}")
                    else:
                        print(f"  Nenhum item encontrado na página {pagina_atual_api} (usando chave '{chave_lista_itens_api_identificada}').")
                elif sucesso_pagina and not chave_lista_itens_api_identificada and total_de_registros_api > 0 and pagina_atual_api == 1 :
                    print(f"  Sucesso ao buscar dados, mas a chave da lista de itens não foi identificada. Verifique 'json_data_keys_to_try'.")
                    total_de_paginas_api = 0 
                    break

            except (requests.exceptions.ConnectionError, requests.exceptions.Timeout, requests.exceptions.HTTPError) as e:
                print(f"  Erro ao buscar dados (tentativa {tentativa_atual}): {e}")
                if tentativa_atual < max_tentativas_por_pagina:
                    delay_atual = delay_entre_tentativas_base * (2 ** (tentativa_atual - 1))
                    print(f"  Aguardando {delay_atual} segundos antes de tentar novamente...")
                    time.sleep(delay_atual)
                else:
                    print(f"  Máximo de {max_tentativas_por_pagina} tentativas atingido. Desistindo.")
                    break 

        if not sucesso_pagina and total_de_paginas_api > 0 : 
             print(f"AVISO: Falha ao processar a página {pagina_atual_api} após todas as tentativas.")
        
        if pagina_atual_api < total_de_paginas_api and total_de_paginas_api > 0:
            time.sleep(1) # Delay entre páginas bem-sucedidas
        
        if total_de_paginas_api == 0: 
            break
        
        pagina_atual_api += 1

    print(f"\nExtração para '{endpoint_display_name}' concluída (ou tentativas esgotadas).")

    if not todos_os_itens_lista:
        print(f"Nenhum item de '{endpoint_display_name}' foi extraído da API para processar.")
    else:
        print(f"Total de {len(todos_os_itens_lista)} registros de '{endpoint_display_name}' extraídos para processamento.")

        # Contratos de Serviço podem ter campos aninhados como 'cabecalho', 'informacoes_adicionais', 'itens_contrato'
        df_itens = pd.json_normalize(todos_os_itens_lista, errors='ignore') 
        
        print("\n--- DataFrame Inicial (após json_normalize) ---")
        if not df_itens.empty:
            print(df_itens.head())
        else:
            print("DataFrame inicial está vazio.")
        
        # Processamento adicional para listas aninhadas (ex: 'itensContrato')
        if 'itensContrato' in df_itens.columns and not df_itens.empty:
            try:
                df_itens['Contrato_Itens_Descricoes'] = df_itens['itensContrato'].apply(
                    lambda lista_itens: ', '.join(
                        [item.get('descricao', '') for item in lista_itens if isinstance(item, dict)]
                    ) if isinstance(lista_itens, list) else ''
                )
                df_itens['Contrato_Itens_Valores_Total'] = df_itens['itensContrato'].apply(
                    lambda lista_itens: sum(
                        [item.get('valor_total', item.get('valor_unitario',0) * item.get('quantidade',1)) for item in lista_itens if isinstance(item, dict) and isinstance(item.get('valor_total', item.get('valor_unitario')), (int, float))]
                    ) if isinstance(lista_itens, list) else 0.0
                )
            except Exception as e:
                print(f"Erro ao processar coluna 'itensContrato': {e}")
                if 'Contrato_Itens_Descricoes' not in df_itens.columns: df_itens['Contrato_Itens_Descricoes'] = ''
                if 'Contrato_Itens_Valores_Total' not in df_itens.columns: df_itens['Contrato_Itens_Valores_Total'] = 0.0


        df_processado = df_itens.replace('', pd.NA)
        df_processado = df_processado.fillna('N/A')

        print("\n--- DataFrame após preencher valores nulos/vazios ---")
        if not df_processado.empty:
            print(df_processado.head())
        else:
            print("DataFrame processado está vazio.")


        print("\n--- Colunas Disponíveis para Seleção/Renomeação ---")
        if not df_processado.empty:
            colunas_disponiveis = df_processado.columns.tolist()
            for col_name in colunas_disponiveis:
                print(f"- {col_name}")
        else:
            print("Nenhuma coluna disponível, DataFrame está vazio.")

        # --- AJUSTE AQUI: Defina as colunas que quer manter e os novos nomes para CONTRATOS DE SERVIÇO ---
        # Este é um EXEMPLO. Você PRECISARÁ ajustar com base na saída de "Colunas Disponíveis".
        # A estrutura do Contrato de Serviço é complexa.
        mapa_renomeacao_e_selecao = {
            'cabecalho.nCodCtr': 'ID Contrato Omie',
            'cabecalho.cNumCtr': 'Número Contrato',
            'cabecalho.dInicio': 'Data Início Contrato',
            'cabecalho.dVenc': 'Data Vencimento Contrato',
            'cabecalho.nCodCli': 'ID Cliente Contrato',
            'cabecalho.cCodCateg': 'Código Categoria Contrato',
            'cabecalho.cCodIntCtr': 'ID Integração Contrato',
            'cabecalho.cStatus': 'Status Contrato', # Ex: ATIVO, CANCELADO, CONCLUIDO
            'cabecalho.nValContrato': 'Valor Total Contrato',
            'informacoesAdicionais.nCodCC': 'ID Centro Custo Contrato',
            'informacoesAdicionais.cObsContrato': 'Observações Contrato',
            # Campos de info (geralmente inclusão/alteração do contrato)
            'infCadastro.dInc': 'Data Inclusão Contrato', # Verifique o nome exato, pode ser 'info.dInc'
            'infCadastro.hInc': 'Hora Inclusão Contrato',
            'infCadastro.uInc': 'Usuário Inclusão Contrato',
            'infCadastro.dAlt': 'Data Alteração Contrato',
            'infCadastro.hAlt': 'Hora Alteração Contrato',
            'infCadastro.uAlt': 'Usuário Alteração Contrato',
            # Novas colunas criadas do processamento de itensContrato
            'Contrato_Itens_Descricoes': 'Descrições Itens do Contrato',
            'Contrato_Itens_Valores_Total': 'Valor Total Itens do Contrato',
        }

        df_final = pd.DataFrame() 
        if not df_processado.empty:
            colunas_para_manter_renomeadas = {
                antigo: novo for antigo, novo in mapa_renomeacao_e_selecao.items() if antigo in df_processado.columns
            }
            colunas_antigas_selecionadas = list(colunas_para_manter_renomeadas.keys())

            if not colunas_antigas_selecionadas:
                print(f"\nAVISO: Nenhuma coluna do 'mapa_renomeacao_e_selecao' para '{endpoint_display_name}' foi encontrada no DataFrame.")
                df_final = df_processado.copy() 
            else:
                df_final = df_processado[colunas_antigas_selecionadas].rename(columns=colunas_para_manter_renomeadas)

                # --- AJUSTE AQUI: Defina a ordem desejada para as colunas de CONTRATOS DE SERVIÇO ---
                ordem_desejada = [
                    'ID Contrato Omie', 'Número Contrato', 'ID Cliente Contrato', 
                    'Data Início Contrato', 'Data Vencimento Contrato', 'Status Contrato', 
                    'Valor Total Contrato', 'Código Categoria Contrato', 'ID Centro Custo Contrato',
                    'Descrições Itens do Contrato', 'Valor Total Itens do Contrato',
                    'Observações Contrato',
                    'Data Inclusão Contrato', 'Hora Inclusão Contrato', 'Usuário Inclusão Contrato',
                    'Data Alteração Contrato', 'Hora Alteração Contrato', 'Usuário Alteração Contrato'
                ]
                ordem_existente = [col for col in ordem_desejada if col in df_final.columns]
                for col in df_final.columns: 
                    if col not in ordem_existente:
                        ordem_existente.append(col)
                if ordem_existente: 
                    df_final = df_final[ordem_existente]
        else:
            print("DataFrame processado está vazio, não é possível criar df_final com colunas selecionadas.")


        print("\n--- DataFrame Final (Limpado, Selecionado e Renomeado) ---")
        if not df_final.empty:
            print(df_final.head())
            print(f"\nTotal de {len(df_final)} itens de '{endpoint_display_name}' no DataFrame final.")

            df_final.to_csv(nome_arquivo_saida_csv, index=False, sep=';', encoding='utf-8-sig')
            print(f"\nDataFrame final salvo em: {nome_arquivo_saida_csv}")
        else:
            print("DataFrame final está vazio. Nenhum arquivo CSV será gerado.")

except requests.exceptions.HTTPError as http_err:
    print(f"Erro HTTP ocorreu durante a chamada à API para '{endpoint_display_name}': {http_err}")
    if 'response' in locals() and response is not None:
        print(f"Conteúdo da resposta: {response.text}")
except requests.exceptions.ConnectionError as conn_err: 
    print(f"Erro de Conexão ocorreu durante a chamada à API para '{endpoint_display_name}': {conn_err}")
except requests.exceptions.Timeout as timeout_err: 
    print(f"Timeout ocorreu durante a chamada à API para '{endpoint_display_name}': {timeout_err}")
except requests.exceptions.RequestException as req_err: 
    print(f"Erro de Requisição ocorreu durante a chamada à API para '{endpoint_display_name}': {req_err}")
except json.JSONDecodeError as json_err:
    print(f"Erro ao decodificar JSON da resposta da API para '{endpoint_display_name}': {json_err}")
    if 'response' in locals() and response is not None: 
        print(f"Conteúdo da resposta que causou o erro: {response.text}")
except Exception as e:
    print(f"Um erro inesperado ocorreu durante o processamento de '{endpoint_display_name}': {e}")

Iniciando extração de 'Contratos de Serviço' da API Omie.
URL Base: https://app.omie.com.br/api/v1/servicos/contrato/
Buscando dados para 'Contratos de Serviço' (Página 1 de ..., Tentativa 1/3)...
API informou: 6 página(s) e 299 registro(s) no total para 'Contratos de Serviço'.
Chave de dados JSON identificada para a lista de itens: 'contratoCadastro'
  50 itens adicionados. Total acumulado: 50
Buscando dados para 'Contratos de Serviço' (Página 2 de 6, Tentativa 1/3)...
  50 itens adicionados. Total acumulado: 100
Buscando dados para 'Contratos de Serviço' (Página 3 de 6, Tentativa 1/3)...
  50 itens adicionados. Total acumulado: 150
Buscando dados para 'Contratos de Serviço' (Página 4 de 6, Tentativa 1/3)...
  50 itens adicionados. Total acumulado: 200
Buscando dados para 'Contratos de Serviço' (Página 5 de 6, Tentativa 1/3)...
  50 itens adicionados. Total acumulado: 250
Buscando dados para 'Contratos de Serviço' (Página 6 de 6, Tentativa 1/3)...
  49 itens adicionados. Total acumula

In [29]:

APP_KEY = "596450736882"
APP_SECRET = "37a10edb420d8cd6c7afddcaaffcac05"

# --- CONFIGURAÇÃO PARA EXTRAÇÃO DE NFS-e (CONSULTAS) ---
endpoint_name = "NFS-e (Consultas)" # Nome da categoria para exibição e uso no DataFrame
base_module = "servicos"             # Módulo para NFS-e na Omie API, conforme sua URL
api_path = "nfse"                    # Parte final da URL do endpoint
call_method = "ListarNFSEs"          # Nome do método de chamada da API
# Chave que contém a lista de NFS-e na resposta JSON (pode variar, verificar docs Omie)
# Tentaremos 'nfseCadastro', 'listaNFSEs', 'notasServico', 'listaRegistros' ou 'nfseList'
json_data_keys_to_try = ["nfseCadastro", "listaNFSEs", "notasServico", "listaRegistros", "nfseList"]


base_url = f"https://app.omie.com.br/api/v1/{base_module}/{api_path}/" 

headers = {
    'Content-Type': 'application/json'
}

# Parâmetros da requisição para NFS-e (Consultas)
body = {
    "call": call_method,
    "app_key": APP_KEY,
    "app_secret": APP_SECRET,
    "param": [
        {
            "nPagina": 1,
            "nRegPorPagina": 20 # Você especificou 20 aqui
        }
    ]
}

print(f"Iniciando extração de dados para: {endpoint_name}")
print(f"URL da Requisição: {base_url}")
print(f"Método da Requisição (call): {call_method}")
print("-" * 50)

df_nfse_consultas = pd.DataFrame() # DataFrame vazio para armazenar os dados de NFS-e

try:
    response = requests.post(base_url, headers=headers, data=json.dumps(body))
    response.raise_for_status() # Lança uma exceção para códigos de status de erro (4xx ou 5xx)

    print(f"Dados de {endpoint_name} extraídos com sucesso!")
    
    data = response.json()

    # Tentar extrair a lista de NFS-e da resposta JSON usando as chaves prováveis
    found_key = None
    for key_attempt in json_data_keys_to_try:
        if key_attempt in data and isinstance(data.get(key_attempt), list):
            df_nfse_consultas = pd.DataFrame(data[key_attempt])
            found_key = key_attempt
            break # Sair do loop assim que a chave for encontrada

    if found_key:
        print(f"Chave de dados JSON identificada: '{found_key}'")
        print(f"\n--- Análise Exploratória para {endpoint_name} ---")
        print(f"Shape do DataFrame: {df_nfse_consultas.shape}")
        print("\nPrimeiras 5 linhas:")
        print(df_nfse_consultas.head())

        print("\nInformações do DataFrame (tipos de dados e contagem de não-nulos):")
        buffer = io.StringIO()
        df_nfse_consultas.info(buf=buffer, verbose=True, show_counts=True)
        print(buffer.getvalue())

        print("\nContagem de Valores Nulos por Coluna:")
        print(df_nfse_consultas.isnull().sum())
        print("-" * 50)
    else:
        print(f"Atenção: Não foi possível identificar a chave dos dados para '{endpoint_name}' na resposta da API.")
        print("Resposta completa da API (para depuração):", json.dumps(data, indent=2, ensure_ascii=False))
        print("-" * 50)

except requests.exceptions.HTTPError as http_err:
    print(f"Erro HTTP ao extrair {endpoint_name}: {http_err}")
    print(f"Resposta da API: {response.text}")
    print("-" * 50)
except requests.exceptions.ConnectionError as conn_err:
    print(f"Erro de Conexão ao extrair {endpoint_name}: {conn_err}")
    print("Verifique sua conexão com a internet ou o status da API Omie.")
    print("-" * 50)
except requests.exceptions.Timeout as timeout_err:
    print(f"Erro de Timeout ao extrair {endpoint_name}: {timeout_err}")
    print("A requisição demorou muito para responder.")
    print("-" * 50)
except requests.exceptions.RequestException as req_err:
    print(f"Erro inesperado ao extrair {endpoint_name}: {req_err}")
    print("-" * 50)
except json.JSONDecodeError:
    print(f"Erro ao decodificar JSON para {endpoint_name}. Resposta não é JSON válido.")
    print(f"Resposta da API: {response.text}")
    print("-" * 50)
except Exception as e:
    print(f"Ocorreu um erro geral ao processar {endpoint_name}: {e}")
    print("-" * 50)

print("\n--- Extração de NFS-e (Consultas) concluída! ---")

# Agora você tem o DataFrame 'df_nfse_consultas' com os dados extraídos.
# Você pode fazer mais análises ou salvá-lo:
# df_nfse_consultas.to_csv('nfse_consultas_omie.csv', index=False, encoding='utf-8')
# df_nfse_consultas.to_excel('nfse_consultas_omie.xlsx', index=False)

Iniciando extração de dados para: NFS-e (Consultas)
URL da Requisição: https://app.omie.com.br/api/v1/servicos/nfse/
Método da Requisição (call): ListarNFSEs
--------------------------------------------------
Dados de NFS-e (Consultas) extraídos com sucesso!
Atenção: Não foi possível identificar a chave dos dados para 'NFS-e (Consultas)' na resposta da API.
Resposta completa da API (para depuração): {
  "nPagina": 1,
  "nTotPaginas": 212,
  "nRegistros": 20,
  "nTotRegistros": 4229,
  "nfseEncontradas": [
    {
      "Adicionais": {
        "cCodigoCategoria": "1.01.02",
        "nCodigoCC": 679106865
      },
      "Alteracao": {
        "cDataAlteracao": "16/01/2019",
        "cHoraAlteracao": "10:31:20",
        "cUsuarioAlteracao": "P000102075"
      },
      "Cabecalho": {
        "cAmbienteNFSe": "P",
        "cCNPJDestinatario": "43.052.497/0001-02",
        "cCNPJEmissor": "42.161.372/0001-40",
        "cCidadeEmissor": "SAO PAULO (SP)",
        "cCodMunDestinatario": "3550308"

In [30]:

APP_KEY = "596450736882"
APP_SECRET = "37a10edb420d8cd6c7afddcaaffcac05"

# --- CONFIGURAÇÕES ESPECÍFICAS PARA O ENDPOINT DE NFS-e (CONSULTAS) ---
endpoint_display_name = "NFS-e (Consultas)" # Nome para exibição
base_module = "servicos"
api_path = "nfse" 
api_call_name = "ListarNFSEs"

# Chaves prováveis no JSON da API que contêm a lista de itens (NFS-e)
# O script tentará estas chaves em ordem.
json_data_keys_to_try = ["nfseCadastro", "listaNFSEs", "notasServico", "listaRegistros", "nfseList", "NFSe", "cadastro"] 

# Nome do arquivo de saída
nome_arquivo_saida_csv = 'omie_nfse_consultas_limpas_todas.csv'
nome_arquivo_saida_excel = 'omie_nfse_consultas_limpas_todas.xlsx'
# -----------------------------------------------------------------

versao = "v1" # Geralmente v1 para endpoints gerais da Omie
base_url = f"https://app.omie.com.br/api/{versao}/{base_module}/{api_path}/"

headers = {
    'Content-Type': 'application/json'
}

# Parâmetros iniciais
pagina_atual_api = 1 
registros_por_pagina_api = 20 # Conforme seu exemplo para NFS-e
max_tentativas_por_pagina = 3
delay_entre_tentativas_base = 5

print(f"Iniciando extração de '{endpoint_display_name}' da API Omie.")
print(f"URL Base: {base_url}")

todos_os_itens_lista = []
total_de_paginas_api = 1 
chave_lista_itens_api_identificada = None 

try:
    while pagina_atual_api <= total_de_paginas_api:
        # Corpo da requisição específico para ListarNFSEs
        body = {
            'call': api_call_name,
            'app_key': APP_KEY,
            'app_secret': APP_SECRET,
            'param': [
                {
                    "nPagina": pagina_atual_api, # Omie usa nPagina para este endpoint
                    "nRegPorPagina": registros_por_pagina_api # Omie usa nRegPorPagina
                    # Adicione outros filtros se necessário, ex: "dEmiInicial", "dEmiFinal", "cTomadorCNPJ"
                }
            ]
        }

        tentativa_atual = 0
        sucesso_pagina = False
        dados_api_pagina = None 

        while tentativa_atual < max_tentativas_por_pagina and not sucesso_pagina:
            tentativa_atual += 1
            print(f"Buscando dados para '{endpoint_display_name}' (Página {pagina_atual_api} de {total_de_paginas_api if total_de_paginas_api > 1 else '...'}, Tentativa {tentativa_atual}/{max_tentativas_por_pagina})...")
            
            try:
                response = requests.post(base_url, headers=headers, data=json.dumps(body), timeout=60) 
                response.raise_for_status()
                
                dados_api_pagina = response.json()
                sucesso_pagina = True 

                # ListarNFSEs geralmente usa nPagina, nTotalPaginas, nRegistros, nTotalRegistros
                if pagina_atual_api == 1 and chave_lista_itens_api_identificada is None: 
                    total_de_paginas_api = dados_api_pagina.get('nTotalPaginas', dados_api_pagina.get('total_de_paginas', 1))
                    total_de_registros_api = dados_api_pagina.get('nTotalRegistros', dados_api_pagina.get('total_de_registros', 0))
                    
                    print(f"API informou: {total_de_paginas_api} página(s) e {total_de_registros_api} registro(s) no total para '{endpoint_display_name}'.")
                    
                    if total_de_registros_api == 0:
                        print(f"Nenhum registro encontrado na API para '{endpoint_display_name}'.")
                        total_de_paginas_api = 0 
                        break

                    for key_attempt in json_data_keys_to_try:
                        if key_attempt in dados_api_pagina and isinstance(dados_api_pagina.get(key_attempt), list):
                            chave_lista_itens_api_identificada = key_attempt
                            print(f"Chave de dados JSON identificada para a lista de itens: '{chave_lista_itens_api_identificada}'")
                            break 
                    
                    if not chave_lista_itens_api_identificada:
                        print(f"ATENÇÃO: Não foi possível identificar a chave da lista de itens para '{endpoint_display_name}' na resposta da API.")
                        print("Resposta da API (primeira página):", json.dumps(dados_api_pagina, indent=2, ensure_ascii=False))
                        total_de_paginas_api = 0 
                        break 
                    
                    if total_de_registros_api == 0 and chave_lista_itens_api_identificada and not dados_api_pagina.get(chave_lista_itens_api_identificada):
                        print(f"Chave '{chave_lista_itens_api_identificada}' encontrada, mas está vazia. Nenhum registro.")
                        total_de_paginas_api = 0
                        break

                if sucesso_pagina and chave_lista_itens_api_identificada:
                    itens_da_pagina = dados_api_pagina.get(chave_lista_itens_api_identificada, [])
                    if itens_da_pagina:
                        todos_os_itens_lista.extend(itens_da_pagina)
                        print(f"  {len(itens_da_pagina)} itens adicionados. Total acumulado: {len(todos_os_itens_lista)}")
                    else:
                        print(f"  Nenhum item encontrado na página {pagina_atual_api} (usando chave '{chave_lista_itens_api_identificada}').")
                elif sucesso_pagina and not chave_lista_itens_api_identificada and total_de_registros_api > 0 and pagina_atual_api == 1 :
                    print(f"  Sucesso ao buscar dados, mas a chave da lista de itens não foi identificada. Verifique 'json_data_keys_to_try'.")
                    total_de_paginas_api = 0 
                    break

            except (requests.exceptions.ConnectionError, requests.exceptions.Timeout, requests.exceptions.HTTPError) as e:
                print(f"  Erro ao buscar dados (tentativa {tentativa_atual}): {e}")
                if tentativa_atual < max_tentativas_por_pagina:
                    delay_atual = delay_entre_tentativas_base * (2 ** (tentativa_atual - 1))
                    print(f"  Aguardando {delay_atual} segundos antes de tentar novamente...")
                    time.sleep(delay_atual)
                else:
                    print(f"  Máximo de {max_tentativas_por_pagina} tentativas atingido. Desistindo.")
                    break 

        if not sucesso_pagina and total_de_paginas_api > 0 : 
             print(f"AVISO: Falha ao processar a página {pagina_atual_api} após todas as tentativas.")
        
        if pagina_atual_api < total_de_paginas_api and total_de_paginas_api > 0:
            time.sleep(1) # Delay entre páginas bem-sucedidas
        
        if total_de_paginas_api == 0: 
            break
        
        pagina_atual_api += 1

    print(f"\nExtração para '{endpoint_display_name}' concluída (ou tentativas esgotadas).")

    if not todos_os_itens_lista:
        print(f"Nenhum item de '{endpoint_display_name}' foi extraído da API para processar.")
    else:
        print(f"Total de {len(todos_os_itens_lista)} registros de '{endpoint_display_name}' extraídos para processamento.")

        # NFS-e pode ter estruturas muito aninhadas (ide, prestador, tomador, servico, impostos, etc.)
        df_itens = pd.json_normalize(todos_os_itens_lista, errors='ignore') 
        
        print("\n--- DataFrame Inicial (após json_normalize) ---")
        if not df_itens.empty:
            print(df_itens.head())
        else:
            print("DataFrame inicial está vazio.")
        
        # Processamento adicional para listas aninhadas (ex: itens de serviço, se houver uma lista)
        # A estrutura de NFS-e pode ter um 'servico' (um dicionário) ou 'listaServico' (uma lista de dicionários).
        # Se for uma lista e você quiser processá-la:
        # if 'listaServico' in df_itens.columns and not df_itens.empty: # Exemplo, a coluna pode se chamar 'servicos' ou 'itens'
        #     try:
        #         df_itens['NFSe_Servicos_Descricoes'] = df_itens['listaServico'].apply(
        #             lambda lista_srv: ', '.join(
        #                 [srv.get('discriminacao', srv.get('descricao', '')) for srv in lista_srv if isinstance(srv, dict)] # Tenta 'discriminacao' ou 'descricao'
        #             ) if isinstance(lista_srv, list) else ''
        #         )
        #         df_itens['NFSe_Servicos_Valores_Total'] = df_itens['listaServico'].apply(
        #             lambda lista_srv: sum(
        #                 [srv.get('valorServicos', srv.get('valor', 0)) for srv in lista_srv if isinstance(srv, dict) and isinstance(srv.get('valorServicos', srv.get('valor')), (int, float))]
        #             ) if isinstance(lista_srv, list) else 0.0
        #         )
        #     except Exception as e:
        #         print(f"Erro ao processar lista de serviços da NFS-e: {e}")
        #         if 'NFSe_Servicos_Descricoes' not in df_itens.columns: df_itens['NFSe_Servicos_Descricoes'] = ''
        #         if 'NFSe_Servicos_Valores_Total' not in df_itens.columns: df_itens['NFSe_Servicos_Valores_Total'] = 0.0


        df_processado = df_itens.replace('', pd.NA)
        df_processado = df_processado.fillna('N/A')

        print("\n--- DataFrame após preencher valores nulos/vazios ---")
        if not df_processado.empty:
            print(df_processado.head())
        else:
            print("DataFrame processado está vazio.")


        print("\n--- Colunas Disponíveis para Seleção/Renomeação ---")
        if not df_processado.empty:
            colunas_disponiveis = df_processado.columns.tolist()
            for col_name in colunas_disponiveis:
                print(f"- {col_name}")
        else:
            print("Nenhuma coluna disponível, DataFrame está vazio.")

        # --- AJUSTE AQUI: Defina as colunas que quer manter e os novos nomes para NFS-e (CONSULTAS) ---
        # Este é um EXEMPLO. Você PRECISARÁ ajustar com base na saída de "Colunas Disponíveis".
        # A estrutura da NFS-e é complexa.
        mapa_renomeacao_e_selecao = {
            # Campos da Identificação da NFS-e (geralmente em 'ide' ou no nível raiz)
            'ide.nNFSe': 'Número NFS-e',
            'ide.dEmi': 'Data Emissão NFS-e',
            'ide.cVerif': 'Código Verificação NFS-e',
            'ide.cStatus': 'Status NFS-e', # Ex: AUTORIZADA, CANCELADA
            'ide.xMotivo': 'Motivo Status NFS-e',
            'ide.cMunGer': 'Cód. Município Gerador',
            'ide.nIdNFSe': 'ID NFS-e Omie', # Se houver um ID interno da Omie para a NFSe

            # Campos do Prestador (geralmente em 'prestador')
            'prestador.cnpj': 'CNPJ Prestador',
            'prestador.razaoSocial': 'Razão Social Prestador',
            'prestador.inscricaoMunicipal': 'IM Prestador',

            # Campos do Tomador (geralmente em 'tomador')
            'tomador.cnpj': 'CNPJ Tomador', # Ou 'tomador.cpf'
            'tomador.razaoSocial': 'Razão Social Tomador',
            'tomador.cMun': 'Cód. Município Tomador',
            'tomador.email': 'E-mail Tomador',

            # Campos do Serviço (geralmente em 'servico', pode ser uma lista se houver 'listaServico')
            # Se for um único serviço por NFS-e, os campos estarão como 'servico.discriminacao'.
            # Se for uma lista, você precisaria de um tratamento específico como o comentado acima,
            # ou json_normalize criaria colunas como 'listaServico.0.discriminacao'.
            'servico.discriminacao': 'Discriminação Serviço',
            'servico.valorServicos': 'Valor Total Serviços',
            'servico.valorDeducoes': 'Valor Deduções',
            'servico.valorPis': 'Valor PIS',
            'servico.valorCofins': 'Valor COFINS',
            'servico.valorIr': 'Valor IR',
            'servico.valorCsll': 'Valor CSLL',
            'servico.valorIss': 'Valor ISS',
            'servico.valorIssRetido': 'Valor ISS Retido',
            'servico.aliquota': 'Alíquota ISS (%)',
            'servico.itemListaServico': 'Cód. Item Lista Serviço (LC116)',
            'servico.cSitTrib': 'Situação Tributária ISS',
            
            # Informações de Inclusão/Alteração (se houver, pode estar em 'info', 'infCadastro', ou similar)
            'infCadastro.dInc': 'Data Inclusão Registro', 
            'infCadastro.hInc': 'Hora Inclusão Registro',
            'infCadastro.uInc': 'Usuário Inclusão Registro',
            'infCadastro.dAlt': 'Data Alteração Registro',
            'infCadastro.hAlt': 'Hora Alteração Registro',
            'infCadastro.uAlt': 'Usuário Alteração Registro',

            # Outros campos úteis
            'nIdOs': 'ID Ordem de Serviço Origem', # Se a NFS-e for originada de uma OS
            'nIdContrato': 'ID Contrato Origem',   # Se a NFS-e for originada de um Contrato
        }

        df_final = pd.DataFrame() 
        if not df_processado.empty:
            colunas_para_manter_renomeadas = {
                antigo: novo for antigo, novo in mapa_renomeacao_e_selecao.items() if antigo in df_processado.columns
            }
            colunas_antigas_selecionadas = list(colunas_para_manter_renomeadas.keys())

            if not colunas_antigas_selecionadas:
                print(f"\nAVISO: Nenhuma coluna do 'mapa_renomeacao_e_selecao' para '{endpoint_display_name}' foi encontrada no DataFrame.")
                df_final = df_processado.copy() 
            else:
                df_final = df_processado[colunas_antigas_selecionadas].rename(columns=colunas_para_manter_renomeadas)

                # --- AJUSTE AQUI: Defina a ordem desejada para as colunas de NFS-e ---
                ordem_desejada = [
                    'ID NFS-e Omie', 'Número NFS-e', 'Data Emissão NFS-e', 'Status NFS-e', 'Código Verificação NFS-e',
                    'CNPJ Prestador', 'Razão Social Prestador', 'IM Prestador',
                    'CNPJ Tomador', 'Razão Social Tomador', 'Cód. Município Tomador', 'E-mail Tomador',
                    'Discriminação Serviço', 'Valor Total Serviços', 'Valor Deduções',
                    'Valor PIS', 'Valor COFINS', 'Valor IR', 'Valor CSLL', 
                    'Valor ISS', 'Valor ISS Retido', 'Alíquota ISS (%)', 'Cód. Item Lista Serviço (LC116)',
                    'Situação Tributária ISS', 'Motivo Status NFS-e', 'Cód. Município Gerador',
                    'ID Ordem de Serviço Origem', 'ID Contrato Origem',
                    'Data Inclusão Registro', 'Hora Inclusão Registro', 'Usuário Inclusão Registro',
                    'Data Alteração Registro', 'Hora Alteração Registro', 'Usuário Alteração Registro'
                ]
                ordem_existente = [col for col in ordem_desejada if col in df_final.columns]
                for col in df_final.columns: 
                    if col not in ordem_existente:
                        ordem_existente.append(col)
                if ordem_existente: 
                    df_final = df_final[ordem_existente]
        else:
            print("DataFrame processado está vazio, não é possível criar df_final com colunas selecionadas.")


        print("\n--- DataFrame Final (Limpado, Selecionado e Renomeado) ---")
        if not df_final.empty:
            print(df_final.head())
            print(f"\nTotal de {len(df_final)} itens de '{endpoint_display_name}' no DataFrame final.")

            df_final.to_csv(nome_arquivo_saida_csv, index=False, sep=';', encoding='utf-8-sig')
            print(f"\nDataFrame final salvo em: {nome_arquivo_saida_csv}")
        else:
            print("DataFrame final está vazio. Nenhum arquivo CSV será gerado.")

except requests.exceptions.HTTPError as http_err:
    print(f"Erro HTTP ocorreu durante a chamada à API para '{endpoint_display_name}': {http_err}")
    if 'response' in locals() and response is not None:
        print(f"Conteúdo da resposta: {response.text}")
except requests.exceptions.ConnectionError as conn_err: 
    print(f"Erro de Conexão ocorreu durante a chamada à API para '{endpoint_display_name}': {conn_err}")
except requests.exceptions.Timeout as timeout_err: 
    print(f"Timeout ocorreu durante a chamada à API para '{endpoint_display_name}': {timeout_err}")
except requests.exceptions.RequestException as req_err: 
    print(f"Erro de Requisição ocorreu durante a chamada à API para '{endpoint_display_name}': {req_err}")
except json.JSONDecodeError as json_err:
    print(f"Erro ao decodificar JSON da resposta da API para '{endpoint_display_name}': {json_err}")
    if 'response' in locals() and response is not None: 
        print(f"Conteúdo da resposta que causou o erro: {response.text}")
except Exception as e:
    print(f"Um erro inesperado ocorreu durante o processamento de '{endpoint_display_name}': {e}")

Iniciando extração de 'NFS-e (Consultas)' da API Omie.
URL Base: https://app.omie.com.br/api/v1/servicos/nfse/
Buscando dados para 'NFS-e (Consultas)' (Página 1 de ..., Tentativa 1/3)...
API informou: 1 página(s) e 0 registro(s) no total para 'NFS-e (Consultas)'.
Nenhum registro encontrado na API para 'NFS-e (Consultas)'.

Extração para 'NFS-e (Consultas)' concluída (ou tentativas esgotadas).
Nenhum item de 'NFS-e (Consultas)' foi extraído da API para processar.


In [31]:

# Suas chaves e definições da API
APP_KEY = "596450736882"
APP_SECRET = "37a10edb420d8cd6c7afddcaaffcac05"

# --- CONFIGURAÇÕES ESPECÍFICAS PARA O ENDPOINT DE LANÇAMENTOS DE CONTA CORRENTE ---
endpoint_display_name = "Lançamentos de Conta Corrente" # Nome para exibição
base_module = "financas"
api_path = "contacorrentelancamentos" 
api_call_name = "ListarLancCC"

# Chaves prováveis no JSON da API que contêm a lista de itens (Lançamentos CC)
# O script tentará estas chaves em ordem.
json_data_keys_to_try = ["lancamentos", "listaLancamentos", "lancamentosCc", "movimentacoes", "listaRegistros", "cadastro"] 

# Nome do arquivo de saída
nome_arquivo_saida_csv = 'omie_lancamentos_cc_limpos_todos.csv'
nome_arquivo_saida_excel = 'omie_lancamentos_cc_limpos_todos.xlsx'
# -----------------------------------------------------------------

versao = "v1" # Geralmente v1 para endpoints gerais da Omie
base_url = f"https://app.omie.com.br/api/{versao}/{base_module}/{api_path}/"

headers = {
    'Content-Type': 'application/json'
}

# Parâmetros iniciais
pagina_atual_api = 1 
registros_por_pagina_api = 20 # Conforme seu exemplo para Lançamentos CC
max_tentativas_por_pagina = 3
delay_entre_tentativas_base = 5

print(f"Iniciando extração de '{endpoint_display_name}' da API Omie.")
print(f"URL Base: {base_url}")

todos_os_itens_lista = []
total_de_paginas_api = 1 
chave_lista_itens_api_identificada = None 

try:
    while pagina_atual_api <= total_de_paginas_api:
        # Corpo da requisição específico para ListarLancCC
        body = {
            'call': api_call_name,
            'app_key': APP_KEY,
            'app_secret': APP_SECRET,
            'param': [
                {
                    "nPagina": pagina_atual_api, 
                    "nRegPorPagina": registros_por_pagina_api
                    # Adicione outros filtros se necessário, ex: "dDtInicial", "dDtFinal", "nCodCC" (código da conta corrente)
                }
            ]
        }

        tentativa_atual = 0
        sucesso_pagina = False
        dados_api_pagina = None 

        while tentativa_atual < max_tentativas_por_pagina and not sucesso_pagina:
            tentativa_atual += 1
            print(f"Buscando dados para '{endpoint_display_name}' (Página {pagina_atual_api} de {total_de_paginas_api if total_de_paginas_api > 1 else '...'}, Tentativa {tentativa_atual}/{max_tentativas_por_pagina})...")
            
            try:
                response = requests.post(base_url, headers=headers, data=json.dumps(body), timeout=60) 
                response.raise_for_status()
                
                dados_api_pagina = response.json()
                sucesso_pagina = True 

                # ListarLancCC geralmente usa nPagina, nTotalPaginas, nRegistros, nTotalRegistros
                if pagina_atual_api == 1 and chave_lista_itens_api_identificada is None: 
                    total_de_paginas_api = dados_api_pagina.get('nTotalPaginas', dados_api_pagina.get('total_de_paginas', 1))
                    total_de_registros_api = dados_api_pagina.get('nTotalRegistros', dados_api_pagina.get('total_de_registros', 0))
                    
                    print(f"API informou: {total_de_paginas_api} página(s) e {total_de_registros_api} registro(s) no total para '{endpoint_display_name}'.")
                    
                    if total_de_registros_api == 0:
                        print(f"Nenhum registro encontrado na API para '{endpoint_display_name}'.")
                        total_de_paginas_api = 0 
                        break

                    for key_attempt in json_data_keys_to_try:
                        if key_attempt in dados_api_pagina and isinstance(dados_api_pagina.get(key_attempt), list):
                            chave_lista_itens_api_identificada = key_attempt
                            print(f"Chave de dados JSON identificada para a lista de itens: '{chave_lista_itens_api_identificada}'")
                            break 
                    
                    if not chave_lista_itens_api_identificada:
                        print(f"ATENÇÃO: Não foi possível identificar a chave da lista de itens para '{endpoint_display_name}' na resposta da API.")
                        print("Resposta da API (primeira página):", json.dumps(dados_api_pagina, indent=2, ensure_ascii=False))
                        total_de_paginas_api = 0 
                        break 
                    
                    if total_de_registros_api == 0 and chave_lista_itens_api_identificada and not dados_api_pagina.get(chave_lista_itens_api_identificada):
                        print(f"Chave '{chave_lista_itens_api_identificada}' encontrada, mas está vazia. Nenhum registro.")
                        total_de_paginas_api = 0
                        break

                if sucesso_pagina and chave_lista_itens_api_identificada:
                    itens_da_pagina = dados_api_pagina.get(chave_lista_itens_api_identificada, [])
                    if itens_da_pagina:
                        todos_os_itens_lista.extend(itens_da_pagina)
                        print(f"  {len(itens_da_pagina)} itens adicionados. Total acumulado: {len(todos_os_itens_lista)}")
                    else:
                        print(f"  Nenhum item encontrado na página {pagina_atual_api} (usando chave '{chave_lista_itens_api_identificada}').")
                elif sucesso_pagina and not chave_lista_itens_api_identificada and total_de_registros_api > 0 and pagina_atual_api == 1 :
                    print(f"  Sucesso ao buscar dados, mas a chave da lista de itens não foi identificada. Verifique 'json_data_keys_to_try'.")
                    total_de_paginas_api = 0 
                    break

            except (requests.exceptions.ConnectionError, requests.exceptions.Timeout, requests.exceptions.HTTPError) as e:
                print(f"  Erro ao buscar dados (tentativa {tentativa_atual}): {e}")
                if tentativa_atual < max_tentativas_por_pagina:
                    delay_atual = delay_entre_tentativas_base * (2 ** (tentativa_atual - 1))
                    print(f"  Aguardando {delay_atual} segundos antes de tentar novamente...")
                    time.sleep(delay_atual)
                else:
                    print(f"  Máximo de {max_tentativas_por_pagina} tentativas atingido. Desistindo.")
                    break 

        if not sucesso_pagina and total_de_paginas_api > 0 : 
             print(f"AVISO: Falha ao processar a página {pagina_atual_api} após todas as tentativas.")
        
        if pagina_atual_api < total_de_paginas_api and total_de_paginas_api > 0:
            time.sleep(1) # Delay entre páginas bem-sucedidas
        
        if total_de_paginas_api == 0: 
            break
        
        pagina_atual_api += 1

    print(f"\nExtração para '{endpoint_display_name}' concluída (ou tentativas esgotadas).")

    if not todos_os_itens_lista:
        print(f"Nenhum item de '{endpoint_display_name}' foi extraído da API para processar.")
    else:
        print(f"Total de {len(todos_os_itens_lista)} registros de '{endpoint_display_name}' extraídos para processamento.")

        df_itens = pd.json_normalize(todos_os_itens_lista, errors='ignore') 
        
        print("\n--- DataFrame Inicial (após json_normalize) ---")
        if not df_itens.empty:
            print(df_itens.head())
        else:
            print("DataFrame inicial está vazio.")
        
        # Lançamentos de CC podem ter campos como 'detalhes', 'rateios', 'impostos' que são aninhados.
        # O json_normalize já deve ter achatado a maioria deles.
        # Se houver uma lista de rateios, por exemplo, em uma coluna 'rateio',
        # você pode precisar de um processamento específico:
        # if 'rateio' in df_itens.columns and not df_itens.empty:
        #     try:
        #         df_itens['LancCC_Rateio_Categorias'] = df_itens['rateio'].apply(
        #             lambda lista_rateios: ', '.join(
        #                 [rateio.get('cCodCategoria', '') for rateio in lista_rateios if isinstance(rateio, dict)]
        #             ) if isinstance(lista_rateios, list) else ''
        #         )
        #     except Exception as e:
        #         print(f"Erro ao processar coluna 'rateio': {e}")
        #         if 'LancCC_Rateio_Categorias' not in df_itens.columns: df_itens['LancCC_Rateio_Categorias'] = ''

        df_processado = df_itens.replace('', pd.NA)
        df_processado = df_processado.fillna('N/A')

        print("\n--- DataFrame após preencher valores nulos/vazios ---")
        if not df_processado.empty:
            print(df_processado.head())
        else:
            print("DataFrame processado está vazio.")


        print("\n--- Colunas Disponíveis para Seleção/Renomeação ---")
        if not df_processado.empty:
            colunas_disponiveis = df_processado.columns.tolist()
            for col_name in colunas_disponiveis:
                print(f"- {col_name}")
        else:
            print("Nenhuma coluna disponível, DataFrame está vazio.")

        # --- AJUSTE AQUI: Defina as colunas que quer manter e os novos nomes para LANÇAMENTOS DE CONTA CORRENTE ---
        # Este é um EXEMPLO. Você PRECISARÁ ajustar com base na saída de "Colunas Disponíveis".
        mapa_renomeacao_e_selecao = {
            'nCodLanc': 'ID Lançamento CC Omie',
            'cCodIntLanc': 'ID Integração Lançamento CC',
            'nCodCC': 'ID Conta Corrente',
            'cGrupo': 'Grupo Lançamento (PAG, REC, TRF)',
            'cOperacao': 'Tipo Operação', # Ex: TED, DOC, PIX, CHQ
            'dDtLanc': 'Data Lançamento',
            'dDtComp': 'Data Competência', # Se houver
            'dDtCred': 'Data Crédito/Débito Efetivo',
            'nValor': 'Valor Lançamento',
            'cCodCliente': 'ID Cliente/Fornecedor', # Se aplicável
            'cNumDoc': 'Número Documento Origem',
            'cObs': 'Observação',
            'cStatus': 'Status Lançamento', # Ex: CONCILIADO, PENDENTE
            'cNatureza': 'Natureza', # Se houver
            'cCodCateg': 'Código Categoria Principal',
            # 'LancCC_Rateio_Categorias': 'Rateio - Categorias', # Exemplo de coluna processada
            # Campos de informações de criação/alteração, se existirem e forem achatados
            # 'info.dInc': 'Data Inclusão Lançamento',
            # 'info.hInc': 'Hora Inclusão Lançamento',
        }

        df_final = pd.DataFrame() 
        if not df_processado.empty:
            colunas_para_manter_renomeadas = {
                antigo: novo for antigo, novo in mapa_renomeacao_e_selecao.items() if antigo in df_processado.columns
            }
            colunas_antigas_selecionadas = list(colunas_para_manter_renomeadas.keys())

            if not colunas_antigas_selecionadas:
                print(f"\nAVISO: Nenhuma coluna do 'mapa_renomeacao_e_selecao' para '{endpoint_display_name}' foi encontrada no DataFrame.")
                df_final = df_processado.copy() 
            else:
                df_final = df_processado[colunas_antigas_selecionadas].rename(columns=colunas_para_manter_renomeadas)

                # --- AJUSTE AQUI: Defina a ordem desejada para as colunas de Lançamentos de Conta Corrente ---
                ordem_desejada = [
                    'ID Lançamento CC Omie', 'ID Integração Lançamento CC', 'ID Conta Corrente',
                    'Grupo Lançamento (PAG, REC, TRF)', 'Tipo Operação', 
                    'Data Lançamento', 'Data Competência', 'Data Crédito/Débito Efetivo',
                    'Valor Lançamento', 'ID Cliente/Fornecedor', 'Número Documento Origem',
                    'Observação', 'Status Lançamento', 'Natureza', 'Código Categoria Principal',
                    # 'Rateio - Categorias',
                    # 'Data Inclusão Lançamento', 'Hora Inclusão Lançamento'
                ]
                ordem_existente = [col for col in ordem_desejada if col in df_final.columns]
                for col in df_final.columns: 
                    if col not in ordem_existente:
                        ordem_existente.append(col)
                if ordem_existente: 
                    df_final = df_final[ordem_existente]
        else:
            print("DataFrame processado está vazio, não é possível criar df_final com colunas selecionadas.")


        print("\n--- DataFrame Final (Limpado, Selecionado e Renomeado) ---")
        if not df_final.empty:
            print(df_final.head())
            print(f"\nTotal de {len(df_final)} itens de '{endpoint_display_name}' no DataFrame final.")

            df_final.to_csv(nome_arquivo_saida_csv, index=False, sep=';', encoding='utf-8-sig')
            print(f"\nDataFrame final salvo em: {nome_arquivo_saida_csv}")
        else:
            print("DataFrame final está vazio. Nenhum arquivo CSV será gerado.")

except requests.exceptions.HTTPError as http_err:
    print(f"Erro HTTP ocorreu durante a chamada à API para '{endpoint_display_name}': {http_err}")
    if 'response' in locals() and response is not None:
        print(f"Conteúdo da resposta: {response.text}")
except requests.exceptions.ConnectionError as conn_err: 
    print(f"Erro de Conexão ocorreu durante a chamada à API para '{endpoint_display_name}': {conn_err}")
except requests.exceptions.Timeout as timeout_err: 
    print(f"Timeout ocorreu durante a chamada à API para '{endpoint_display_name}': {timeout_err}")
except requests.exceptions.RequestException as req_err: 
    print(f"Erro de Requisição ocorreu durante a chamada à API para '{endpoint_display_name}': {req_err}")
except json.JSONDecodeError as json_err:
    print(f"Erro ao decodificar JSON da resposta da API para '{endpoint_display_name}': {json_err}")
    if 'response' in locals() and response is not None: 
        print(f"Conteúdo da resposta que causou o erro: {response.text}")
except Exception as e:
    print(f"Um erro inesperado ocorreu durante o processamento de '{endpoint_display_name}': {e}")

Iniciando extração de 'Lançamentos de Conta Corrente' da API Omie.
URL Base: https://app.omie.com.br/api/v1/financas/contacorrentelancamentos/
Buscando dados para 'Lançamentos de Conta Corrente' (Página 1 de ..., Tentativa 1/3)...
API informou: 1 página(s) e 0 registro(s) no total para 'Lançamentos de Conta Corrente'.
Nenhum registro encontrado na API para 'Lançamentos de Conta Corrente'.

Extração para 'Lançamentos de Conta Corrente' concluída (ou tentativas esgotadas).
Nenhum item de 'Lançamentos de Conta Corrente' foi extraído da API para processar.


In [32]:

arquivos_csv = [
    'omie_clientes_limpos_todos.csv',
    'omie_contas_a_pagar_limpas_todas.csv',
    'omie_contas_correntes_limpas_todas.csv',
    'omie_contratos_servico_limpos_todos.csv',
    'omie_departamentos_limpos_todos.csv',
    'omie_nfe_consultas_limpas_todas.csv',
    'omie_ordens_servico_limpas_todas.csv',
    'omie_projetos_limpos_todos.csv',
    # Adicione aqui qualquer outro arquivo CSV que você tenha gerado
    'omie_plano_contas_dre_limpos_todos.csv',
    'omie_servicos_limpos_todos.csv'
]

print("--- Análise das Colunas dos Arquivos CSV Gerados ---")

# Dicionário para armazenar as colunas de cada arquivo
colunas_por_arquivo = {}

for arquivo in arquivos_csv:
    print(f"\n---------------------------------------------")
    print(f"Analisando arquivo: {arquivo}")
    print(f"---------------------------------------------")
    try:
        # Tenta ler o arquivo CSV. O separador é ';' conforme gerado anteriormente.
        # nrows=5 lê apenas as primeiras 5 linhas, o que é suficiente e rápido para pegar os cabeçalhos.
        df = pd.read_csv(arquivo, sep=';', nrows=5) 
        
        print("Colunas encontradas:")
        colunas = df.columns.tolist()
        colunas_por_arquivo[arquivo] = colunas
        for i, coluna in enumerate(colunas):
            print(f"  {i+1:02d}. {coluna}")

    except FileNotFoundError:
        print(f"ATENÇÃO: Arquivo '{arquivo}' não encontrado. Verifique se o nome está correto e se o arquivo está no mesmo diretório do script.")
    except Exception as e:
        print(f"Ocorreu um erro ao ler o arquivo '{arquivo}': {e}")

print("\n\n--- Análise Concluída ---")
print("\n>>> PRÓXIMO PASSO: Copie toda esta saída (a lista de arquivos e suas respectivas colunas, desde '--- Análise das Colunas...' até aqui) e cole na nossa conversa.")
print("Com essa informação, poderemos identificar as colunas em comum para fazer o merge.")

--- Análise das Colunas dos Arquivos CSV Gerados ---

---------------------------------------------
Analisando arquivo: omie_clientes_limpos_todos.csv
---------------------------------------------
Colunas encontradas:
  01. ID Cliente Omie
  02. CNPJ/CPF
  03. Razão Social
  04. Nome Fantasia
  05. E-mail Principal
  06. DDD Tel Principal
  07. Número Tel Principal
  08. Endereço Principal
  09. Número End.
  10. Bairro
  11. Cidade
  12. UF
  13. CEP
  14. Data Inclusão
  15. Hora Inclusão
  16. Data Alteração
  17. Hora Alteração
  18. Tags
  19. Inativo (S/N)
  20. Cód. Banco
  21. Gera Boletos (S/N)

---------------------------------------------
Analisando arquivo: omie_contas_a_pagar_limpas_todas.csv
---------------------------------------------
Colunas encontradas:
  01. ID Lançamento Omie
  02. ID Cliente/Fornecedor
  03. NF
  04. Data Emissão NF
  05. Data Vencimento
  06. Valor Documento
  07. Status Título
  08. Código Categoria Principal
  09. Códigos Categorias (Todas)
  10

In [33]:
print("--- Iniciando a Unificação (Merge) dos Arquivos CSV ---")

# Dicionário para carregar os DataFrames
dataframes = {}

# Lista de arquivos a serem carregados e suas respectivas chaves de cliente
# A chave do lado esquerdo é a do arquivo, a do lado direito é a do arquivo de clientes.
arquivos_para_merge = {
    'clientes': {
        'arquivo': 'omie_clientes_limpos_todos.csv',
        'chave': 'ID Cliente Omie' # Esta é a chave principal
    },
    'contas_a_pagar': {
        'arquivo': 'omie_contas_a_pagar_limpas_todas.csv',
        'chave': 'ID Cliente/Fornecedor'
    },
    'contratos': {
        'arquivo': 'omie_contratos_servico_limpos_todos.csv',
        'chave': 'ID Cliente Contrato'
    },
    'ordens_servico': {
        'arquivo': 'omie_ordens_servico_limpas_todas.csv',
        'chave': 'ID Cliente OS'
    }
}

# Carregar os arquivos em DataFrames
for nome_df, info in arquivos_para_merge.items():
    arquivo_path = info['arquivo']
    try:
        print(f"Lendo o arquivo: {arquivo_path}")
        # Especificamos o separador e o tipo da chave como string para evitar problemas no merge
        chave = info['chave']
        dataframes[nome_df] = pd.read_csv(arquivo_path, sep=';', dtype={chave: str})
        print(f"  -> Sucesso! {len(dataframes[nome_df])} linhas carregadas.")
    except FileNotFoundError:
        print(f"ATENÇÃO: Arquivo '{arquivo_path}' não encontrado. Ele será ignorado no merge.")
    except Exception as e:
        print(f"Ocorreu um erro ao ler o arquivo '{arquivo_path}': {e}")

# Verifica se o DataFrame base de clientes foi carregado
if 'clientes' not in dataframes:
    print("\nERRO CRÍTICO: O arquivo base 'omie_clientes_limpos_todos.csv' não foi encontrado. O merge não pode continuar.")
else:
    # Inicia o DataFrame consolidado com a base de clientes
    df_consolidado = dataframes['clientes']
    print(f"\nIniciando merge a partir de 'clientes' com {len(df_consolidado)} registros.")

    # Faz o merge dos outros DataFrames um a um
    for nome_df, info in arquivos_para_merge.items():
        if nome_df == 'clientes':
            continue # Pula o próprio DataFrame de clientes

        if nome_df in dataframes:
            print(f"Fazendo merge com '{info['arquivo']}'...")
            
            df_para_merge = dataframes[nome_df]
            chave_esquerda = info['chave']
            chave_direita = arquivos_para_merge['clientes']['chave'] # Chave do df_clientes

            # Renomeia a chave do DataFrame secundário para corresponder à chave do df_clientes,
            # caso os nomes sejam diferentes. Isso evita colunas duplicadas de ID.
            if chave_esquerda != chave_direita:
                df_para_merge = df_para_merge.rename(columns={chave_esquerda: chave_direita})

            # Usa 'left' join para manter todos os clientes, mesmo que não tenham correspondência no outro arquivo.
            # O sufixo é adicionado para colunas com o mesmo nome (exceto a chave do merge)
            df_consolidado = pd.merge(
                df_consolidado, 
                df_para_merge, 
                on=chave_direita, 
                how='left',
                suffixes=('', f'_{nome_df}') # Deixa a coluna do cliente com nome original e adiciona sufixo para a do outro df
            )
            print(f"  -> Merge concluído. DataFrame consolidado agora tem {len(df_consolidado)} linhas e {len(df_consolidado.columns)} colunas.")

    # --- Análise e Salvamento do Resultado Final ---
    if not df_consolidado.empty:
        print("\n--- Merge Finalizado com Sucesso! ---")
        print("\nPrimeiras 5 linhas do DataFrame consolidado:")
        print(df_consolidado.head())

        print("\nColunas do DataFrame Consolidado:")
        for i, coluna in enumerate(df_consolidado.columns):
            print(f"  {i+1:02d}. {coluna}")

        # Salvar o DataFrame consolidado em um novo arquivo CSV
        nome_arquivo_saida = 'relatorio_consolidado.csv'
        try:
            df_consolidado.to_csv(nome_arquivo_saida, index=False, sep=';', encoding='utf-8-sig')
            print(f"\nArquivo consolidado '{nome_arquivo_saida}' salvo com sucesso!")
        except Exception as e:
            print(f"\nOcorreu um erro ao salvar o arquivo consolidado: {e}")
    else:
        print("\nO DataFrame consolidado está vazio. Verifique os logs de erro.")

print("\n--- Processo de Unificação Concluído ---")

--- Iniciando a Unificação (Merge) dos Arquivos CSV ---
Lendo o arquivo: omie_clientes_limpos_todos.csv
  -> Sucesso! 2996 linhas carregadas.
Lendo o arquivo: omie_contas_a_pagar_limpas_todas.csv
  -> Sucesso! 43834 linhas carregadas.
Lendo o arquivo: omie_contratos_servico_limpos_todos.csv
  -> Sucesso! 299 linhas carregadas.
Lendo o arquivo: omie_ordens_servico_limpas_todas.csv
  -> Sucesso! 4263 linhas carregadas.

Iniciando merge a partir de 'clientes' com 2996 registros.
Fazendo merge com 'omie_contas_a_pagar_limpas_todas.csv'...
  -> Merge concluído. DataFrame consolidado agora tem 44756 linhas e 31 colunas.
Fazendo merge com 'omie_contratos_servico_limpos_todos.csv'...
  -> Merge concluído. DataFrame consolidado agora tem 45632 linhas e 36 colunas.
Fazendo merge com 'omie_ordens_servico_limpas_todas.csv'...
  -> Merge concluído. DataFrame consolidado agora tem 851888 linhas e 44 colunas.

--- Merge Finalizado com Sucesso! ---

Primeiras 5 linhas do DataFrame consolidado:
  ID Cl